In [2]:
import numpy as np
import os
import pandas as pd
from arcpy import env
from arcpy import TableToTable_conversion
import matplotlib.ticker as mtick

In [3]:
dirWork = os.getcwd()
dirData = os.path.join(dirWork, r'data')
dirTDM  = os.path.join(dirData, r'tdm')
dirFactors   = os.path.join(dirWork, r'data-processed\factors')
dirProcessed = os.path.join(dirWork, r'data-processed\tdm')
dirResults   = os.path.join(dirWork, r'results')

# Run Settings

In [4]:
#set to true for first time to create CSVs, false to speed up for subsequent times
process_TDMdbftocsv = False

#export results
export_results = True

# Define Scenarios

In [5]:
df_Scenarios = pd.DataFrame()


#define AdjCode for pivot when model is not related to forecast
#AADT will be adjusted based on difference between scenario and forecast scenario TDMs
#if ForecastScenarioID is note defined, then the associated forecast volume will be used for that year
#DBF don't include filename extension

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'Base19',
                                    'ScenarioName'   : '2019 Base',
                                    'ScenarioYear'   :  2019,
                                    'ScenarioGroup'  : 'Base',
                                    'SegSummaryDBF'  : 'v831_SE19_Net19_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'TIP24',
                                    'ScenarioName'   : '2024 TIP',
                                    'ScenarioYear'   :  2024,
                                    'ScenarioGroup'  : 'TIP',
                                    'SegSummaryDBF'  : 'v831_SE24_Net24_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'Base19'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'RTP30',
                                    'ScenarioName'   : '2030 RTP',
                                    'ScenarioYear'   :  2030,
                                    'ScenarioGroup'  : 'RTP',
                                    'SegSummaryDBF'  : 'v831_SE30_Net30_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'TIP24'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'RTP40',
                                    'ScenarioName'   : '2040 RTP',
                                    'ScenarioYear'   :  2040,
                                    'ScenarioGroup'  : 'RTP',
                                    'SegSummaryDBF'  : 'v831_SE40_Net40_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'RTP30'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode' : 'RTP50',
                                    'ScenarioName' : '2050 RTP',
                                    'ScenarioYear' :  2050,
                                    'ScenarioGroup': 'RTP',
                                    'SegSummaryDBF': 'v831_SE50_Net50_Summary_SEGID_Detailed_BH',
                                    'Process'      : 'Yes',
                                    'PrvScenarioCode': 'RTP40'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB30',
                                    'ScenarioName'   : '2030 No Build',
                                    'ScenarioYear'   :  2030,
                                    'ScenarioGroup'  : 'No Build',
                                    'AdjScenarioCode': 'RTP30',
                                    'SegSummaryDBF'  : 'v831_SE30_Net24_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'TIP24'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB40',
                                    'ScenarioName'   : '2040 No Build',
                                    'ScenarioYear'   :  2040,
                                    'ScenarioGroup'  : 'No Build',
                                    'AdjScenarioCode': 'RTP40',
                                    'SegSummaryDBF'  : 'v831_SE40_Net24_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'NB30'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB50',
                                    'ScenarioName'   : '2050 No Build',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'No Build',
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryDBF'  : 'v831_SE50_Net24_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'NB40'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF30',
                                    'ScenarioName'   : '2030 Needs',
                                    'ScenarioYear'   :  2030,
                                    'ScenarioGroup'  : 'Needs',
                                    'AdjScenarioCode': 'RTP30',
                                    'SegSummaryDBF'  : 'v831_SE30_Net30UF_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'TIP24'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF40',
                                    'ScenarioName'   : '2040 Needs',
                                    'ScenarioYear'   :  2040,
                                    'ScenarioGroup'  : 'Needs',
                                    'AdjScenarioCode': 'RTP40',
                                    'SegSummaryDBF'  : 'v831_SE40_Net40UF_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'UF30'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF50',
                                    'ScenarioName'   : '2050 Needs',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'Needs',
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryDBF'  : 'v831_SE50_Net50UF_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'UF40'
                                   },ignore_index=True)

#all scenarios will be read in to allow for pivoting if needed
df_Scenarios.index.name = 'ScenarioID'
df_Scenarios['ScenarioYear'] = pd.to_numeric(df_Scenarios['ScenarioYear'], downcast='integer')
df_Scenarios

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryDBF,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Yes,Base19,Base,2019 Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed_BH,NaN,NaN
1,Yes,TIP24,TIP,2024 TIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed_BH,Base19,NaN
2,Yes,RTP30,RTP,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed_BH,TIP24,NaN
3,Yes,RTP40,RTP,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed_BH,RTP30,NaN
4,Yes,RTP50,RTP,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed_BH,RTP40,NaN
5,Yes,NB30,No Build,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,TIP24,RTP30
6,Yes,NB40,No Build,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,NB30,RTP40
7,Yes,NB50,No Build,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,NB40,RTP50
8,Yes,UF30,Needs,2030 Needs,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,TIP24,RTP30


In [7]:
df_Scenarios_toProcess = df_Scenarios[df_Scenarios['Process']=='Yes']
df_Scenarios_toProcess

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryDBF,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Yes,Base19,Base,2019 Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed_BH,NaN,NaN
1,Yes,TIP24,TIP,2024 TIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed_BH,Base19,NaN
2,Yes,RTP30,RTP,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed_BH,TIP24,NaN
3,Yes,RTP40,RTP,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed_BH,RTP30,NaN
4,Yes,RTP50,RTP,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed_BH,RTP40,NaN
5,Yes,NB30,No Build,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,TIP24,RTP30
6,Yes,NB40,No Build,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,NB30,RTP40
7,Yes,NB50,No Build,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,NB40,RTP50
8,Yes,UF30,Needs,2030 Needs,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,TIP24,RTP30


In [8]:
df_Scenarios_toAdjust = df_Scenarios_toProcess[df_Scenarios_toProcess['AdjScenarioCode'].notnull()]
df_Scenarios_toAdjust

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryDBF,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
5,Yes,NB30,No Build,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,TIP24,RTP30
6,Yes,NB40,No Build,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,NB30,RTP40
7,Yes,NB50,No Build,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,NB40,RTP50
8,Yes,UF30,Needs,2030 Needs,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,TIP24,RTP30
9,Yes,UF40,Needs,2040 Needs,2040,v831_SE40_Net40UF_Summary_SEGID_Detailed_BH,UF30,RTP40
10,Yes,UF50,Needs,2050 Needs,2050,v831_SE50_Net50UF_Summary_SEGID_Detailed_BH,UF40,RTP50


In [9]:
#define functions to get scenario data
def idFromCode (sCode):
    return df_Scenarios.index[df_Scenarios['ScenarioCode'] == sCode].tolist()[0]
def yrFromCode (sCode):
    return df_Scenarios[df_Scenarios['ScenarioCode'] == sCode].iloc[0]['ScenarioYear']
def codeFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioCode']
def yearFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioYear']

#quick tests of functions
#display(idFromCode('50RTP'))
#display(yrFromCode('50RTP'))
#display(codeFromID(4))
#display(yearFromID(4))
#display(df_Scenarios.loc[7]['ScenarioCode'])

# Define Desired V/C Groupings

In [10]:
#season groups

df_VCGroup_Seasons = pd.DataFrame()

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Avg',
                                                'SeasonName'    : 'Average',
                                                'SeasonGroup'   : 'S00-Avg',
                                                'SeasonType'    : 'Season'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Win',
                                                'SeasonName'    : 'Winter',
                                                'SeasonGroup'   : 'S01-Win',
                                                'SeasonType'    : 'Season'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Spr',
                                                'SeasonName'    : 'Spring',
                                                'SeasonGroup'   : 'S02-Spr',
                                                'SeasonType'    : 'Season'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Sum',
                                                'SeasonName'    : 'Summer',
                                                'SeasonGroup'   : 'S03-Sum',
                                                'SeasonType'    : 'Season'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Fal',
                                                'SeasonName'    : 'Fall',
                                                'SeasonGroup'   : 'S04-Fal',
                                                'SeasonType'    : 'Season'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jan',
                                                'SeasonName'    : 'January',
                                                'SeasonGroup'   : 'M01-Jan',
                                                'SeasonType'    : 'Month',
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Feb',
                                                'SeasonName'    : 'February',
                                                'SeasonGroup'   : 'M02-Feb',
                                                'SeasonType'    : 'Month'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Mar',
                                                'SeasonName'    : 'March',
                                                'SeasonGroup'   : 'M03-Mar',
                                                'SeasonType'    : 'Month'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Apr',
                                                'SeasonName'    : 'April',
                                                'SeasonGroup'   : 'M04-Apr',
                                                'SeasonType'    : 'Month'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'May',
                                                'SeasonName'    : 'May',
                                                'SeasonGroup'   : 'M05-May',
                                                'SeasonType'    : 'Month'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jun',
                                                'SeasonName'    : 'June',
                                                'SeasonGroup'   : 'M06-Jun',
                                                'SeasonType'    : 'Month'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jul',
                                                'SeasonName'    : 'July',
                                                'SeasonGroup'   : 'M07-Jul',
                                                'SeasonType'    : 'Month'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Aug',
                                                'SeasonName'    : 'August',
                                                'SeasonGroup'   : 'M08-Aug',
                                                'SeasonType'    : 'Month'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Sep',
                                                'SeasonName'    : 'September',
                                                'SeasonGroup'   : 'M09-Sep',
                                                'SeasonType'    : 'Month'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Oct',
                                                'SeasonName'    : 'October',
                                                'SeasonGroup'   : 'M10-Oct',
                                                'SeasonType'    : 'Month'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Nov',
                                                'SeasonName'    : 'November',
                                                'SeasonGroup'   : 'M11-Nov',
                                                'SeasonType'    : 'Month'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Dec',
                                                'SeasonName'    : 'December',
                                                'SeasonGroup'   : 'M12-Dec',
                                                'SeasonType'    : 'Month'
                                               },ignore_index=True)

display(df_VCGroup_Seasons)

#day-of-week groups

df_VCGroup_DOWPeak = pd.DataFrame()

df_VCGroup_DOWPeak = df_VCGroup_DOWPeak.append({'DOWPkCode'     : 'WkAM',
                                                'DOWPkName'     : 'AM Peak - Weekday',
                                                'DOWGroup'      : 'W1-Weekday',
                                                'Period'        : 'AM',
                                                'HourPercentOf' : 'Period' #calculate peak hour using period volume
                                               },ignore_index=True)

df_VCGroup_DOWPeak = df_VCGroup_DOWPeak.append({'DOWPkCode'     : 'WkMD',
                                                'DOWPkName'     : 'Midday Peak - Weekday',
                                                'DOWGroup'      : 'W1-Weekday',
                                                'Period'        : 'MD',
                                                'HourPercentOf' : 'Period' #calculate peak hour using period volume
                                               },ignore_index=True)

df_VCGroup_DOWPeak = df_VCGroup_DOWPeak.append({'DOWPkCode'     : 'WkPM',
                                                'DOWPkName'     : 'PM Peak - Weekday',
                                                'DOWGroup'      : 'W1-Weekday',
                                                'Period'        : 'PM',
                                                'HourPercentOf' : 'Period' #calculate peak hour using period volume
                                               },ignore_index=True)

df_VCGroup_DOWPeak = df_VCGroup_DOWPeak.append({'DOWPkCode'     : 'FrPM',
                                                'DOWPkName'     : 'PM Peak - Friday',
                                                'DOWGroup'      : '5-Friday',
                                                'Period'        : 'PM',
                                                'HourPercentOf' : 'Daily' #calculate peak hour using daily volume
                                               },ignore_index=True)

df_VCGroup_DOWPeak = df_VCGroup_DOWPeak.append({'DOWPkCode'     : 'SaDY',
                                                'DOWPkName'     : 'Saturday Peak',
                                                'DOWGroup'      : '6-Saturday',
                                                'Period'        : 'Daily',
                                                'HourPercentOf' : 'Daily' #calculate peak hour using daily volume
                                               },ignore_index=True)

df_VCGroup_DOWPeak = df_VCGroup_DOWPeak.append({'DOWPkCode'     : 'SuDY',
                                                'DOWPkName'     : 'Sunday Peak',
                                                'DOWGroup'      : '7-Sunday',
                                                'Period'        : 'Daily',
                                                'HourPercentOf' : 'Daily' #calculate peak hour using daily volume
                                               },ignore_index=True)

display(df_VCGroup_DOWPeak)

#directional factor when using daily volumes for peak hour calcs
#taken from HCM 2000 Exhibit 9-2
dDailyDirectionalFactor = 0.6

#directional factor not used in period volume calcs, since those include directionality

,SeasonCode,SeasonGroup,SeasonName,SeasonType
0,Avg,S00-Avg,Average,Season
1,Win,S01-Win,Winter,Season
2,Spr,S02-Spr,Spring,Season
3,Sum,S03-Sum,Summer,Season
4,Fal,S04-Fal,Fall,Season
5,Jan,M01-Jan,January,Month
6,Feb,M02-Feb,February,Month
7,Mar,M03-Mar,March,Month
8,Apr,M04-Apr,April,Month
9,May,M05-May,May,Month


,DOWGroup,DOWPkCode,DOWPkName,HourPercentOf,Period
0,W1-Weekday,WkAM,AM Peak - Weekday,Period,AM
1,W1-Weekday,WkMD,Midday Peak - Weekday,Period,MD
2,W1-Weekday,WkPM,PM Peak - Weekday,Period,PM
3,5-Friday,FrPM,PM Peak - Friday,Daily,PM
4,6-Saturday,SaDY,Saturday Peak,Daily,Daily
5,7-Sunday,SuDY,Sunday Peak,Daily,Daily


In [11]:
import itertools

#create df with all possible season and day-of-weak/peak groups
df_VCGroups = pd.DataFrame(list(itertools.product(df_VCGroup_Seasons['SeasonCode'],df_VCGroup_DOWPeak['DOWPkCode'])),columns=['SeasonCode','DOWPkCode'])

df_VCGroups = pd.DataFrame(df_VCGroups)
pd.set_option('display.max_rows', df_VCGroups.shape[0]+1)

#display(df_VCGroups)

#add season data
df_VCGroups = pd.DataFrame.merge(df_VCGroups, df_VCGroup_Seasons, on='SeasonCode')

#add dow-peak data
df_VCGroups = pd.DataFrame.merge(df_VCGroups, df_VCGroup_DOWPeak, on='DOWPkCode')

#set id name
df_VCGroups.index.name = 'VCGroupID'

df_VCGroups['VCGroupCode'] = df_VCGroups['SeasonCode'] + df_VCGroups['DOWPkCode']

df_VCGroups = df_VCGroups.drop(columns=['SeasonCode','DOWPkCode','SeasonName','DOWPkName'])

display(df_VCGroups)

,SeasonGroup,SeasonType,DOWGroup,HourPercentOf,Period,VCGroupCode
VCGroupID,,,,,,
0,S00-Avg,Season,W1-Weekday,Period,AM,AvgWkAM
1,S01-Win,Season,W1-Weekday,Period,AM,WinWkAM
2,S02-Spr,Season,W1-Weekday,Period,AM,SprWkAM
3,S03-Sum,Season,W1-Weekday,Period,AM,SumWkAM
4,S04-Fal,Season,W1-Weekday,Period,AM,FalWkAM
5,M01-Jan,Month,W1-Weekday,Period,AM,JanWkAM
6,M02-Feb,Month,W1-Weekday,Period,AM,FebWkAM
7,M03-Mar,Month,W1-Weekday,Period,AM,MarWkAM
8,M04-Apr,Month,W1-Weekday,Period,AM,AprWkAM


# Convert DBFs to CSVs

In [20]:
# Create function that converts dbfs to csvs using arcpy
def dbfToCSV(dbf, csv):
    TableToTable_conversion(dbf, os.path.dirname(csv), os.path.basename(csv))

# Allow arcpy to overwrite output files
env.overwriteOutput = True

#convert DBFs to CSVs
if process_TDMdbftocsv:
    
    for (idx, row) in df_Scenarios.iterrows():    

        print(row.loc[nameS] + ' TDM DBF converting to CSV')

        dbfFileName = os.path.join(dirTDM, row.SegSummaryDBF + ".dbf")
        csvFileName = os.path.join(dirProcessed, row.SegSummaryDBF + ".csv")

        dbfToCSV(dbfFileName, csvFileName)
        
        print(row.loc[nameS] + ' TDM done.')
else:
    print("Don't convert TDM Segment Summaries DBFs.")

print('Done')
        

Don't convert TDM Segment Summaries DBFs.
Done


# TDM SEGID Detail

In [48]:
#column names from TDM
nameSegID   = 'SEGID'
nameFG      = 'FUNCGROUP'
nameLinks   = 'LINKS'
nameLanes   = 'LANES'
nameVol     = 'VOL'
nameCap1HL  = 'CAP1HL'
nameAT      = 'AREATYPE'
nameFT      = 'FT'

aNameDirPeriodVol = ['D1_AM_VOL','D1_PM_VOL','D1_MD_VOL','D1_EV_VOL','D1_AM_LT','D1_MD_LT','D1_PM_LT','D1_EV_LT','D1_AM_MD','D1_MD_MD','D1_PM_MD','D1_EV_MD','D1_AM_HV','D1_MD_HV','D1_PM_HV','D1_EV_HV','D2_AM_VOL','D2_PM_VOL','D2_MD_VOL','D2_EV_VOL','D2_AM_LT','D2_MD_LT','D2_PM_LT','D2_EV_LT','D2_AM_MD','D2_MD_MD','D2_PM_MD','D2_EV_MD','D2_AM_HV','D2_MD_HV','D2_PM_HV','D2_EV_HV']

#value for functional group (FG) total row, so that it can be removed
nameFGTotalValue = 'Total'

#column for scenario year
nameSID = 'ScenarioID'
nameS =   'ScenarioName'
nameSYr = 'ScenarioYear'

#additional column name variables
nameLT = 'LT'
nameMD = 'MD'
nameHV = 'HV'

nameVolType = 'VolType'
namePeriod  = 'Period'
nameCapPer  = 'CapPer'
nameCap1Hr  = 'Cap1Hr'
nameDir     = 'Dir'

nameSegTotal        = 'SegTotalDYVol'
namePercentSegTotal = 'VolPercentSegDY'
namePercentTrucksMD = 'PercentTrucksMD'
namePercentTrucksHV = 'PercentTrucksHV'

indexDirPeriodwSegData = [nameSegID,nameFG,nameCap1HL,nameLinks,nameLanes,nameDir,namePeriod,nameAT,nameFT]
indexDirPeriod         = [nameSegID,nameDir,namePeriod]
indexDir               = [nameSegID,nameDir,nameAT]
indexTotal             = [nameSegID]
indexMelt              = [nameSegID,nameFG,nameAT,nameFT,'D1_' + nameLinks,'D1_' + nameLanes,'D1_' + nameCap1HL,'D2_' + nameLinks,'D2_' + nameLanes,'D2_' + nameCap1HL]
indexDaily             = [nameSegID,nameFG,nameCap1HL,nameLinks,nameLanes,nameAT,nameFT]

#hours within period, for capacity
dPeriodDuration = {
     'Period'     : ['AM','MD','PM','EV','Daily'],
     'PeriodHours': [   3,   6,   3,  12,     24]
     
}
df_PeriodDuration = pd.DataFrame(dPeriodDuration)

In [22]:
dfs_TDM_import = []
dfs_TDM =[]
dfs_TDM_Total = []

#all scenarios will be read in to allow for pivoting

for (idx, row) in df_Scenarios.iterrows():

    print(row.loc[nameS] + ' CSV Importing')

    csvFileName = os.path.join(dirProcessed, row.SegSummaryDBF + ".csv")
    #csvFileName = os.path.splitext(dbfFileName)[0]+'.csv'

    df_TDM_import = pd.read_csv(csvFileName)

    #Remove where exclude not 0
    df_TDM_import = df_TDM_import[df_TDM_import.LINKS > 0]
    df_TDM_import = df_TDM_import[df_TDM_import.FUNCGROUP != nameFGTotalValue]
    dfs_TDM_import.append(df_TDM_import)
        
    print(row.loc[nameS] + ' Processing.')

    #normalize data with period/volume type fields
    df_TDM_melt = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNameDirPeriodVol)

    #Calculate new columns for period, volume type
    df_TDM_melt[nameDir]     = df_TDM_melt['variable'].str.get(0) + df_TDM_melt['variable'].str.get(1)
    df_TDM_melt[namePeriod]  = df_TDM_melt['variable'].str.get(3) + df_TDM_melt['variable'].str.get(4)
    df_TDM_melt[nameVolType] = df_TDM_melt['variable'].str[6:]
    
    #calculate direction level links, lanes, capacity
    
    df_TDM_melt[nameLinks] = np.where(df_TDM_melt[nameDir].eq('D1'), df_TDM_melt['D1_'+nameLinks], df_TDM_melt['D2_'+nameLinks])
    df_TDM_melt[nameLanes] = np.where(df_TDM_melt[nameDir].eq('D1'), df_TDM_melt['D1_'+nameLanes], df_TDM_melt['D2_'+nameLanes])
    df_TDM_melt[nameCap1HL] = np.where(df_TDM_melt[nameDir].eq('D1'), df_TDM_melt['D1_'+nameCap1HL], df_TDM_melt['D2_'+nameCap1HL])

    df_TDM_melt = df_TDM_melt.drop(columns=['D1_'+nameLinks,'D2_'+nameLinks,'D1_'+nameLanes,'D2_'+nameLanes,'D1_'+nameCap1HL,'D2_'+nameCap1HL])
    
    #display(df_TDM_melt[(df_TDM_melt[nameSegID]=='0015_295.6')])
    
    df_TDM_byDirPeriod = pd.pivot_table(df_TDM_melt,
        values='value',
        index=indexDirPeriodwSegData,
        columns=[nameVolType]
    )
    
    df_TDM_byDirPeriod.reset_index(level=df_TDM_byDirPeriod.index.names, inplace=True)
    #display(df_TDM_byDirPeriod)
    #display(df_TDM_byDirPeriod[(df_TDM_byDirPeriod[nameSegID]=='0015_295.6')])
    
    df_TDM_Total         = df_TDM_byDirPeriod.groupby(indexTotal).agg({nameVol: [np.sum]})
    df_TDM_Total.columns = [nameSegTotal]
    dfs_TDM_Total.append(df_TDM_Total) #add to collection for volume adjusting
    
    #display(df_TDM_Total)
    #display(df_TDM_Total[(df_TDM_Total.index=='0015_295.6')])

    #display(df_TDM_byDirPeriod)
    #display(df_TDM_byDirPeriod[(df_TDM_byDirPeriod[nameSegID]=='0015_295.6')])
    
    #calculate daily directional for peak-hour-percent-of-daily calcs
    df_TDM_Daily             = df_TDM_byDirPeriod.groupby(indexDaily, as_index=False).agg({nameVol: [np.sum], nameLT: [np.sum], nameMD: [np.sum], nameHV: [np.sum]})
    df_TDM_Daily.columns     = df_TDM_Daily.columns.droplevel(1)
    df_TDM_Daily[namePeriod] = 'Daily'
    
    df_TDM_Daily_D1          = df_TDM_Daily.copy()
    df_TDM_Daily_D2          = df_TDM_Daily.copy()
    
    df_TDM_Daily_D1[nameDir] = 'D1'
    df_TDM_Daily_D2[nameDir] = 'D2'
    
    df_TDM_Daily_D1[nameVol] = df_TDM_Daily[nameVol] * dDailyDirectionalFactor
    df_TDM_Daily_D2[nameVol] = df_TDM_Daily[nameVol] * (1-dDailyDirectionalFactor)

    df_TDM_Daily_D1[nameLT]  = df_TDM_Daily[nameLT] * dDailyDirectionalFactor
    df_TDM_Daily_D2[nameLT]  = df_TDM_Daily[nameLT] * (1-dDailyDirectionalFactor)
    
    df_TDM_Daily_D1[nameMD]  = df_TDM_Daily[nameMD] * dDailyDirectionalFactor
    df_TDM_Daily_D2[nameMD]  = df_TDM_Daily[nameMD] * (1-dDailyDirectionalFactor)
    
    df_TDM_Daily_D1[nameHV]  = df_TDM_Daily[nameHV] * dDailyDirectionalFactor
    df_TDM_Daily_D2[nameHV]  = df_TDM_Daily[nameHV] * (1-dDailyDirectionalFactor)
    
    #display(df_TDM_Daily)
    #display(df_TDM_Daily_D1)
    #display(df_TDM_Daily_D2)
    
    df_TDM_byDirPeriod = df_TDM_byDirPeriod.append(df_TDM_Daily_D1)
    df_TDM_byDirPeriod = df_TDM_byDirPeriod.append(df_TDM_Daily_D2)
    
    #display(df_TDM_byDirPeriod)
    #display(df_TDM_byDirPeriod[(df_TDM_byDirPeriod[nameSegID]=='0015_295.6')])
    #display(df_TDM_Total[(df_TDM_Total.index=='0015_295.6')])
    
    df_TDM_byDirPeriodwTotal = pd.DataFrame.merge(df_TDM_byDirPeriod,df_TDM_Total, on=nameSegID)
    #display(df_TDM_byDirPeriodwTotal)
    #display(df_TDM_byDirPeriodwTotal[(df_TDM_byDirPeriodwTotal[nameSegID]=='0015_295.6')])
    
    #calculate percent trucks
    df_TDM_byDirPeriodwTotal[namePercentTrucksMD] = df_TDM_byDirPeriodwTotal[nameMD] / df_TDM_byDirPeriodwTotal[nameVol]
    df_TDM_byDirPeriodwTotal[namePercentTrucksHV] = df_TDM_byDirPeriodwTotal[nameHV] / df_TDM_byDirPeriodwTotal[nameVol]
    
    #calculate period volume as percent of segment total
    df_TDM_byDirPeriodwTotal[namePercentSegTotal] = df_TDM_byDirPeriodwTotal[nameVol] / df_TDM_byDirPeriodwTotal[nameSegTotal]
 
    #display(df_TDM_byDirPeriodwTotal)
    #display(df_TDM_byDirPeriodwTotal[(df_TDM_byDirPeriodwTotal[nameSegID]=='0015_295.6')]) 

    #calculate capacities
    df_TDM_byDirPeriodwTotal[nameCap1Hr] = df_TDM_byDirPeriodwTotal[nameCap1HL] * df_TDM_byDirPeriodwTotal[nameLanes]
    
    #limit columns in dataframe
    df_TDM_byDirPeriodwTotal = df_TDM_byDirPeriodwTotal[[nameSegID,nameFG,nameAT,nameFT,nameDir,namePeriod,nameVol,namePercentTrucksMD,namePercentTrucksHV,namePercentSegTotal,nameSegTotal,nameLanes,nameCap1Hr]]
    #display(df_TDM_byDirPeriodwTotal)
    #display(df_TDM_byDirPeriodwTotal[(df_TDM_byDirPeriodwTotal[nameSegID]=='0015_295.6')])
    
    dfs_TDM.append(df_TDM_byDirPeriodwTotal)
    
    #The sum of all volume percent of segment daily should be 1.0, add little extra on each side for rounding
    df_Check = df_TDM_byDirPeriodwTotal.groupby(indexTotal).agg({namePercentSegTotal:[np.sum]})
    df_Check.columns = df_Check.columns.droplevel(1)
    df_Check = df_Check[(df_Check[namePercentSegTotal] > 1.0000000001) | (df_Check[namePercentSegTotal] < 0.99999999)]
    #display(df_Check)

print('Done')

2019 Base CSV Importing
2019 Base Processing.
2024 TIP CSV Importing
2024 TIP Processing.
2030 RTP CSV Importing
2030 RTP Processing.
2040 RTP CSV Importing
2040 RTP Processing.
2050 RTP CSV Importing
2050 RTP Processing.
2030 No Build CSV Importing
2030 No Build Processing.
2040 No Build CSV Importing
2040 No Build Processing.
2050 No Build CSV Importing
2050 No Build Processing.
2030 Needs CSV Importing
2030 Needs Processing.
2040 Needs CSV Importing
2040 Needs Processing.
2050 Needs CSV Importing
2050 Needs Processing.
Done


In [23]:
dfs_TDM[0]

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr
0,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,1392.4,1.0,1065.0
1,0006_146.9,Arterial,1.0,2.0,D1,EV,166.40,0.088341,0.000000,0.119506,1392.4,1.0,1065.0
2,0006_146.9,Arterial,1.0,2.0,D1,MD,222.60,0.092992,0.069182,0.159868,1392.4,1.0,1065.0
3,0006_146.9,Arterial,1.0,2.0,D1,PM,179.60,0.067929,0.051782,0.128986,1392.4,1.0,1065.0
4,0006_146.9,Arterial,1.0,2.0,D2,AM,127.20,0.063679,0.044811,0.091353,1392.4,1.0,1065.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37817,WFRC_8218,CD Road,3.0,31.0,D2,EV,3624.60,0.138277,0.000000,0.126594,28631.8,1.0,1473.0
37818,WFRC_8218,CD Road,3.0,31.0,D2,MD,4648.90,0.145282,0.074491,0.162368,28631.8,1.0,1473.0
37819,WFRC_8218,CD Road,3.0,31.0,D2,PM,3819.30,0.114157,0.058649,0.133394,28631.8,1.0,1473.0
37820,WFRC_8218,CD Road,3.0,31.0,D1,Daily,17179.08,0.129960,0.049752,0.600000,28631.8,1.0,1473.0


In [24]:
df_Scenarios_toProcess

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryDBF,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Yes,Base19,Base,2019 Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed_BH,NaN,NaN
1,Yes,TIP24,TIP,2024 TIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed_BH,Base19,NaN
2,Yes,RTP30,RTP,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed_BH,TIP24,NaN
3,Yes,RTP40,RTP,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed_BH,RTP30,NaN
4,Yes,RTP50,RTP,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed_BH,RTP40,NaN
5,Yes,NB30,No Build,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,TIP24,RTP30
6,Yes,NB40,No Build,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,NB30,RTP40
7,Yes,NB50,No Build,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,NB40,RTP50
8,Yes,UF30,Needs,2030 Needs,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,TIP24,RTP30


In [25]:
df_Scenarios_toAdjust

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryDBF,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
5,Yes,NB30,No Build,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,TIP24,RTP30
6,Yes,NB40,No Build,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,NB30,RTP40
7,Yes,NB50,No Build,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,NB40,RTP50
8,Yes,UF30,Needs,2030 Needs,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,TIP24,RTP30
9,Yes,UF40,Needs,2040 Needs,2040,v831_SE40_Net40UF_Summary_SEGID_Detailed_BH,UF30,RTP40
10,Yes,UF50,Needs,2050 Needs,2050,v831_SE50_Net50UF_Summary_SEGID_Detailed_BH,UF40,RTP50


In [26]:
#calculate adjustment for forecast for those non-RTP scenarios

dfs_ForecastAdj = []

for (idx, row) in df_Scenarios_toAdjust.iterrows():

    #print(row.loc[nameS] + ' CSV Importing')
    
    df_base = dfs_TDM_Total[idFromCode(row.loc['AdjScenarioCode'])]
    df_base.columns = (['DailyVolBase'])
    df_scenario  = dfs_TDM_Total[idFromCode(row.loc['ScenarioCode'])]
    df_scenario.columns = (['DailyVolScenario'])
    
    display(df_base)
    display(df_scenario)
    
    df_ForecastAdj = pd.DataFrame.merge(df_scenario,df_base,on=nameSegID,how='left')
    df_ForecastAdj['DailyVolAdj'] = df_ForecastAdj['DailyVolScenario'] - df_ForecastAdj['DailyVolBase']
    
    df_ForecastAdj.to_csv(os.path.join(dirResults,'ForecastAdj_' + row.loc['ScenarioCode'] + '.csv'))
    
    display(df_ForecastAdj)
    
    dfs_ForecastAdj.append(df_ForecastAdj)

,DailyVolBase
SEGID,
0006_146.9,1559.0
0006_149.9,1755.5
0006_150.6,1970.7
0006_152.6,2894.4
0006_152.9,3986.8
...,...
WFRC_8215,7150.5
WFRC_8216,18404.1
WFRC_8217,0.0


,DailyVolScenario
SEGID,
0006_146.9,1546.0
0006_149.9,1998.1
0006_150.6,2226.3
0006_152.6,3243.1
0006_152.9,4373.1
...,...
WFRC_8213,706.9
WFRC_8215,6656.0
WFRC_8216,17437.9


,DailyVolScenario,DailyVolBase,DailyVolAdj
SEGID,,,
0006_146.9,1546.0,1559.0,-13.0
0006_149.9,1998.1,1755.5,242.6
0006_150.6,2226.3,1970.7,255.6
0006_152.6,3243.1,2894.4,348.7
0006_152.9,4373.1,3986.8,386.3
...,...,...,...
WFRC_8213,706.9,709.8,-2.9
WFRC_8215,6656.0,7150.5,-494.5
WFRC_8216,17437.9,18404.1,-966.2


,DailyVolBase
SEGID,
0006_146.9,1684.9
0006_149.9,2230.4
0006_150.6,2456.5
0006_152.6,3346.8
0006_152.9,4750.8
...,...
WFRC_8215,9600.4
WFRC_8216,22968.9
WFRC_8217,0.0


,DailyVolScenario
SEGID,
0006_146.9,1669.8
0006_149.9,3098.9
0006_150.6,3352.3
0006_152.6,4473.5
0006_152.9,6005.6
...,...
WFRC_8213,829.4
WFRC_8215,8957.0
WFRC_8216,23391.1


...

,DailyVolScenario
SEGID,
0006_146.9,1685.8
0006_149.9,2111.0
0006_150.6,2340.3
0006_152.6,3251.5
0006_152.9,4679.9
...,...
WFRC_8215,9404.0
WFRC_8216,24993.0
WFRC_8217,0.0


,DailyVolScenario,DailyVolBase,DailyVolAdj
SEGID,,,
0006_146.9,1685.8,1684.9,0.9
0006_149.9,2111.0,2230.4,-119.4
0006_150.6,2340.3,2456.5,-116.2
0006_152.6,3251.5,3346.8,-95.3
0006_152.9,4679.9,4750.8,-70.9
...,...,...,...
WFRC_8215,9404.0,9600.4,-196.4
WFRC_8216,24993.0,22968.9,2024.1
WFRC_8217,0.0,0.0,0.0


,DailyVolBase
SEGID,
0006_146.9,1811.9
0006_149.9,3020.6
0006_150.6,3267.3
0006_152.6,4370.0
0006_152.9,5979.0
...,...
WFRC_8215,12274.5
WFRC_8216,28432.9
WFRC_8217,25756.0


,DailyVolScenario
SEGID,
0006_146.9,1813.4
0006_149.9,3029.6
0006_150.6,3276.8
0006_152.6,4372.0
0006_152.9,5982.6
...,...
WFRC_8215,11926.5
WFRC_8216,28419.9
WFRC_8217,25320.7


,DailyVolScenario,DailyVolBase,DailyVolAdj
SEGID,,,
0006_146.9,1813.4,1811.9,1.5
0006_149.9,3029.6,3020.6,9.0
0006_150.6,3276.8,3267.3,9.5
0006_152.6,4372.0,4370.0,2.0
0006_152.9,5982.6,5979.0,3.6
...,...,...,...
WFRC_8215,11926.5,12274.5,-348.0
WFRC_8216,28419.9,28432.9,-13.0
WFRC_8217,25320.7,25756.0,-435.3


# AADT and AAPeriodT Forecasts (Segments, Forecasts, TDM)

In [27]:
#import arcgis libraries
from arcgis.gis import *
gis = GIS()

shp_Forecasts = os.path.join(dirData,  r'forecasts\WFRCForecasts_2019RTP_v831_20200508_StateWideMerged_20200608h.shp')
                        
#import into spatially-enabled DataFrame
sdf_Forecasts = pd.DataFrame.spatial.from_featureclass(shp_Forecasts)
sdf_Forecasts

,FID,RELEASE,SEGID,ROUTE_ID,BMP,EMP,FULLNAME,CO_FIPS,CO_NAME,X,...,FAC_SUM,FAC_FAL,FAC_MAXMO,FAC_MAX,FACMANADJ,SUTRUCKS,CUTRUCKS,PLANAREA,Shape_Leng,SHAPE
0,0,2019 RTP v8.3.1 20200508,0068_036.0,0068,36.037998,38.106998,CAMP WILLIAMS R,35,SALT LAKE,420614.3190,...,1.057,1.022,6,1.067,0,0.063,0.030,WFRC,0.000000,"{""paths"": [[[421160.5499999998, 4476386.34], [..."
1,1,2019 RTP v8.3.1 20200508,WFRC_8000,WFRC,1.488000,1.488000,,35,SALT LAKE,420880.8540,...,1.057,1.022,6,1.067,0,0.035,0.025,WFRC,0.000000,"{""paths"": [[[420067.91000000015, 4479534.80000..."
2,2,2019 RTP v8.3.1 20200508,0131_001.7,0131,1.742000,3.047000,PORTER ROCKWELL,35,SALT LAKE,422891.2500,...,1.026,1.018,9,1.044,0,0.032,0.018,WFRC,0.000000,"{""paths"": [[[422222.28000000026, 4480359.77], ..."
3,3,2019 RTP v8.3.1 20200508,0068_038.1,0068,38.106998,39.167000,CAMP WILLIAMS R,35,SALT LAKE,419789.5850,...,1.057,1.022,6,1.067,0,0.040,0.020,WFRC,0.000000,"{""paths"": [[[420067.91000000015, 4479534.80000..."
4,4,2019 RTP v8.3.1 20200508,2927_000.0,2927,0.000000,1.019000,DEER RIDGE DR,35,SALT LAKE,428613.1110,...,1.057,1.022,6,1.067,0,0.035,0.025,WFRC,0.000000,"{""paths"": [[[427935.76999999955, 4480756.36999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8510,8510,2019 RTP v8.3.1 20200508,MAG_6248,0,0.000000,0.000000,,49,UTAH,432054.9277,...,1.026,1.018,9,1.044,0,0.032,0.018,MAG,473.796140,"{""paths"": [[[435487.7599999998, 4462445.02], [..."
8511,8511,2019 RTP v8.3.1 20200508,2822_000.0,2822,0.000000,3.328000,NEBO LOOP RD,49,UTAH,439736.1222,...,1.088,1.033,8,1.092,0,0.017,0.012,MAG,5344.226776,"{""paths"": [[[438394.3700000001, 4430771.32], [..."
8512,8512,2019 RTP v8.3.1 20200508,MAG_6249,0,0.000000,0.000000,,0,,0.0000,...,1.057,1.022,6,1.067,0,0.120,0.100,MAG,595.109691,"{""paths"": [[[442185.8794999998, 4440527.9736],..."
8513,8513,2019 RTP v8.3.1 20200508,MAG_6250,0,0.000000,0.000000,,0,,0.0000,...,1.057,1.022,6,1.067,0,0.120,0.100,MAG,441.873728,"{""paths"": [[[442531.98900000006, 4440253.285],..."


In [28]:
for col in sdf_Forecasts.columns: 
    print(col) 

FID
RELEASE
SEGID
ROUTE_ID
BMP
EMP
FULLNAME
CO_FIPS
CO_NAME
X
Y
DISTANCE
F2050
F2040
F2030
F2024
F2019
CH17TO50
CH40TO50
CH30TO40
CH24TO30
CH19TO24
CH15TO19
CH17TO19
FNOTES
MOREINFO
WFRC_FLG
AADT2017
AADT2016
AADT2015
AADT2014
AADT2013
AADT2012
AADT2011
AADT2010
AADT2009
AADT2008
AADT2007
AADT2006
AADT2005
AADT2004
AADT2003
AADT2002
AADT2001
AADT2000
AADT1999
AADT1998
AADT1997
AADT1996
AADT1994
AADT1993
AADT1992
AADT1991
AADT1990
AADT1989
AADT1988
AADT1987
AADT1986
AADT1985
AADT1984
AADT1983
AADT1982
AADT1981
SUTRK2017
CUTRK2017
DOWFACFC
DOWFACAT
FAC_MON
FAC_TUE
FAC_WED
FAC_THU
FAC_FRI
FAC_SAT
FAC_SUN
FAC_WDAVG
FAC_WEAVG
FAC_WEMAX
SSNGRP
SSNVOLCLS
SSNATGROUP
FAC_JAN
FAC_FEB
FAC_MAR
FAC_APR
FAC_MAY
FAC_JUN
FAC_JUL
FAC_AUG
FAC_SEP
FAC_OCT
FAC_NOV
FAC_DEC
FAC_WIN
FAC_SPR
FAC_SUM
FAC_FAL
FAC_MAXMO
FAC_MAX
FACMANADJ
SUTRUCKS
CUTRUCKS
PLANAREA
Shape_Leng
SHAPE


In [29]:
df = sdf_Forecasts

df = df[(df[nameSegID]=='0015_295.6')]

df

,FID,RELEASE,SEGID,ROUTE_ID,BMP,EMP,FULLNAME,CO_FIPS,CO_NAME,X,...,FAC_SUM,FAC_FAL,FAC_MAXMO,FAC_MAX,FACMANADJ,SUTRUCKS,CUTRUCKS,PLANAREA,Shape_Leng,SHAPE
388,388,2019 RTP v8.3.1 20200508,0015_295.6,0015,295.640991,297.944,I-15 NB FWY,35,SALT LAKE,423474.649,...,1.033,1.012,8,1.039,0,0.24,0.067,WFRC,0.0,"{""paths"": [[[423819.33999999985, 4493412.10999..."


In [30]:
#normalize data with forecasts

meltIndex    = ['SEGID','DOWFACFC','SSNGRP']
valueColumns = ['AADT2017','F2019','F2024','F2030','F2040','F2050']
df_Forecasts = pd.melt(sdf_Forecasts, id_vars=meltIndex, value_vars=valueColumns)
df_Forecasts.columns = ('SEGID','DOWFACFC','SsnGrp','FCOL','ForecastAADT')
df_Forecasts['FYEAR'] = (df_Forecasts['FCOL'].str[-4:]).astype('int64')
df_Forecasts = df_Forecasts[['SEGID','DOWFACFC','SsnGrp','FYEAR','ForecastAADT']]
df_Forecasts

,SEGID,DOWFACFC,SsnGrp,FYEAR,ForecastAADT
0,0068_036.0,Arterial,StatewideArterial,2017,31000
1,WFRC_8000,Arterial,StatewideArterial,2017,0
2,0131_001.7,Arterial,StatewideArterial,2017,1000
3,0068_038.1,Arterial,StatewideArterial,2017,22000
4,2927_000.0,Arterial,StatewideArterial,2017,500
...,...,...,...,...,...
51085,MAG_6248,Expressway,StatewideArterial,2050,118000
51086,2822_000.0,Arterial,StatewideArterial,2050,2100
51087,MAG_6249,Arterial,StatewideArterial,2050,15000
51088,MAG_6250,Arterial,StatewideArterial,2050,23000


In [31]:
df_Scenarios_toProcess

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryDBF,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Yes,Base19,Base,2019 Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed_BH,NaN,NaN
1,Yes,TIP24,TIP,2024 TIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed_BH,Base19,NaN
2,Yes,RTP30,RTP,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed_BH,TIP24,NaN
3,Yes,RTP40,RTP,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed_BH,RTP30,NaN
4,Yes,RTP50,RTP,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed_BH,RTP40,NaN
5,Yes,NB30,No Build,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,TIP24,RTP30
6,Yes,NB40,No Build,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,NB30,RTP40
7,Yes,NB50,No Build,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,NB40,RTP50
8,Yes,UF30,Needs,2030 Needs,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,TIP24,RTP30


In [47]:
#calculate forecast daily volume for each scenario to process


dfs_PeriodForecasts = []

for (idx, row) in df_Scenarios_toProcess.iterrows():

    _scenarioID = idx
    
    print(row.loc[nameS] + ' Processing')
    print(idx)
    #df_base = dfs_TDM_Total[idFromCode(row.loc['AdjScenarioCode'])]

    #display(dfs_TDM[idx])
    
    df_TDM = dfs_TDM[idx]
    
    #get TDM year
    _scenarioyear = yearFromID(_scenarioID)
    
    #filter only for year of forecast
    df_Forecasts_SYEAR = df_Forecasts[df_Forecasts.FYEAR == _scenarioyear]
    
    #adjust forecast
    
    for (idx, row) in df_Scenarios_toAdjust.iterrows():
        if (_scenarioID == idx):
            #display("adjust this")
            
            #get adjust id
            _adjustId = df_Scenarios_toAdjust.index.get_loc(idx)

            #display(_adjustId)
            #display(dfs_ForecastAdj[_adjustId])
            
            df_Forecasts_Adjust = pd.DataFrame.merge(df_Forecasts_SYEAR,dfs_ForecastAdj[_adjustId],on='SEGID')
            
            #adjust volume. if scenario vol = 0 then zero out forecast volume, otherwise forecast volume is adjusted
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DailyVolScenario'] == 0), 'ForecastAADTAdj'] = 0
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DailyVolScenario']  > 0), 'ForecastAADTAdj'] = df_Forecasts_Adjust['ForecastAADT'] + df_Forecasts_Adjust['DailyVolAdj'] 
            
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DailyVolScenario'] == 0), 'ForecastAADTAdj'] = 0
            
            #if adjustment < 0 then bring to 0
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['ForecastAADTAdj']   < 0), 'ForecastAADTAdj'] = 0  
            
            #display(df_Forecasts_Adjust)
            
            df_Forecasts_Adjust = df_Forecasts_Adjust[['SEGID','DOWFACFC','SsnGrp','FYEAR','ForecastAADTAdj']]
            df_Forecasts_Adjust = df_Forecasts_Adjust.rename(columns={"ForecastAADTAdj": "ForecastAADT"})
            
            df_Forecasts_SYEAR = df_Forecasts_Adjust.copy()            
            
    df_Forecasts_SYEAR = df_Forecasts_SYEAR.fillna(0)
    df_Forecasts_SYEAR['ForecastAADT'] = df_Forecasts_SYEAR['ForecastAADT'].astype(int)
    
    #display(df_Forecasts_SYEAR)
    
    df_Forecasts_SYEAR_export = df_Forecasts_SYEAR.copy()
    df_Forecasts_SYEAR_export['ScenarioCode'] = codeFromID(_scenarioID)
    df_Forecasts_SYEAR_export = df_Forecasts_SYEAR_export[['ScenarioCode','SEGID','ForecastAADT']]
    
    df_Forecasts_SYEAR_tocsv = pd.concat([df_Forecasts_SYEAR_tocsv,df_Forecasts_SYEAR_export])
    
    #display(os.path.join(dirResults + "0_Forecast_" + codeFromID(_scenarioID) + ".csv"))
    #df_Forecasts_SYEAR_export.to_csv(os.path.join(dirResults + r"\\0_Forecast_" + codeFromID(_scenarioID) + ".csv"),index=False)
    
    #join with TDM data
    df_Forecasts_withPeriod = pd.DataFrame.merge(df_TDM, df_Forecasts_SYEAR, on=('SEGID'))
    
    df_Forecasts_withPeriod['ForecastAAPeriodT'] = round(df_Forecasts_withPeriod['ForecastAADT'] * df_Forecasts_withPeriod['VolPercentSegDY'],0)
    
    #display(df_Forecasts_withPeriod.head(20))
    
    dfs_PeriodForecasts.append(df_Forecasts_withPeriod)

display(os.path.join(dirResults + "0_Forecast_" + codeFromID(_scenarioID) + ".csv"))
df_Forecasts_SYEAR_tocsv.to_csv(os.path.join(dirResults + r"\\0_Forecasts.csv"),index=False)


2019 Base Processing
0
2024 TIP Processing
1
2030 RTP Processing
2
2040 RTP Processing
3
2050 RTP Processing
4
2030 No Build Processing
5
2040 No Build Processing
6
2050 No Build Processing
7
2030 Needs Processing
8
2040 Needs Processing
9
2050 Needs Processing
10


'E:\\GitHub\\V-over-C-Calculations\\results0_Forecast_UF50.csv'

In [23]:
dfs_PeriodForecasts[0]

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT
0,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,1392.4,1.0,1065.0,Arterial,StatewideArterial,2019,0,0.0
1,0006_146.9,Arterial,1.0,2.0,D1,EV,166.40,0.088341,0.000000,0.119506,1392.4,1.0,1065.0,Arterial,StatewideArterial,2019,0,0.0
2,0006_146.9,Arterial,1.0,2.0,D1,MD,222.60,0.092992,0.069182,0.159868,1392.4,1.0,1065.0,Arterial,StatewideArterial,2019,0,0.0
3,0006_146.9,Arterial,1.0,2.0,D1,PM,179.60,0.067929,0.051782,0.128986,1392.4,1.0,1065.0,Arterial,StatewideArterial,2019,0,0.0
4,0006_146.9,Arterial,1.0,2.0,D2,AM,127.20,0.063679,0.044811,0.091353,1392.4,1.0,1065.0,Arterial,StatewideArterial,2019,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37789,WFRC_8218,CD Road,3.0,31.0,D2,EV,3624.60,0.138277,0.000000,0.126594,28631.8,1.0,1473.0,FreewayI215,FreewayI215,2019,25500,3228.0
37790,WFRC_8218,CD Road,3.0,31.0,D2,MD,4648.90,0.145282,0.074491,0.162368,28631.8,1.0,1473.0,FreewayI215,FreewayI215,2019,25500,4140.0
37791,WFRC_8218,CD Road,3.0,31.0,D2,PM,3819.30,0.114157,0.058649,0.133394,28631.8,1.0,1473.0,FreewayI215,FreewayI215,2019,25500,3402.0
37792,WFRC_8218,CD Road,3.0,31.0,D1,Daily,17179.08,0.129960,0.049752,0.600000,28631.8,1.0,1473.0,FreewayI215,FreewayI215,2019,25500,15300.0


In [24]:
df = dfs_PeriodForecasts[0]

df = df[(df[nameSegID]=='0085_011.3')]

display(df)

df2 = df[df[namePeriod]!='Daily']

display(df2)

df2 = df2.groupby(nameSegID).agg({'VOL':[np.sum],'VolPercentSegDY':[np.sum]})

display(df2)

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT
5216,0085_011.3,Arterial,2.0,13.0,D1,AM,5161.8,0.104324,0.072417,0.122399,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,2081.0
5217,0085_011.3,Arterial,2.0,13.0,D1,EV,4598.1,0.179879,0.000000,0.109032,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,1854.0
5218,0085_011.3,Arterial,2.0,13.0,D1,MD,7016.1,0.176466,0.125796,0.166369,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,2828.0
5219,0085_011.3,Arterial,2.0,13.0,D1,PM,4998.4,0.155030,0.108115,0.118524,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,2015.0
5220,0085_011.3,Arterial,2.0,13.0,D2,AM,3027.4,0.161789,0.116370,0.071787,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,1220.0
5221,0085_011.3,Arterial,2.0,13.0,D2,EV,4819.5,0.161594,0.000000,0.114282,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,1943.0
5222,0085_011.3,Arterial,2.0,13.0,D2,MD,6511.6,0.180662,0.131212,0.154406,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,2625.0
5223,0085_011.3,Arterial,2.0,13.0,D2,PM,6039.1,0.117865,0.080741,0.143202,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,2434.0
5224,0085_011.3,Arterial,2.0,13.0,D1,Daily,25303.2,0.154970,0.082782,0.600000,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,10200.0
5225,0085_011.3,Arterial,2.0,13.0,D2,Daily,16868.8,0.154970,0.082782,0.400000,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,6800.0


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT
5216,0085_011.3,Arterial,2.0,13.0,D1,AM,5161.8,0.104324,0.072417,0.122399,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,2081.0
5217,0085_011.3,Arterial,2.0,13.0,D1,EV,4598.1,0.179879,0.000000,0.109032,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,1854.0
5218,0085_011.3,Arterial,2.0,13.0,D1,MD,7016.1,0.176466,0.125796,0.166369,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,2828.0
5219,0085_011.3,Arterial,2.0,13.0,D1,PM,4998.4,0.155030,0.108115,0.118524,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,2015.0
5220,0085_011.3,Arterial,2.0,13.0,D2,AM,3027.4,0.161789,0.116370,0.071787,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,1220.0
5221,0085_011.3,Arterial,2.0,13.0,D2,EV,4819.5,0.161594,0.000000,0.114282,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,1943.0
5222,0085_011.3,Arterial,2.0,13.0,D2,MD,6511.6,0.180662,0.131212,0.154406,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,2625.0
5223,0085_011.3,Arterial,2.0,13.0,D2,PM,6039.1,0.117865,0.080741,0.143202,42172.0,2.0,2042.0,Expressway,StatewideArterial,2019,17000,2434.0


,VOL,VolPercentSegDY
,sum,sum
SEGID,,
0085_011.3,42172.0,1.0


# Adjust for Season, DOW

In [25]:
#read in data from other notebook pre-processing

csv_SeasonFactors = os.path.join(dirFactors, r'SeasonFactors.csv')
csv_DOWFactors = os.path.join(dirFactors, r'DOWFactors.csv')
csv_TODFactors = os.path.join(dirFactors, r'TODFactors.csv')

df_SeasonFactors = pd.read_csv(csv_SeasonFactors)
df_DOWFactors = pd.read_csv(csv_DOWFactors)
df_TODFactors = pd.read_csv(csv_TODFactors)

df_SeasonFactors = df_SeasonFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})
df_DOWFactors = df_DOWFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})
df_TODFactors = df_TODFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})

display(df_SeasonFactors)
display(df_DOWFactors)
display(df_TODFactors)

,SsnGrp,AREATYPE,Vol_From,Vol_To,FUNCGROUP,SiteGroupSeason,SeasonGroup,SeasonFactor
0,AlpineLoop,1,0,1000000,Arterial,SN0,M01-Jan,0.273
1,AlpineLoop,1,0,1000000,Arterial,SN0,M02-Feb,0.303
2,AlpineLoop,1,0,1000000,Arterial,SN0,M03-Mar,0.397
3,AlpineLoop,1,0,1000000,Arterial,SN0,M04-Apr,0.499
4,AlpineLoop,1,0,1000000,Arterial,SN0,M05-May,1.005
...,...,...,...,...,...,...,...,...
8887,WashingtonFreeway,5,0,1000000,CD Road,SNX,S03-Sum,1.066
8888,WashingtonFreeway,5,0,1000000,CD Road,SNX,S04-Fal,1.001
8889,WashingtonFreeway,5,0,1000000,CD Road,SNX,MaxMonth,1.099
8890,WashingtonFreeway,5,0,1000000,CD Road,SNX,MaxSeason,1.066


,DOWFACFC,AREATYPE,FUNCGROUP,SiteGroupDOW,DOWGroup,DOWFactor
0,Arterial,1,Arterial,WD0,1-Monday,0.907
1,Arterial,1,Arterial,WD0,2-Tuesday,0.902
2,Arterial,1,Arterial,WD0,3-Wednesday,0.943
3,Arterial,1,Arterial,WD0,4-Thursday,0.995
4,Arterial,1,Arterial,WD0,5-Friday,1.170
...,...,...,...,...,...,...
2335,FreewayUS40,5,CD Road,WDK,5-Friday,1.170
2336,FreewayUS40,5,CD Road,WDK,6-Saturday,1.029
2337,FreewayUS40,5,CD Road,WDK,7-Sunday,0.777
2338,FreewayUS40,5,CD Road,WDK,W1-Weekday,1.007


,SiteGroupSeason,SeasonGroup,DOWGroup,Period,HourPercentOf,VolMaxHourPercent
0,SN0,M01-Jan,1-Monday,AM,Period,0.552833
1,SN0,M01-Jan,1-Monday,Daily,Period,0.135094
2,SN0,M01-Jan,1-Monday,EV,Period,0.292623
3,SN0,M01-Jan,1-Monday,MD,Period,0.253412
4,SN0,M01-Jan,1-Monday,PM,Period,0.417949
...,...,...,...,...,...,...
58135,XX4,S04-Fal,W2-Weekend,AM,Daily,0.061062
58136,XX4,S04-Fal,W2-Weekend,Daily,Daily,0.087213
58137,XX4,S04-Fal,W2-Weekend,EV,Daily,0.062317
58138,XX4,S04-Fal,W2-Weekend,MD,Daily,0.065813


In [26]:
df_SD = pd.DataFrame.merge(df_SeasonFactors,df_DOWFactors,on=(nameAT,nameFG))
df_SD

,SsnGrp,AREATYPE,Vol_From,Vol_To,FUNCGROUP,SiteGroupSeason,SeasonGroup,SeasonFactor,DOWFACFC,SiteGroupDOW,DOWGroup,DOWFactor
0,AlpineLoop,1,0,1000000,Arterial,SN0,M01-Jan,0.273,Arterial,WD0,1-Monday,0.907
1,AlpineLoop,1,0,1000000,Arterial,SN0,M01-Jan,0.273,Arterial,WD0,2-Tuesday,0.902
2,AlpineLoop,1,0,1000000,Arterial,SN0,M01-Jan,0.273,Arterial,WD0,3-Wednesday,0.943
3,AlpineLoop,1,0,1000000,Arterial,SN0,M01-Jan,0.273,Arterial,WD0,4-Thursday,0.995
4,AlpineLoop,1,0,1000000,Arterial,SN0,M01-Jan,0.273,Arterial,WD0,5-Friday,1.170
...,...,...,...,...,...,...,...,...,...,...,...,...
1040359,WashingtonFreeway,5,0,1000000,CD Road,SNX,S00-Avg,1.000,FreewayUS40,WDK,5-Friday,1.170
1040360,WashingtonFreeway,5,0,1000000,CD Road,SNX,S00-Avg,1.000,FreewayUS40,WDK,6-Saturday,1.029
1040361,WashingtonFreeway,5,0,1000000,CD Road,SNX,S00-Avg,1.000,FreewayUS40,WDK,7-Sunday,0.777
1040362,WashingtonFreeway,5,0,1000000,CD Road,SNX,S00-Avg,1.000,FreewayUS40,WDK,W1-Weekday,1.007


In [27]:
#add season factor

dfs_PeriodForecasts_Season = []

#for (idx, row) in dfs_PeriodForecasts.iterrows():

for df_PeriodForecasts in dfs_PeriodForecasts: 
    
    #joing forecasts data with season factors
    df_PeriodForecasts_Season = pd.DataFrame.merge(df_PeriodForecasts,df_SeasonFactors,on=('SsnGrp',nameFG,nameAT),how='left')
    
    #remove unapplicable volume classes
    df_PeriodForecasts_Season = df_PeriodForecasts_Season[(df_PeriodForecasts_Season.ForecastAADT >= df_PeriodForecasts_Season.Vol_From) & (df_PeriodForecasts_Season.ForecastAADT < df_PeriodForecasts_Season.Vol_To)]
    
    #calculate Average Season
    df_PeriodForecasts_Season['ForecastASeasonPeriodT'] = df_PeriodForecasts_Season['ForecastAAPeriodT'] *df_PeriodForecasts_Season['SeasonFactor'] 
    
    display(df_PeriodForecasts_Season)
    
    dfs_PeriodForecasts_Season.append(df_PeriodForecasts_Season)
    

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
19,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,...,StatewideArterial,2019,0,0.0,0,2500,SNP,M01-Jan,0.815,0.0
20,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,...,StatewideArterial,2019,0,0.0,0,2500,SNP,M02-Feb,0.862,0.0
21,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,...,StatewideArterial,2019,0,0.0,0,2500,SNP,M03-Mar,0.949,0.0
22,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,...,StatewideArterial,2019,0,0.0,0,2500,SNP,M04-Apr,1.023,0.0
23,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,...,StatewideArterial,2019,0,0.0,0,2500,SNP,M05-May,1.079,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779337,WFRC_8218,CD Road,3.0,31.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,...,FreewayI215,2019,25500,10200.0,0,1000000,SN8,S03-Sum,1.012,10322.4
779338,WFRC_8218,CD Road,3.0,31.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,...,FreewayI215,2019,25500,10200.0,0,1000000,SN8,S04-Fal,1.003,10230.6
779339,WFRC_8218,CD Road,3.0,31.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,...,FreewayI215,2019,25500,10200.0,0,1000000,SN8,MaxMonth,1.025,10455.0
779340,WFRC_8218,CD Road,3.0,31.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,...,FreewayI215,2019,25500,10200.0,0,1000000,SN8,MaxSeason,1.015,10353.0


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
19,0006_146.9,Arterial,1.0,2.0,D1,AM,126.00,0.067460,0.049206,0.08596,...,StatewideArterial,2024,0,0.0,0,2500,SNP,M01-Jan,0.815,0.0
20,0006_146.9,Arterial,1.0,2.0,D1,AM,126.00,0.067460,0.049206,0.08596,...,StatewideArterial,2024,0,0.0,0,2500,SNP,M02-Feb,0.862,0.0
21,0006_146.9,Arterial,1.0,2.0,D1,AM,126.00,0.067460,0.049206,0.08596,...,StatewideArterial,2024,0,0.0,0,2500,SNP,M03-Mar,0.949,0.0
22,0006_146.9,Arterial,1.0,2.0,D1,AM,126.00,0.067460,0.049206,0.08596,...,StatewideArterial,2024,0,0.0,0,2500,SNP,M04-Apr,1.023,0.0
23,0006_146.9,Arterial,1.0,2.0,D1,AM,126.00,0.067460,0.049206,0.08596,...,StatewideArterial,2024,0,0.0,0,2500,SNP,M05-May,1.079,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780249,WFRC_8218,CD Road,3.0,31.0,D2,Daily,12412.32,0.134747,0.051133,0.40000,...,FreewayI215,2024,27500,11000.0,0,1000000,SN8,S03-Sum,1.012,11132.0
780250,WFRC_8218,CD Road,3.0,31.0,D2,Daily,12412.32,0.134747,0.051133,0.40000,...,FreewayI215,2024,27500,11000.0,0,1000000,SN8,S04-Fal,1.003,11033.0
780251,WFRC_8218,CD Road,3.0,31.0,D2,Daily,12412.32,0.134747,0.051133,0.40000,...,FreewayI215,2024,27500,11000.0,0,1000000,SN8,MaxMonth,1.025,11275.0
780252,WFRC_8218,CD Road,3.0,31.0,D2,Daily,12412.32,0.134747,0.051133,0.40000,...,FreewayI215,2024,27500,11000.0,0,1000000,SN8,MaxSeason,1.015,11165.0


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
19,0006_146.9,Arterial,1.0,2.0,D1,AM,142.30,0.063247,0.046381,0.091276,...,StatewideArterial,2030,0,0.0,0,2500,SNP,M01-Jan,0.815,0.00
20,0006_146.9,Arterial,1.0,2.0,D1,AM,142.30,0.063247,0.046381,0.091276,...,StatewideArterial,2030,0,0.0,0,2500,SNP,M02-Feb,0.862,0.00
21,0006_146.9,Arterial,1.0,2.0,D1,AM,142.30,0.063247,0.046381,0.091276,...,StatewideArterial,2030,0,0.0,0,2500,SNP,M03-Mar,0.949,0.00
22,0006_146.9,Arterial,1.0,2.0,D1,AM,142.30,0.063247,0.046381,0.091276,...,StatewideArterial,2030,0,0.0,0,2500,SNP,M04-Apr,1.023,0.00
23,0006_146.9,Arterial,1.0,2.0,D1,AM,142.30,0.063247,0.046381,0.091276,...,StatewideArterial,2030,0,0.0,0,2500,SNP,M05-May,1.079,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797501,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,...,StatewideArterial,2030,6200,2480.0,0,1000000,SNS,S03-Sum,1.057,2621.36
797502,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,...,StatewideArterial,2030,6200,2480.0,0,1000000,SNS,S04-Fal,1.022,2534.56
797503,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,...,StatewideArterial,2030,6200,2480.0,0,1000000,SNS,MaxMonth,1.067,2646.16
797504,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,...,StatewideArterial,2030,6200,2480.0,0,1000000,SNS,MaxSeason,1.057,2621.36


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
19,0006_146.9,Arterial,1.0,2.0,D1,AM,163.30,0.059400,0.044091,0.09692,...,StatewideArterial,2040,0,0.0,0,2500,SNP,M01-Jan,0.815,0.00
20,0006_146.9,Arterial,1.0,2.0,D1,AM,163.30,0.059400,0.044091,0.09692,...,StatewideArterial,2040,0,0.0,0,2500,SNP,M02-Feb,0.862,0.00
21,0006_146.9,Arterial,1.0,2.0,D1,AM,163.30,0.059400,0.044091,0.09692,...,StatewideArterial,2040,0,0.0,0,2500,SNP,M03-Mar,0.949,0.00
22,0006_146.9,Arterial,1.0,2.0,D1,AM,163.30,0.059400,0.044091,0.09692,...,StatewideArterial,2040,0,0.0,0,2500,SNP,M04-Apr,1.023,0.00
23,0006_146.9,Arterial,1.0,2.0,D1,AM,163.30,0.059400,0.044091,0.09692,...,StatewideArterial,2040,0,0.0,0,2500,SNP,M05-May,1.079,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811371,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1974.12,0.094239,0.040403,0.40000,...,StatewideArterial,2040,6400,2560.0,0,1000000,SNS,S03-Sum,1.057,2705.92
811372,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1974.12,0.094239,0.040403,0.40000,...,StatewideArterial,2040,6400,2560.0,0,1000000,SNS,S04-Fal,1.022,2616.32
811373,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1974.12,0.094239,0.040403,0.40000,...,StatewideArterial,2040,6400,2560.0,0,1000000,SNS,MaxMonth,1.067,2731.52
811374,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1974.12,0.094239,0.040403,0.40000,...,StatewideArterial,2040,6400,2560.0,0,1000000,SNS,MaxSeason,1.057,2705.92


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
19,0006_146.9,Arterial,1.0,2.0,D1,AM,194.50,0.053985,0.039589,0.107346,...,StatewideArterial,2050,2100,225.0,0,2500,SNP,M01-Jan,0.815,183.375
20,0006_146.9,Arterial,1.0,2.0,D1,AM,194.50,0.053985,0.039589,0.107346,...,StatewideArterial,2050,2100,225.0,0,2500,SNP,M02-Feb,0.862,193.950
21,0006_146.9,Arterial,1.0,2.0,D1,AM,194.50,0.053985,0.039589,0.107346,...,StatewideArterial,2050,2100,225.0,0,2500,SNP,M03-Mar,0.949,213.525
22,0006_146.9,Arterial,1.0,2.0,D1,AM,194.50,0.053985,0.039589,0.107346,...,StatewideArterial,2050,2100,225.0,0,2500,SNP,M04-Apr,1.023,230.175
23,0006_146.9,Arterial,1.0,2.0,D1,AM,194.50,0.053985,0.039589,0.107346,...,StatewideArterial,2050,2100,225.0,0,2500,SNP,M05-May,1.079,242.775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818325,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5245.32,0.156581,0.058498,0.400000,...,StatewideArterial,2050,13500,5400.0,0,1000000,SNS,S03-Sum,1.057,5707.800
818326,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5245.32,0.156581,0.058498,0.400000,...,StatewideArterial,2050,13500,5400.0,0,1000000,SNS,S04-Fal,1.022,5518.800
818327,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5245.32,0.156581,0.058498,0.400000,...,StatewideArterial,2050,13500,5400.0,0,1000000,SNS,MaxMonth,1.067,5761.800
818328,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5245.32,0.156581,0.058498,0.400000,...,StatewideArterial,2050,13500,5400.0,0,1000000,SNS,MaxSeason,1.057,5707.800


...

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
19,0006_146.9,Arterial,1.0,2.0,D1,AM,144.20,0.068655,0.049931,0.086358,...,StatewideArterial,2040,0.0,0.0,0,2500,SNP,M01-Jan,0.815,0.000
20,0006_146.9,Arterial,1.0,2.0,D1,AM,144.20,0.068655,0.049931,0.086358,...,StatewideArterial,2040,0.0,0.0,0,2500,SNP,M02-Feb,0.862,0.000
21,0006_146.9,Arterial,1.0,2.0,D1,AM,144.20,0.068655,0.049931,0.086358,...,StatewideArterial,2040,0.0,0.0,0,2500,SNP,M03-Mar,0.949,0.000
22,0006_146.9,Arterial,1.0,2.0,D1,AM,144.20,0.068655,0.049931,0.086358,...,StatewideArterial,2040,0.0,0.0,0,2500,SNP,M04-Apr,1.023,0.000
23,0006_146.9,Arterial,1.0,2.0,D1,AM,144.20,0.068655,0.049931,0.086358,...,StatewideArterial,2040,0.0,0.0,0,2500,SNP,M05-May,1.079,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762009,WFRC_8218,CD Road,3.0,31.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,...,FreewayI215,2040,34437.0,13775.0,0,1000000,SN8,S03-Sum,1.012,13940.300
762010,WFRC_8218,CD Road,3.0,31.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,...,FreewayI215,2040,34437.0,13775.0,0,1000000,SN8,S04-Fal,1.003,13816.325
762011,WFRC_8218,CD Road,3.0,31.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,...,FreewayI215,2040,34437.0,13775.0,0,1000000,SN8,MaxMonth,1.025,14119.375
762012,WFRC_8218,CD Road,3.0,31.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,...,FreewayI215,2040,34437.0,13775.0,0,1000000,SN8,MaxSeason,1.015,13981.625


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
19,0006_146.9,Arterial,1.0,2.0,D1,AM,156.1,0.067905,0.049327,0.086959,...,StatewideArterial,2050,2083.2,181.0,0,2500,SNP,M01-Jan,0.815,147.515
20,0006_146.9,Arterial,1.0,2.0,D1,AM,156.1,0.067905,0.049327,0.086959,...,StatewideArterial,2050,2083.2,181.0,0,2500,SNP,M02-Feb,0.862,156.022
21,0006_146.9,Arterial,1.0,2.0,D1,AM,156.1,0.067905,0.049327,0.086959,...,StatewideArterial,2050,2083.2,181.0,0,2500,SNP,M03-Mar,0.949,171.769
22,0006_146.9,Arterial,1.0,2.0,D1,AM,156.1,0.067905,0.049327,0.086959,...,StatewideArterial,2050,2083.2,181.0,0,2500,SNP,M04-Apr,1.023,185.163
23,0006_146.9,Arterial,1.0,2.0,D1,AM,156.1,0.067905,0.049327,0.086959,...,StatewideArterial,2050,2083.2,181.0,0,2500,SNP,M05-May,1.079,195.299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745593,WFRC_8218,CD Road,3.0,31.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,...,FreewayI215,2050,37550.0,15020.0,0,1000000,SN8,S03-Sum,1.012,15200.240
745594,WFRC_8218,CD Road,3.0,31.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,...,FreewayI215,2050,37550.0,15020.0,0,1000000,SN8,S04-Fal,1.003,15065.060
745595,WFRC_8218,CD Road,3.0,31.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,...,FreewayI215,2050,37550.0,15020.0,0,1000000,SN8,MaxMonth,1.025,15395.500
745596,WFRC_8218,CD Road,3.0,31.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,...,FreewayI215,2050,37550.0,15020.0,0,1000000,SN8,MaxSeason,1.015,15245.300


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
19,0006_146.9,Arterial,1.0,2.0,D1,AM,142.60,0.063114,0.046283,0.091457,...,StatewideArterial,2030,0.2,0.0,0,2500,SNP,M01-Jan,0.815,0.000
20,0006_146.9,Arterial,1.0,2.0,D1,AM,142.60,0.063114,0.046283,0.091457,...,StatewideArterial,2030,0.2,0.0,0,2500,SNP,M02-Feb,0.862,0.000
21,0006_146.9,Arterial,1.0,2.0,D1,AM,142.60,0.063114,0.046283,0.091457,...,StatewideArterial,2030,0.2,0.0,0,2500,SNP,M03-Mar,0.949,0.000
22,0006_146.9,Arterial,1.0,2.0,D1,AM,142.60,0.063114,0.046283,0.091457,...,StatewideArterial,2030,0.2,0.0,0,2500,SNP,M04-Apr,1.023,0.000
23,0006_146.9,Arterial,1.0,2.0,D1,AM,142.60,0.063114,0.046283,0.091457,...,StatewideArterial,2030,0.2,0.0,0,2500,SNP,M05-May,1.079,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798451,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,...,StatewideArterial,2030,6005.4,2402.0,0,1000000,SNS,S03-Sum,1.057,2538.914
798452,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,...,StatewideArterial,2030,6005.4,2402.0,0,1000000,SNS,S04-Fal,1.022,2454.844
798453,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,...,StatewideArterial,2030,6005.4,2402.0,0,1000000,SNS,MaxMonth,1.067,2562.934
798454,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,...,StatewideArterial,2030,6005.4,2402.0,0,1000000,SNS,MaxSeason,1.057,2538.914


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
19,0006_146.9,Arterial,1.0,2.0,D1,AM,163.50,0.059327,0.044037,0.096987,...,StatewideArterial,2040,0.9,0.0,0,2500,SNP,M01-Jan,0.815,0.000
20,0006_146.9,Arterial,1.0,2.0,D1,AM,163.50,0.059327,0.044037,0.096987,...,StatewideArterial,2040,0.9,0.0,0,2500,SNP,M02-Feb,0.862,0.000
21,0006_146.9,Arterial,1.0,2.0,D1,AM,163.50,0.059327,0.044037,0.096987,...,StatewideArterial,2040,0.9,0.0,0,2500,SNP,M03-Mar,0.949,0.000
22,0006_146.9,Arterial,1.0,2.0,D1,AM,163.50,0.059327,0.044037,0.096987,...,StatewideArterial,2040,0.9,0.0,0,2500,SNP,M04-Apr,1.023,0.000
23,0006_146.9,Arterial,1.0,2.0,D1,AM,163.50,0.059327,0.044037,0.096987,...,StatewideArterial,2040,0.9,0.0,0,2500,SNP,M05-May,1.079,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814943,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1982.12,0.092648,0.037374,0.400000,...,StatewideArterial,2040,6420.0,2568.0,0,1000000,SNS,S03-Sum,1.057,2714.376
814944,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1982.12,0.092648,0.037374,0.400000,...,StatewideArterial,2040,6420.0,2568.0,0,1000000,SNS,S04-Fal,1.022,2624.496
814945,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1982.12,0.092648,0.037374,0.400000,...,StatewideArterial,2040,6420.0,2568.0,0,1000000,SNS,MaxMonth,1.067,2740.056
814946,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1982.12,0.092648,0.037374,0.400000,...,StatewideArterial,2040,6420.0,2568.0,0,1000000,SNS,MaxSeason,1.057,2714.376


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
19,0006_146.9,Arterial,1.0,2.0,D1,AM,195.50,0.053708,0.039386,0.107809,...,StatewideArterial,2050,2101.5,227.0,0,2500,SNP,M01-Jan,0.815,185.005
20,0006_146.9,Arterial,1.0,2.0,D1,AM,195.50,0.053708,0.039386,0.107809,...,StatewideArterial,2050,2101.5,227.0,0,2500,SNP,M02-Feb,0.862,195.674
21,0006_146.9,Arterial,1.0,2.0,D1,AM,195.50,0.053708,0.039386,0.107809,...,StatewideArterial,2050,2101.5,227.0,0,2500,SNP,M03-Mar,0.949,215.423
22,0006_146.9,Arterial,1.0,2.0,D1,AM,195.50,0.053708,0.039386,0.107809,...,StatewideArterial,2050,2101.5,227.0,0,2500,SNP,M04-Apr,1.023,232.221
23,0006_146.9,Arterial,1.0,2.0,D1,AM,195.50,0.053708,0.039386,0.107809,...,StatewideArterial,2050,2101.5,227.0,0,2500,SNP,M05-May,1.079,244.933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819009,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,...,StatewideArterial,2050,13750.6,5500.0,0,1000000,SNS,S03-Sum,1.057,5813.500
819010,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,...,StatewideArterial,2050,13750.6,5500.0,0,1000000,SNS,S04-Fal,1.022,5621.000
819011,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,...,StatewideArterial,2050,13750.6,5500.0,0,1000000,SNS,MaxMonth,1.067,5868.500
819012,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,...,StatewideArterial,2050,13750.6,5500.0,0,1000000,SNS,MaxSeason,1.057,5813.500


In [28]:
df = dfs_PeriodForecasts_Season[0]

df = df[(df[nameSegID]=='0085_011.3')]

display(df)

df2 = df[df[namePeriod]!='Daily']

display(df2)

df2 = df2.groupby(nameSegID).agg({'VOL':[np.sum],'VolPercentSegDY':[np.sum]})

display(df2)

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
111036,0085_011.3,Arterial,2.0,13.0,D1,AM,5161.8,0.104324,0.072417,0.122399,...,StatewideArterial,2019,17000,2081.0,0,1000000,SNS,M01-Jan,0.851,1770.931
111037,0085_011.3,Arterial,2.0,13.0,D1,AM,5161.8,0.104324,0.072417,0.122399,...,StatewideArterial,2019,17000,2081.0,0,1000000,SNS,M02-Feb,0.896,1864.576
111038,0085_011.3,Arterial,2.0,13.0,D1,AM,5161.8,0.104324,0.072417,0.122399,...,StatewideArterial,2019,17000,2081.0,0,1000000,SNS,M03-Mar,0.980,2039.380
111039,0085_011.3,Arterial,2.0,13.0,D1,AM,5161.8,0.104324,0.072417,0.122399,...,StatewideArterial,2019,17000,2081.0,0,1000000,SNS,M04-Apr,1.015,2112.215
111040,0085_011.3,Arterial,2.0,13.0,D1,AM,5161.8,0.104324,0.072417,0.122399,...,StatewideArterial,2019,17000,2081.0,0,1000000,SNS,M05-May,1.057,2199.617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111221,0085_011.3,Arterial,2.0,13.0,D2,Daily,16868.8,0.154970,0.082782,0.400000,...,StatewideArterial,2019,17000,6800.0,0,1000000,SNS,S03-Sum,1.057,7187.600
111222,0085_011.3,Arterial,2.0,13.0,D2,Daily,16868.8,0.154970,0.082782,0.400000,...,StatewideArterial,2019,17000,6800.0,0,1000000,SNS,S04-Fal,1.022,6949.600
111223,0085_011.3,Arterial,2.0,13.0,D2,Daily,16868.8,0.154970,0.082782,0.400000,...,StatewideArterial,2019,17000,6800.0,0,1000000,SNS,MaxMonth,1.067,7255.600
111224,0085_011.3,Arterial,2.0,13.0,D2,Daily,16868.8,0.154970,0.082782,0.400000,...,StatewideArterial,2019,17000,6800.0,0,1000000,SNS,MaxSeason,1.057,7187.600


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
111036,0085_011.3,Arterial,2.0,13.0,D1,AM,5161.8,0.104324,0.072417,0.122399,...,StatewideArterial,2019,17000,2081.0,0,1000000,SNS,M01-Jan,0.851,1770.931
111037,0085_011.3,Arterial,2.0,13.0,D1,AM,5161.8,0.104324,0.072417,0.122399,...,StatewideArterial,2019,17000,2081.0,0,1000000,SNS,M02-Feb,0.896,1864.576
111038,0085_011.3,Arterial,2.0,13.0,D1,AM,5161.8,0.104324,0.072417,0.122399,...,StatewideArterial,2019,17000,2081.0,0,1000000,SNS,M03-Mar,0.980,2039.380
111039,0085_011.3,Arterial,2.0,13.0,D1,AM,5161.8,0.104324,0.072417,0.122399,...,StatewideArterial,2019,17000,2081.0,0,1000000,SNS,M04-Apr,1.015,2112.215
111040,0085_011.3,Arterial,2.0,13.0,D1,AM,5161.8,0.104324,0.072417,0.122399,...,StatewideArterial,2019,17000,2081.0,0,1000000,SNS,M05-May,1.057,2199.617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111183,0085_011.3,Arterial,2.0,13.0,D2,PM,6039.1,0.117865,0.080741,0.143202,...,StatewideArterial,2019,17000,2434.0,0,1000000,SNS,S03-Sum,1.057,2572.738
111184,0085_011.3,Arterial,2.0,13.0,D2,PM,6039.1,0.117865,0.080741,0.143202,...,StatewideArterial,2019,17000,2434.0,0,1000000,SNS,S04-Fal,1.022,2487.548
111185,0085_011.3,Arterial,2.0,13.0,D2,PM,6039.1,0.117865,0.080741,0.143202,...,StatewideArterial,2019,17000,2434.0,0,1000000,SNS,MaxMonth,1.067,2597.078
111186,0085_011.3,Arterial,2.0,13.0,D2,PM,6039.1,0.117865,0.080741,0.143202,...,StatewideArterial,2019,17000,2434.0,0,1000000,SNS,MaxSeason,1.057,2572.738


,VOL,VolPercentSegDY
,sum,sum
SEGID,,
0085_011.3,801268.0,19.0


In [29]:
#add dow factor

dfs_PeriodForecasts_Season_DOW = []

#for (idx, row) in dfs_PeriodForecasts.iterrows():

for df_PeriodForecasts_Season in dfs_PeriodForecasts_Season: 
    
    #joing forecasts data with season factors
    df_PeriodForecasts_Season_DOW = pd.DataFrame.merge(df_PeriodForecasts_Season,df_DOWFactors,on=('DOWFACFC',nameFG,nameAT),how='left')
    
    #calculate Average Season DOW
    df_PeriodForecasts_Season_DOW['ForecastASeasonDOWPeriodT'] = df_PeriodForecasts_Season_DOW['ForecastASeasonPeriodT'] *df_PeriodForecasts_Season_DOW['DOWFactor'] 
    
    display(df_PeriodForecasts_Season_DOW)
    
    dfs_PeriodForecasts_Season_DOW.append(df_PeriodForecasts_Season_DOW)

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,1-Monday,0.907,0.0
1,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,2-Tuesday,0.902,0.0
2,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,3-Wednesday,0.943,0.0
3,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,4-Thursday,0.995,0.0
4,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,5-Friday,1.170,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6462769,WFRC_8218,CD Road,3.0,31.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,...,0,1000000,SN8,S00-Avg,1.000,10200.0,WDB,5-Friday,1.154,11770.8
6462770,WFRC_8218,CD Road,3.0,31.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,...,0,1000000,SN8,S00-Avg,1.000,10200.0,WDB,6-Saturday,0.788,8037.6
6462771,WFRC_8218,CD Road,3.0,31.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,...,0,1000000,SN8,S00-Avg,1.000,10200.0,WDB,7-Sunday,0.576,5875.2
6462772,WFRC_8218,CD Road,3.0,31.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,...,0,1000000,SN8,S00-Avg,1.000,10200.0,WDB,W1-Weekday,1.123,11454.6


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0006_146.9,Arterial,1.0,2.0,D1,AM,126.00,0.067460,0.049206,0.08596,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,1-Monday,0.907,0.0
1,0006_146.9,Arterial,1.0,2.0,D1,AM,126.00,0.067460,0.049206,0.08596,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,2-Tuesday,0.902,0.0
2,0006_146.9,Arterial,1.0,2.0,D1,AM,126.00,0.067460,0.049206,0.08596,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,3-Wednesday,0.943,0.0
3,0006_146.9,Arterial,1.0,2.0,D1,AM,126.00,0.067460,0.049206,0.08596,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,4-Thursday,0.995,0.0
4,0006_146.9,Arterial,1.0,2.0,D1,AM,126.00,0.067460,0.049206,0.08596,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,5-Friday,1.170,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6576313,WFRC_8218,CD Road,3.0,31.0,D2,Daily,12412.32,0.134747,0.051133,0.40000,...,0,1000000,SN8,S00-Avg,1.000,11000.0,WDB,5-Friday,1.154,12694.0
6576314,WFRC_8218,CD Road,3.0,31.0,D2,Daily,12412.32,0.134747,0.051133,0.40000,...,0,1000000,SN8,S00-Avg,1.000,11000.0,WDB,6-Saturday,0.788,8668.0
6576315,WFRC_8218,CD Road,3.0,31.0,D2,Daily,12412.32,0.134747,0.051133,0.40000,...,0,1000000,SN8,S00-Avg,1.000,11000.0,WDB,7-Sunday,0.576,6336.0
6576316,WFRC_8218,CD Road,3.0,31.0,D2,Daily,12412.32,0.134747,0.051133,0.40000,...,0,1000000,SN8,S00-Avg,1.000,11000.0,WDB,W1-Weekday,1.123,12353.0


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0006_146.9,Arterial,1.0,2.0,D1,AM,142.30,0.063247,0.046381,0.091276,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,1-Monday,0.907,0.00
1,0006_146.9,Arterial,1.0,2.0,D1,AM,142.30,0.063247,0.046381,0.091276,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,2-Tuesday,0.902,0.00
2,0006_146.9,Arterial,1.0,2.0,D1,AM,142.30,0.063247,0.046381,0.091276,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,3-Wednesday,0.943,0.00
3,0006_146.9,Arterial,1.0,2.0,D1,AM,142.30,0.063247,0.046381,0.091276,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,4-Thursday,0.995,0.00
4,0006_146.9,Arterial,1.0,2.0,D1,AM,142.30,0.063247,0.046381,0.091276,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,5-Friday,1.170,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6782197,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,...,0,1000000,SNS,S00-Avg,1.000,2480.0,WD1,5-Friday,1.142,2832.16
6782198,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,...,0,1000000,SNS,S00-Avg,1.000,2480.0,WD1,6-Saturday,1.036,2569.28
6782199,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,...,0,1000000,SNS,S00-Avg,1.000,2480.0,WD1,7-Sunday,0.773,1917.04
6782200,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,...,0,1000000,SNS,S00-Avg,1.000,2480.0,WD1,W1-Weekday,1.012,2509.76


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0006_146.9,Arterial,1.0,2.0,D1,AM,163.30,0.059400,0.044091,0.09692,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,1-Monday,0.907,0.00
1,0006_146.9,Arterial,1.0,2.0,D1,AM,163.30,0.059400,0.044091,0.09692,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,2-Tuesday,0.902,0.00
2,0006_146.9,Arterial,1.0,2.0,D1,AM,163.30,0.059400,0.044091,0.09692,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,3-Wednesday,0.943,0.00
3,0006_146.9,Arterial,1.0,2.0,D1,AM,163.30,0.059400,0.044091,0.09692,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,4-Thursday,0.995,0.00
4,0006_146.9,Arterial,1.0,2.0,D1,AM,163.30,0.059400,0.044091,0.09692,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,5-Friday,1.170,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6992527,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1974.12,0.094239,0.040403,0.40000,...,0,1000000,SNS,S00-Avg,1.000,2560.0,WD1,5-Friday,1.142,2923.52
6992528,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1974.12,0.094239,0.040403,0.40000,...,0,1000000,SNS,S00-Avg,1.000,2560.0,WD1,6-Saturday,1.036,2652.16
6992529,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1974.12,0.094239,0.040403,0.40000,...,0,1000000,SNS,S00-Avg,1.000,2560.0,WD1,7-Sunday,0.773,1978.88
6992530,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1974.12,0.094239,0.040403,0.40000,...,0,1000000,SNS,S00-Avg,1.000,2560.0,WD1,W1-Weekday,1.012,2590.72


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0006_146.9,Arterial,1.0,2.0,D1,AM,194.50,0.053985,0.039589,0.107346,...,0,2500,SNP,M01-Jan,0.815,183.375,WD0,1-Monday,0.907,166.321125
1,0006_146.9,Arterial,1.0,2.0,D1,AM,194.50,0.053985,0.039589,0.107346,...,0,2500,SNP,M01-Jan,0.815,183.375,WD0,2-Tuesday,0.902,165.404250
2,0006_146.9,Arterial,1.0,2.0,D1,AM,194.50,0.053985,0.039589,0.107346,...,0,2500,SNP,M01-Jan,0.815,183.375,WD0,3-Wednesday,0.943,172.922625
3,0006_146.9,Arterial,1.0,2.0,D1,AM,194.50,0.053985,0.039589,0.107346,...,0,2500,SNP,M01-Jan,0.815,183.375,WD0,4-Thursday,0.995,182.458125
4,0006_146.9,Arterial,1.0,2.0,D1,AM,194.50,0.053985,0.039589,0.107346,...,0,2500,SNP,M01-Jan,0.815,183.375,WD0,5-Friday,1.170,214.548750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7196701,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5245.32,0.156581,0.058498,0.400000,...,0,1000000,SNS,S00-Avg,1.000,5400.000,WD1,5-Friday,1.142,6166.800000
7196702,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5245.32,0.156581,0.058498,0.400000,...,0,1000000,SNS,S00-Avg,1.000,5400.000,WD1,6-Saturday,1.036,5594.400000
7196703,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5245.32,0.156581,0.058498,0.400000,...,0,1000000,SNS,S00-Avg,1.000,5400.000,WD1,7-Sunday,0.773,4174.200000
7196704,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5245.32,0.156581,0.058498,0.400000,...,0,1000000,SNS,S00-Avg,1.000,5400.000,WD1,W1-Weekday,1.012,5464.800000


...

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0006_146.9,Arterial,1.0,2.0,D1,AM,144.20,0.068655,0.049931,0.086358,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,1-Monday,0.907,0.000
1,0006_146.9,Arterial,1.0,2.0,D1,AM,144.20,0.068655,0.049931,0.086358,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,2-Tuesday,0.902,0.000
2,0006_146.9,Arterial,1.0,2.0,D1,AM,144.20,0.068655,0.049931,0.086358,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,3-Wednesday,0.943,0.000
3,0006_146.9,Arterial,1.0,2.0,D1,AM,144.20,0.068655,0.049931,0.086358,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,4-Thursday,0.995,0.000
4,0006_146.9,Arterial,1.0,2.0,D1,AM,144.20,0.068655,0.049931,0.086358,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,5-Friday,1.170,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571867,WFRC_8218,CD Road,3.0,31.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,...,0,1000000,SN8,S00-Avg,1.000,13775.0,WDB,5-Friday,1.154,15896.350
6571868,WFRC_8218,CD Road,3.0,31.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,...,0,1000000,SN8,S00-Avg,1.000,13775.0,WDB,6-Saturday,0.788,10854.700
6571869,WFRC_8218,CD Road,3.0,31.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,...,0,1000000,SN8,S00-Avg,1.000,13775.0,WDB,7-Sunday,0.576,7934.400
6571870,WFRC_8218,CD Road,3.0,31.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,...,0,1000000,SN8,S00-Avg,1.000,13775.0,WDB,W1-Weekday,1.123,15469.325


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0006_146.9,Arterial,1.0,2.0,D1,AM,156.1,0.067905,0.049327,0.086959,...,0,2500,SNP,M01-Jan,0.815,147.515,WD0,1-Monday,0.907,133.796105
1,0006_146.9,Arterial,1.0,2.0,D1,AM,156.1,0.067905,0.049327,0.086959,...,0,2500,SNP,M01-Jan,0.815,147.515,WD0,2-Tuesday,0.902,133.058530
2,0006_146.9,Arterial,1.0,2.0,D1,AM,156.1,0.067905,0.049327,0.086959,...,0,2500,SNP,M01-Jan,0.815,147.515,WD0,3-Wednesday,0.943,139.106645
3,0006_146.9,Arterial,1.0,2.0,D1,AM,156.1,0.067905,0.049327,0.086959,...,0,2500,SNP,M01-Jan,0.815,147.515,WD0,4-Thursday,0.995,146.777425
4,0006_146.9,Arterial,1.0,2.0,D1,AM,156.1,0.067905,0.049327,0.086959,...,0,2500,SNP,M01-Jan,0.815,147.515,WD0,5-Friday,1.170,172.592550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571867,WFRC_8218,CD Road,3.0,31.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,...,0,1000000,SN8,S00-Avg,1.000,15020.000,WDB,5-Friday,1.154,17333.080000
6571868,WFRC_8218,CD Road,3.0,31.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,...,0,1000000,SN8,S00-Avg,1.000,15020.000,WDB,6-Saturday,0.788,11835.760000
6571869,WFRC_8218,CD Road,3.0,31.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,...,0,1000000,SN8,S00-Avg,1.000,15020.000,WDB,7-Sunday,0.576,8651.520000
6571870,WFRC_8218,CD Road,3.0,31.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,...,0,1000000,SN8,S00-Avg,1.000,15020.000,WDB,W1-Weekday,1.123,16867.460000


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0006_146.9,Arterial,1.0,2.0,D1,AM,142.60,0.063114,0.046283,0.091457,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,1-Monday,0.907,0.000
1,0006_146.9,Arterial,1.0,2.0,D1,AM,142.60,0.063114,0.046283,0.091457,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,2-Tuesday,0.902,0.000
2,0006_146.9,Arterial,1.0,2.0,D1,AM,142.60,0.063114,0.046283,0.091457,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,3-Wednesday,0.943,0.000
3,0006_146.9,Arterial,1.0,2.0,D1,AM,142.60,0.063114,0.046283,0.091457,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,4-Thursday,0.995,0.000
4,0006_146.9,Arterial,1.0,2.0,D1,AM,142.60,0.063114,0.046283,0.091457,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,5-Friday,1.170,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6794167,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,...,0,1000000,SNS,S00-Avg,1.000,2402.0,WD1,5-Friday,1.142,2743.084
6794168,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,...,0,1000000,SNS,S00-Avg,1.000,2402.0,WD1,6-Saturday,1.036,2488.472
6794169,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,...,0,1000000,SNS,S00-Avg,1.000,2402.0,WD1,7-Sunday,0.773,1856.746
6794170,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,...,0,1000000,SNS,S00-Avg,1.000,2402.0,WD1,W1-Weekday,1.012,2430.824


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0006_146.9,Arterial,1.0,2.0,D1,AM,163.50,0.059327,0.044037,0.096987,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,1-Monday,0.907,0.000
1,0006_146.9,Arterial,1.0,2.0,D1,AM,163.50,0.059327,0.044037,0.096987,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,2-Tuesday,0.902,0.000
2,0006_146.9,Arterial,1.0,2.0,D1,AM,163.50,0.059327,0.044037,0.096987,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,3-Wednesday,0.943,0.000
3,0006_146.9,Arterial,1.0,2.0,D1,AM,163.50,0.059327,0.044037,0.096987,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,4-Thursday,0.995,0.000
4,0006_146.9,Arterial,1.0,2.0,D1,AM,163.50,0.059327,0.044037,0.096987,...,0,2500,SNP,M01-Jan,0.815,0.0,WD0,5-Friday,1.170,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7016125,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1982.12,0.092648,0.037374,0.400000,...,0,1000000,SNS,S00-Avg,1.000,2568.0,WD1,5-Friday,1.142,2932.656
7016126,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1982.12,0.092648,0.037374,0.400000,...,0,1000000,SNS,S00-Avg,1.000,2568.0,WD1,6-Saturday,1.036,2660.448
7016127,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1982.12,0.092648,0.037374,0.400000,...,0,1000000,SNS,S00-Avg,1.000,2568.0,WD1,7-Sunday,0.773,1985.064
7016128,WFRC_8219,Arterial,2.0,3.0,D2,Daily,1982.12,0.092648,0.037374,0.400000,...,0,1000000,SNS,S00-Avg,1.000,2568.0,WD1,W1-Weekday,1.012,2598.816


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0006_146.9,Arterial,1.0,2.0,D1,AM,195.50,0.053708,0.039386,0.107809,...,0,2500,SNP,M01-Jan,0.815,185.005,WD0,1-Monday,0.907,167.799535
1,0006_146.9,Arterial,1.0,2.0,D1,AM,195.50,0.053708,0.039386,0.107809,...,0,2500,SNP,M01-Jan,0.815,185.005,WD0,2-Tuesday,0.902,166.874510
2,0006_146.9,Arterial,1.0,2.0,D1,AM,195.50,0.053708,0.039386,0.107809,...,0,2500,SNP,M01-Jan,0.815,185.005,WD0,3-Wednesday,0.943,174.459715
3,0006_146.9,Arterial,1.0,2.0,D1,AM,195.50,0.053708,0.039386,0.107809,...,0,2500,SNP,M01-Jan,0.815,185.005,WD0,4-Thursday,0.995,184.079975
4,0006_146.9,Arterial,1.0,2.0,D1,AM,195.50,0.053708,0.039386,0.107809,...,0,2500,SNP,M01-Jan,0.815,185.005,WD0,5-Friday,1.170,216.455850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7197385,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,...,0,1000000,SNS,S00-Avg,1.000,5500.000,WD1,5-Friday,1.142,6281.000000
7197386,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,...,0,1000000,SNS,S00-Avg,1.000,5500.000,WD1,6-Saturday,1.036,5698.000000
7197387,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,...,0,1000000,SNS,S00-Avg,1.000,5500.000,WD1,7-Sunday,0.773,4251.500000
7197388,WFRC_8219,Arterial,2.0,3.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,...,0,1000000,SNS,S00-Avg,1.000,5500.000,WD1,W1-Weekday,1.012,5566.000000


In [30]:
#add tod factor

dfs_PeriodForecasts_Season_DOW_TOD_wGroups = []

#for (idx, row) in dfs_PeriodForecasts.iterrows():

for df_PeriodForecasts_Season_DOW in dfs_PeriodForecasts_Season_DOW: 
    
    df_PeriodForecasts_Season_DOW = df_PeriodForecasts_Season_DOW.rename(columns={namePeriod: namePeriod + '_TDM'})
    #joing forecasts data with season factors
    
    df_TODFactors_Period = df_TODFactors[df_TODFactors['HourPercentOf'] == 'Period']
    df_TODFactors_Daily  = df_TODFactors[df_TODFactors['HourPercentOf'] == 'Daily']
    
    #get peak hour as a percent of period
    df_PeriodForecasts_Season_DOW_TOD_Period = pd.DataFrame.merge(df_PeriodForecasts_Season_DOW,df_TODFactors_Period,left_on=('SiteGroupSeason','SeasonGroup','DOWGroup',namePeriod+'_TDM'),right_on=('SiteGroupSeason','SeasonGroup','DOWGroup',namePeriod))
    
    #get peak hour as a percent of daily
    df_PeriodForecasts_Season_DOW_TOD_Daily = pd.DataFrame.merge(df_PeriodForecasts_Season_DOW,df_TODFactors_Daily,left_on=('SiteGroupSeason','SeasonGroup','DOWGroup',namePeriod+'_TDM'),right_on=('SiteGroupSeason','SeasonGroup','DOWGroup','HourPercentOf'))
 
    df_PeriodForecasts_Season_DOW_TOD = pd.concat([df_PeriodForecasts_Season_DOW_TOD_Period, df_PeriodForecasts_Season_DOW_TOD_Daily])

    #display(df_PeriodForecasts_Season_DOW_TOD)
    
    #filter by vc groups
    df_PeriodForecasts_Season_DOW_TOD_wGroups = pd.DataFrame.merge(df_PeriodForecasts_Season_DOW_TOD,df_VCGroups,on=('SeasonGroup','DOWGroup','HourPercentOf','Period'))
    
    dfs_PeriodForecasts_Season_DOW_TOD_wGroups.append(df_PeriodForecasts_Season_DOW_TOD_wGroups)
    
    display(df_PeriodForecasts_Season_DOW_TOD_wGroups)


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode
0,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
1,0006_146.9,Arterial,1.0,2.0,D2,AM,127.20,0.063679,0.044811,0.091353,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
2,0006_149.9,Arterial,1.0,2.0,D1,AM,131.10,0.176201,0.145690,0.086769,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
3,0006_149.9,Arterial,1.0,2.0,D2,AM,112.40,0.201957,0.169039,0.074393,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
4,0006_150.6,Arterial,1.0,2.0,D1,AM,155.40,0.160232,0.131918,0.090454,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789781,1084_001.0,Arterial,1.0,5.0,D2,Daily,365.68,0.218442,0.123824,0.400000,...,140.0,WD0,7-Sunday,0.937,131.18,Daily,Daily,0.100282,Season,AvgSuDY
789782,2865_001.3,Arterial,2.0,4.0,D1,Daily,1397.52,0.016400,0.009274,0.600000,...,0.0,WD1,7-Sunday,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY
789783,2865_001.3,Arterial,2.0,4.0,D2,Daily,931.68,0.016400,0.009274,0.400000,...,0.0,WD1,7-Sunday,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY
789784,3004_000.9,Arterial,2.0,4.0,D1,Daily,133.26,0.072040,0.025664,0.600000,...,0.0,WD1,7-Sunday,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode
0,0006_146.9,Arterial,1.0,2.0,D1,AM,126.00,0.067460,0.049206,0.085960,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
1,0006_146.9,Arterial,1.0,2.0,D2,AM,139.40,0.060976,0.044476,0.095102,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
2,0006_149.9,Arterial,1.0,2.0,D1,AM,143.50,0.179094,0.144251,0.087880,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
3,0006_149.9,Arterial,1.0,2.0,D2,AM,121.50,0.204115,0.167901,0.074407,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
4,0006_150.6,Arterial,1.0,2.0,D1,AM,169.60,0.163915,0.130307,0.091770,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805285,1084_001.0,Arterial,1.0,5.0,D2,Daily,353.68,0.245646,0.139222,0.400000,...,160.0,WD0,7-Sunday,0.937,149.92,Daily,Daily,0.100282,Season,AvgSuDY
805286,2865_001.3,Arterial,2.0,4.0,D1,Daily,1514.22,0.017197,0.009906,0.600000,...,0.0,WD1,7-Sunday,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY
805287,2865_001.3,Arterial,2.0,4.0,D2,Daily,1009.48,0.017197,0.009906,0.400000,...,0.0,WD1,7-Sunday,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY
805288,3004_000.9,Arterial,2.0,4.0,D1,Daily,331.44,0.098117,0.060825,0.600000,...,0.0,WD1,7-Sunday,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode
0,0006_146.9,Arterial,1.0,2.0,D1,AM,142.30,0.063247,0.046381,0.091276,...,0.0,WD0,W1-Weekday,0.938,0.0,AM,Period,0.413058,Month,JanWkAM
1,0006_146.9,Arterial,1.0,2.0,D2,AM,145.10,0.062026,0.045486,0.093072,...,0.0,WD0,W1-Weekday,0.938,0.0,AM,Period,0.413058,Month,JanWkAM
2,0006_149.9,Arterial,1.0,2.0,D1,AM,144.70,0.179682,0.143055,0.082427,...,0.0,WD0,W1-Weekday,0.938,0.0,AM,Period,0.413058,Month,JanWkAM
3,0006_149.9,Arterial,1.0,2.0,D2,AM,146.70,0.176551,0.141786,0.083566,...,0.0,WD0,W1-Weekday,0.938,0.0,AM,Period,0.413058,Month,JanWkAM
4,0006_150.6,Arterial,1.0,2.0,D1,AM,169.70,0.165586,0.132587,0.086112,...,0.0,WD0,W1-Weekday,0.938,0.0,AM,Period,0.413058,Month,JanWkAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832417,1084_001.0,Arterial,1.0,5.0,D2,Daily,311.84,0.273345,0.161878,0.400000,...,200.0,WD0,7-Sunday,0.937,187.4,Daily,Daily,0.100282,Season,AvgSuDY
832418,2865_001.3,Arterial,2.0,4.0,D1,Daily,1669.02,0.016752,0.009850,0.600000,...,0.0,WD1,7-Sunday,0.773,0.0,Daily,Daily,0.100282,Season,AvgSuDY
832419,2865_001.3,Arterial,2.0,4.0,D2,Daily,1112.68,0.016752,0.009850,0.400000,...,0.0,WD1,7-Sunday,0.773,0.0,Daily,Daily,0.100282,Season,AvgSuDY
832420,3004_000.9,Arterial,2.0,3.0,D1,Daily,1407.36,0.080278,0.027370,0.600000,...,0.0,WD1,7-Sunday,0.773,0.0,Daily,Daily,0.100282,Season,AvgSuDY


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode
0,0006_146.9,Arterial,1.0,2.0,D1,AM,163.30,0.059400,0.044091,0.096920,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
1,0006_146.9,Arterial,1.0,2.0,D2,AM,149.20,0.065013,0.048257,0.088551,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
2,0006_149.9,Arterial,1.0,2.0,D1,AM,170.60,0.181125,0.151231,0.076489,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
3,0006_149.9,Arterial,1.0,2.0,D2,AM,189.60,0.161392,0.136603,0.085007,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
4,0006_150.6,Arterial,1.0,2.0,D1,AM,195.30,0.168971,0.141321,0.079503,...,0.0,WD0,W1-Weekday,0.938,0.00,AM,Period,0.413058,Month,JanWkAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863629,1084_001.0,Arterial,1.0,5.0,D2,Daily,357.04,0.285010,0.222832,0.400000,...,280.0,WD0,7-Sunday,0.937,262.36,Daily,Daily,0.100282,Season,AvgSuDY
863630,2865_001.3,Arterial,2.0,4.0,D1,Daily,1913.10,0.018065,0.011040,0.600000,...,0.0,WD1,7-Sunday,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY
863631,2865_001.3,Arterial,2.0,4.0,D2,Daily,1275.40,0.018065,0.011040,0.400000,...,0.0,WD1,7-Sunday,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY
863632,3004_000.9,Arterial,3.0,3.0,D1,Daily,2380.02,0.096655,0.027958,0.600000,...,0.0,WD2,7-Sunday,0.566,0.00,Daily,Daily,0.100282,Season,AvgSuDY


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode
0,0006_146.9,Arterial,1.0,2.0,D1,AM,194.50,0.053985,0.039589,0.107346,...,183.375,WD0,W1-Weekday,0.938,172.00575,AM,Period,0.413058,Month,JanWkAM
1,0006_146.9,Arterial,1.0,2.0,D2,AM,145.90,0.071967,0.052776,0.080523,...,137.735,WD0,W1-Weekday,0.938,129.19543,AM,Period,0.413058,Month,JanWkAM
2,0147_003.1,Arterial,1.0,3.0,D1,AM,63.20,0.231013,0.356013,0.071477,...,70.090,WD0,W1-Weekday,0.938,65.74442,AM,Period,0.413058,Month,JanWkAM
3,0147_003.1,Arterial,1.0,3.0,D2,AM,61.30,0.213703,0.339315,0.069328,...,67.645,WD0,W1-Weekday,0.938,63.45101,AM,Period,0.413058,Month,JanWkAM
4,0147_003.9,Arterial,1.0,3.0,D1,AM,61.20,0.230392,0.354575,0.072831,...,88.835,WD0,W1-Weekday,0.938,83.32723,AM,Period,0.413058,Month,JanWkAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890557,1084_001.0,Arterial,1.0,5.0,D2,Daily,259.04,0.208616,0.174336,0.400000,...,320.000,WD0,7-Sunday,0.937,299.84000,Daily,Daily,0.100282,Season,AvgSuDY
890558,2865_001.3,Arterial,2.0,4.0,D1,Daily,2150.16,0.017134,0.011441,0.600000,...,2280.000,WD1,7-Sunday,0.773,1762.44000,Daily,Daily,0.100282,Season,AvgSuDY
890559,2865_001.3,Arterial,2.0,4.0,D2,Daily,1433.44,0.017134,0.011441,0.400000,...,1520.000,WD1,7-Sunday,0.773,1174.96000,Daily,Daily,0.100282,Season,AvgSuDY
890560,3004_000.9,Arterial,3.0,3.0,D1,Daily,9789.30,0.145800,0.041690,0.600000,...,21300.000,WD2,7-Sunday,0.566,12055.80000,Daily,Daily,0.100282,Season,AvgSuDY


...

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode
0,0006_146.9,Arterial,1.0,2.0,D1,AM,144.20,0.068655,0.049931,0.086358,...,0.000,WD0,W1-Weekday,0.938,0.00000,AM,Period,0.413058,Month,JanWkAM
1,0006_146.9,Arterial,1.0,2.0,D2,AM,159.20,0.062186,0.045226,0.095341,...,0.000,WD0,W1-Weekday,0.938,0.00000,AM,Period,0.413058,Month,JanWkAM
2,0006_149.9,Arterial,1.0,2.0,D1,AM,436.00,0.109174,0.078211,0.140695,...,99.430,WD0,W1-Weekday,0.938,93.26534,AM,Period,0.413058,Month,JanWkAM
3,0006_149.9,Arterial,1.0,2.0,D2,AM,166.60,0.226291,0.179472,0.053761,...,38.305,WD0,W1-Weekday,0.938,35.93009,AM,Period,0.413058,Month,JanWkAM
4,0006_150.6,Arterial,1.0,2.0,D1,AM,466.10,0.107702,0.077666,0.139039,...,101.875,WD0,W1-Weekday,0.938,95.55875,AM,Period,0.413058,Month,JanWkAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804877,1084_001.0,Arterial,1.0,5.0,D2,Daily,350.36,0.285992,0.206645,0.400000,...,273.000,WD0,7-Sunday,0.937,255.80100,Daily,Daily,0.100282,Season,AvgSuDY
804878,2865_001.3,Arterial,2.0,4.0,D1,Daily,1895.46,0.020259,0.009243,0.600000,...,0.000,WD1,7-Sunday,0.773,0.00000,Daily,Daily,0.100282,Season,AvgSuDY
804879,2865_001.3,Arterial,2.0,4.0,D2,Daily,1263.64,0.020259,0.009243,0.400000,...,0.000,WD1,7-Sunday,0.773,0.00000,Daily,Daily,0.100282,Season,AvgSuDY
804880,3004_000.9,Arterial,3.0,4.0,D1,Daily,430.08,0.078125,0.027344,0.600000,...,0.000,WD2,7-Sunday,0.566,0.00000,Daily,Daily,0.100282,Season,AvgSuDY


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode
0,0006_146.9,Arterial,1.0,2.0,D1,AM,156.10,0.067905,0.049327,0.086959,...,147.515,WD0,W1-Weekday,0.938,138.36907,AM,Period,0.413058,Month,JanWkAM
1,0006_146.9,Arterial,1.0,2.0,D2,AM,171.10,0.061952,0.045003,0.095315,...,162.185,WD0,W1-Weekday,0.938,152.12953,AM,Period,0.413058,Month,JanWkAM
2,0141_000.0,Arterial,1.0,3.0,D1,AM,285.50,0.135552,0.096322,0.143223,...,180.930,WD0,W1-Weekday,0.938,169.71234,AM,Period,0.413058,Month,JanWkAM
3,0141_000.0,Arterial,1.0,3.0,D2,AM,106.90,0.260992,0.218896,0.053627,...,67.645,WD0,W1-Weekday,0.938,63.45101,AM,Period,0.413058,Month,JanWkAM
4,0141_000.9,Arterial,1.0,3.0,D1,AM,332.10,0.128576,0.088829,0.134036,...,213.530,WD0,W1-Weekday,0.938,200.29114,AM,Period,0.413058,Month,JanWkAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804877,1084_001.0,Arterial,1.0,5.0,D2,Daily,297.88,0.212837,0.181550,0.400000,...,359.000,WD0,7-Sunday,0.937,336.38300,Daily,Daily,0.100282,Season,AvgSuDY
804878,2865_001.3,Arterial,2.0,4.0,D1,Daily,2126.34,0.019357,0.010779,0.600000,...,2256.000,WD1,7-Sunday,0.773,1743.88800,Daily,Daily,0.100282,Season,AvgSuDY
804879,2865_001.3,Arterial,2.0,4.0,D2,Daily,1417.56,0.019357,0.010779,0.400000,...,1504.000,WD1,7-Sunday,0.773,1162.59200,Daily,Daily,0.100282,Season,AvgSuDY
804880,3004_000.9,Arterial,3.0,4.0,D1,Daily,467.10,0.077200,0.025305,0.600000,...,11978.000,WD2,7-Sunday,0.566,6779.54800,Daily,Daily,0.100282,Season,AvgSuDY


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode
0,0006_146.9,Arterial,1.0,2.0,D1,AM,142.60,0.063114,0.046283,0.091457,...,0.0,WD0,W1-Weekday,0.938,0.000,AM,Period,0.413058,Month,JanWkAM
1,0006_146.9,Arterial,1.0,2.0,D2,AM,145.10,0.062026,0.045486,0.093061,...,0.0,WD0,W1-Weekday,0.938,0.000,AM,Period,0.413058,Month,JanWkAM
2,0006_149.9,Arterial,1.0,2.0,D1,AM,145.10,0.179187,0.142660,0.082579,...,0.0,WD0,W1-Weekday,0.938,0.000,AM,Period,0.413058,Month,JanWkAM
3,0006_149.9,Arterial,1.0,2.0,D2,AM,146.70,0.176551,0.141786,0.083490,...,0.0,WD0,W1-Weekday,0.938,0.000,AM,Period,0.413058,Month,JanWkAM
4,0006_150.6,Arterial,1.0,2.0,D1,AM,169.70,0.165586,0.132587,0.086081,...,0.0,WD0,W1-Weekday,0.938,0.000,AM,Period,0.413058,Month,JanWkAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834457,1084_001.0,Arterial,1.0,5.0,D2,Daily,326.16,0.261467,0.155016,0.400000,...,214.0,WD0,7-Sunday,0.937,200.518,Daily,Daily,0.100282,Season,AvgSuDY
834458,2865_001.3,Arterial,2.0,4.0,D1,Daily,1670.22,0.016740,0.009843,0.600000,...,1.0,WD1,7-Sunday,0.773,0.773,Daily,Daily,0.100282,Season,AvgSuDY
834459,2865_001.3,Arterial,2.0,4.0,D2,Daily,1113.48,0.016740,0.009843,0.400000,...,1.0,WD1,7-Sunday,0.773,0.773,Daily,Daily,0.100282,Season,AvgSuDY
834460,3004_000.9,Arterial,2.0,3.0,D1,Daily,1387.20,0.080796,0.026773,0.600000,...,0.0,WD1,7-Sunday,0.773,0.000,Daily,Daily,0.100282,Season,AvgSuDY


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode
0,0006_146.9,Arterial,1.0,2.0,D1,AM,163.50,0.059327,0.044037,0.096987,...,0.0,WD0,W1-Weekday,0.938,0.000,AM,Period,0.413058,Month,JanWkAM
1,0006_146.9,Arterial,1.0,2.0,D2,AM,149.20,0.065013,0.048257,0.088504,...,0.0,WD0,W1-Weekday,0.938,0.000,AM,Period,0.413058,Month,JanWkAM
2,0006_149.9,Arterial,1.0,2.0,D1,AM,166.80,0.180456,0.153477,0.079015,...,0.0,WD0,W1-Weekday,0.938,0.000,AM,Period,0.413058,Month,JanWkAM
3,0006_149.9,Arterial,1.0,2.0,D2,AM,180.00,0.165000,0.142222,0.085268,...,0.0,WD0,W1-Weekday,0.938,0.000,AM,Period,0.413058,Month,JanWkAM
4,0006_150.6,Arterial,1.0,2.0,D1,AM,192.60,0.167186,0.142783,0.082297,...,0.0,WD0,W1-Weekday,0.938,0.000,AM,Period,0.413058,Month,JanWkAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867607,1084_001.0,Arterial,1.0,5.0,D2,Daily,368.72,0.287047,0.225754,0.400000,...,292.0,WD0,7-Sunday,0.937,273.604,Daily,Daily,0.100282,Season,AvgSuDY
867608,2865_001.3,Arterial,2.0,4.0,D1,Daily,1915.38,0.018043,0.011027,0.600000,...,2.0,WD1,7-Sunday,0.773,1.546,Daily,Daily,0.100282,Season,AvgSuDY
867609,2865_001.3,Arterial,2.0,4.0,D2,Daily,1276.92,0.018043,0.011027,0.400000,...,2.0,WD1,7-Sunday,0.773,1.546,Daily,Daily,0.100282,Season,AvgSuDY
867610,3004_000.9,Arterial,3.0,3.0,D1,Daily,3062.70,0.099677,0.032285,0.600000,...,683.0,WD2,7-Sunday,0.566,386.578,Daily,Daily,0.100282,Season,AvgSuDY


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode
0,0006_146.9,Arterial,1.0,2.0,D1,AM,195.50,0.053708,0.039386,0.107809,...,185.005,WD0,W1-Weekday,0.938,173.53469,AM,Period,0.413058,Month,JanWkAM
1,0006_146.9,Arterial,1.0,2.0,D2,AM,146.10,0.071869,0.052704,0.080567,...,137.735,WD0,W1-Weekday,0.938,129.19543,AM,Period,0.413058,Month,JanWkAM
2,0147_003.1,Arterial,1.0,3.0,D1,AM,62.40,0.230769,0.355769,0.071691,...,69.275,WD0,W1-Weekday,0.938,64.97995,AM,Period,0.413058,Month,JanWkAM
3,0147_003.1,Arterial,1.0,3.0,D2,AM,60.30,0.213930,0.341625,0.069278,...,66.830,WD0,W1-Weekday,0.938,62.68654,AM,Period,0.413058,Month,JanWkAM
4,0147_003.9,Arterial,1.0,3.0,D1,AM,61.10,0.229133,0.351882,0.072929,...,88.835,WD0,W1-Weekday,0.938,83.32723,AM,Period,0.413058,Month,JanWkAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890761,1084_001.0,Arterial,1.0,5.0,D2,Daily,229.04,0.202934,0.162592,0.400000,...,290.000,WD0,7-Sunday,0.937,271.73000,Daily,Daily,0.100282,Season,AvgSuDY
890762,2865_001.3,Arterial,2.0,4.0,D1,Daily,2150.16,0.017134,0.011441,0.600000,...,2280.000,WD1,7-Sunday,0.773,1762.44000,Daily,Daily,0.100282,Season,AvgSuDY
890763,2865_001.3,Arterial,2.0,4.0,D2,Daily,1433.44,0.017134,0.011441,0.400000,...,1520.000,WD1,7-Sunday,0.773,1174.96000,Daily,Daily,0.100282,Season,AvgSuDY
890764,3004_000.9,Arterial,3.0,3.0,D1,Daily,5086.44,0.077937,0.026411,0.600000,...,16597.000,WD2,7-Sunday,0.566,9393.90200,Daily,Daily,0.100282,Season,AvgSuDY


In [31]:
df_PeriodForecasts_Season_DOW_TOD_Daily[(df_PeriodForecasts_Season_DOW_TOD_Daily['SEGID']=='0006_146.9') & (df_PeriodForecasts_Season_DOW_TOD_Daily['DOWGroup']=='6-Saturday')].head(20)

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent
1000,0006_146.9,Arterial,1.0,2.0,D1,Daily,1088.04,0.078968,0.043234,0.6,...,M01-Jan,0.815,1027.715,WD0,6-Saturday,1.136,1167.484240,AM,Daily,0.049787
1001,0006_146.9,Arterial,1.0,2.0,D1,Daily,1088.04,0.078968,0.043234,0.6,...,M01-Jan,0.815,1027.715,WD0,6-Saturday,1.136,1167.484240,Daily,Daily,0.089609
1002,0006_146.9,Arterial,1.0,2.0,D1,Daily,1088.04,0.078968,0.043234,0.6,...,M01-Jan,0.815,1027.715,WD0,6-Saturday,1.136,1167.484240,EV,Daily,0.058250
1003,0006_146.9,Arterial,1.0,2.0,D1,Daily,1088.04,0.078968,0.043234,0.6,...,M01-Jan,0.815,1027.715,WD0,6-Saturday,1.136,1167.484240,MD,Daily,0.082687
1004,0006_146.9,Arterial,1.0,2.0,D1,Daily,1088.04,0.078968,0.043234,0.6,...,M01-Jan,0.815,1027.715,WD0,6-Saturday,1.136,1167.484240,PM,Daily,0.085296
1005,0006_146.9,Arterial,1.0,2.0,D2,Daily,725.36,0.078968,0.043234,0.4,...,M01-Jan,0.815,685.415,WD0,6-Saturday,1.136,778.631440,AM,Daily,0.049787
1006,0006_146.9,Arterial,1.0,2.0,D2,Daily,725.36,0.078968,0.043234,0.4,...,M01-Jan,0.815,685.415,WD0,6-Saturday,1.136,778.631440,Daily,Daily,0.089609
1007,0006_146.9,Arterial,1.0,2.0,D2,Daily,725.36,0.078968,0.043234,0.4,...,M01-Jan,0.815,685.415,WD0,6-Saturday,1.136,778.631440,EV,Daily,0.058250
1008,0006_146.9,Arterial,1.0,2.0,D2,Daily,725.36,0.078968,0.043234,0.4,...,M01-Jan,0.815,685.415,WD0,6-Saturday,1.136,778.631440,MD,Daily,0.082687
1009,0006_146.9,Arterial,1.0,2.0,D2,Daily,725.36,0.078968,0.043234,0.4,...,M01-Jan,0.815,685.415,WD0,6-Saturday,1.136,778.631440,PM,Daily,0.085296


In [32]:
df_PeriodForecasts_Season_DOW_TOD[df_PeriodForecasts_Season_DOW_TOD['SEGID']=='0006_146.9'].head(20)

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent
0,0006_146.9,Arterial,1.0,2.0,D1,AM,195.5,0.053708,0.039386,0.107809,...,M01-Jan,0.815,185.005,WD0,1-Monday,0.907,167.799535,AM,Period,0.411987
1,0006_146.9,Arterial,1.0,2.0,D2,AM,146.1,0.071869,0.052704,0.080567,...,M01-Jan,0.815,137.735,WD0,1-Monday,0.907,124.925645,AM,Period,0.411987
38,0006_146.9,Arterial,1.0,2.0,D1,AM,195.5,0.053708,0.039386,0.107809,...,M01-Jan,0.815,185.005,WD0,2-Tuesday,0.902,166.874510,AM,Period,0.406506
39,0006_146.9,Arterial,1.0,2.0,D2,AM,146.1,0.071869,0.052704,0.080567,...,M01-Jan,0.815,137.735,WD0,2-Tuesday,0.902,124.236970,AM,Period,0.406506
76,0006_146.9,Arterial,1.0,2.0,D1,AM,195.5,0.053708,0.039386,0.107809,...,M01-Jan,0.815,185.005,WD0,3-Wednesday,0.943,174.459715,AM,Period,0.416881
77,0006_146.9,Arterial,1.0,2.0,D2,AM,146.1,0.071869,0.052704,0.080567,...,M01-Jan,0.815,137.735,WD0,3-Wednesday,0.943,129.884105,AM,Period,0.416881
114,0006_146.9,Arterial,1.0,2.0,D1,AM,195.5,0.053708,0.039386,0.107809,...,M01-Jan,0.815,185.005,WD0,4-Thursday,0.995,184.079975,AM,Period,0.416574
115,0006_146.9,Arterial,1.0,2.0,D2,AM,146.1,0.071869,0.052704,0.080567,...,M01-Jan,0.815,137.735,WD0,4-Thursday,0.995,137.046325,AM,Period,0.416574
152,0006_146.9,Arterial,1.0,2.0,D1,AM,195.5,0.053708,0.039386,0.107809,...,M01-Jan,0.815,185.005,WD0,5-Friday,1.170,216.455850,AM,Period,0.411742
153,0006_146.9,Arterial,1.0,2.0,D2,AM,146.1,0.071869,0.052704,0.080567,...,M01-Jan,0.815,137.735,WD0,5-Friday,1.170,161.149950,AM,Period,0.411742


In [33]:
#dftemp = dfs_PeriodForecasts_Season_DOW_TOD_wGroups[0].groupby(['VCGroupCode']).agg({'VolMaxHourPercent':[np.size]})
df_TODFactors_Daily

,SiteGroupSeason,SeasonGroup,DOWGroup,Period,HourPercentOf,VolMaxHourPercent
24480,SN0,M01-Jan,1-Monday,AM,Daily,0.045409
24481,SN0,M01-Jan,1-Monday,Daily,Daily,0.135094
24482,SN0,M01-Jan,1-Monday,EV,Daily,0.044906
24483,SN0,M01-Jan,1-Monday,MD,Daily,0.119119
24484,SN0,M01-Jan,1-Monday,PM,Daily,0.123019
...,...,...,...,...,...,...
58135,XX4,S04-Fal,W2-Weekend,AM,Daily,0.061062
58136,XX4,S04-Fal,W2-Weekend,Daily,Daily,0.087213
58137,XX4,S04-Fal,W2-Weekend,EV,Daily,0.062317
58138,XX4,S04-Fal,W2-Weekend,MD,Daily,0.065813


In [34]:
df = dfs_PeriodForecasts_Season_DOW_TOD_wGroups[4]

df = df[(df[nameSegID]=='0085_011.3')]

df = df.groupby([nameSegID,'SeasonGroup','DOWGroup',namePeriod,'HourPercentOf']).agg({'VOL':[np.sum],'VolPercentSegDY':[np.sum]})

display(df)

VOL  \
                                                             sum   
SEGID      SeasonGroup DOWGroup   Period HourPercentOf             
0085_011.3 M01-Jan     5-Friday   PM     Daily          145295.6   
                       6-Saturday Daily  Daily          145295.6   
                       7-Sunday   Daily  Daily          145295.6   
                       W1-Weekday AM     Period          29327.9   
                                  MD     Period          45625.5   
                                  PM     Period          40296.8   
           M02-Feb     5-Friday   PM     Daily          145295.6   
                       6-Saturday Daily  Daily          145295.6   
                       7-Sunday   Daily  Daily          145295.6   
                       W1-Weekday AM     Period          29327.9   
                                  MD     Period          45625.5   
                                  PM     Period          40296.8   
           M03-Mar     5-Friday   PM     Daily          145295.6   
                       6-Saturday Daily  Daily          145295.6   
                       7-Sunday   Daily  Daily          145295.6   
                       W1-Weekday AM     Period          29327.9   
                                  MD     Period          45625.5   
                                  PM     Period          40296.8   
           M04-Apr     5-Friday   PM     Daily          145295.6   
                       6-Saturday Daily  Daily          145295.6   
                       7-Sunday   Daily  Daily          145295.6   
                       W1-Weekday AM     Period          29327.9   
                                  MD     Period          45625.5   
                                  PM     Period          40296.8   
           M05-May     5-Friday   PM     Daily          145295.6   
                       6-Saturday Daily  Daily          145295.6   
                       7-Sunday   Daily  Daily          145295.6   
                       W1-Weekday AM     Period          29327.9   
                                  MD     Period          45625.5   
                                  PM     Period          40296.8   
           M06-Jun     5-Friday   PM     Daily          145295.6   
                       6-Saturday Daily  Daily          145295.6   
                       7-Sunday   Daily  Daily          145295.6   
                       W1-Weekday AM     Period          29327.9   
                                  MD     Period          45625.5   
                                  PM     Period          40296.8   
           M07-Jul     5-Friday   PM     Daily          145295.6   
                       6-Saturday Daily  Daily          145295.6   
                       7-Sunday   Daily  Daily          145295.6   
                       W1-Weekday AM     Period          29327.9   
                                  MD     Period          45625.5   
                                  PM     Period          40296.8   
           M08-Aug     5-Friday   PM     Daily          145295.6   
                       6-Saturday Daily  Daily          145295.6   
                       7-Sunday   Daily  Daily          145295.6   
                       W1-Weekday AM     Period          29327.9   
                                  MD     Period          45625.5   
                                  PM     Period          40296.8   
           M09-Sep     5-Friday   PM     Daily          145295.6   
                       6-Saturday Daily  Daily          145295.6   
                       7-Sunday   Daily  Daily          145295.6   
                       W1-Weekday AM     Period          29327.9   
                                  MD     Period          45625.5   
                                  PM     Period          40296.8   
           M10-Oct     5-Friday   PM     Daily          145295.6   
                       6-Saturday Daily  Daily          145295.6   
                       7-Sunday   Daily  Daily   

In [35]:
dfs_PeakHourForecasts = []

for df_PeriodForecasts_Season_DOW_TOD_wGroups in dfs_PeriodForecasts_Season_DOW_TOD_wGroups: 
    
    df_PeakHourForecasts = df_PeriodForecasts_Season_DOW_TOD_wGroups

    df_PeakHourForecasts['ForecastPeakHour'] = df_PeakHourForecasts['ForecastASeasonDOWPeriodT'] * df_PeakHourForecasts['VolMaxHourPercent']

    #add truck percentage adjustments
    #PCE for medium and heavy trucks from HCM 2000 Exhibit 23-8
    Emd = 1.2 #same as RVs
    Ehv = 1.5
    df_PeakHourForecasts['FactorTrucks'] = 1 + (df_PeakHourForecasts[namePercentTrucksMD] * (Emd-1)) + (df_PeakHourForecasts[namePercentTrucksHV] * (Ehv-1))
    
    #calculate Average Season DOW
    df_PeakHourForecasts['ForecastPeakHourPCE'] = df_PeakHourForecasts['ForecastPeakHour'] * df_PeakHourForecasts['FactorTrucks']
    
    display(df_PeakHourForecasts)
    
    dfs_PeakHourForecasts.append(df_PeakHourForecasts)

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035059,0.000000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,127.20,0.063679,0.044811,0.091353,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035142,0.000000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,131.10,0.176201,0.145690,0.086769,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.000000,1.108085,0.000000
3,0006_149.9,Arterial,1.0,2.0,D2,AM,112.40,0.201957,0.169039,0.074393,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.000000,1.124911,0.000000
4,0006_150.6,Arterial,1.0,2.0,D1,AM,155.40,0.160232,0.131918,0.090454,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.000000,1.098005,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789781,1084_001.0,Arterial,1.0,5.0,D2,Daily,365.68,0.218442,0.123824,0.400000,...,0.937,131.18,Daily,Daily,0.100282,Season,AvgSuDY,13.155039,1.105601,14.544218
789782,2865_001.3,Arterial,2.0,4.0,D1,Daily,1397.52,0.016400,0.009274,0.600000,...,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.007917,0.000000
789783,2865_001.3,Arterial,2.0,4.0,D2,Daily,931.68,0.016400,0.009274,0.400000,...,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.007917,0.000000
789784,3004_000.9,Arterial,2.0,4.0,D1,Daily,133.26,0.072040,0.025664,0.600000,...,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.027240,0.000000


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,126.00,0.067460,0.049206,0.085960,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.00000,1.038095,0.000000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,139.40,0.060976,0.044476,0.095102,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.00000,1.034433,0.000000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,143.50,0.179094,0.144251,0.087880,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.00000,1.107944,0.000000
3,0006_149.9,Arterial,1.0,2.0,D2,AM,121.50,0.204115,0.167901,0.074407,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.00000,1.124774,0.000000
4,0006_150.6,Arterial,1.0,2.0,D1,AM,169.60,0.163915,0.130307,0.091770,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.00000,1.097936,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805285,1084_001.0,Arterial,1.0,5.0,D2,Daily,353.68,0.245646,0.139222,0.400000,...,0.937,149.92,Daily,Daily,0.100282,Season,AvgSuDY,15.03433,1.118740,16.819508
805286,2865_001.3,Arterial,2.0,4.0,D1,Daily,1514.22,0.017197,0.009906,0.600000,...,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY,0.00000,1.008392,0.000000
805287,2865_001.3,Arterial,2.0,4.0,D2,Daily,1009.48,0.017197,0.009906,0.400000,...,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY,0.00000,1.008392,0.000000
805288,3004_000.9,Arterial,2.0,4.0,D1,Daily,331.44,0.098117,0.060825,0.600000,...,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY,0.00000,1.050036,0.000000


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,142.30,0.063247,0.046381,0.091276,...,0.938,0.0,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035840,0.000000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,145.10,0.062026,0.045486,0.093072,...,0.938,0.0,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035148,0.000000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,144.70,0.179682,0.143055,0.082427,...,0.938,0.0,AM,Period,0.413058,Month,JanWkAM,0.000000,1.107464,0.000000
3,0006_149.9,Arterial,1.0,2.0,D2,AM,146.70,0.176551,0.141786,0.083566,...,0.938,0.0,AM,Period,0.413058,Month,JanWkAM,0.000000,1.106203,0.000000
4,0006_150.6,Arterial,1.0,2.0,D1,AM,169.70,0.165586,0.132587,0.086112,...,0.938,0.0,AM,Period,0.413058,Month,JanWkAM,0.000000,1.099411,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832417,1084_001.0,Arterial,1.0,5.0,D2,Daily,311.84,0.273345,0.161878,0.400000,...,0.937,187.4,Daily,Daily,0.100282,Season,AvgSuDY,18.792913,1.135608,21.341382
832418,2865_001.3,Arterial,2.0,4.0,D1,Daily,1669.02,0.016752,0.009850,0.600000,...,0.773,0.0,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.008276,0.000000
832419,2865_001.3,Arterial,2.0,4.0,D2,Daily,1112.68,0.016752,0.009850,0.400000,...,0.773,0.0,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.008276,0.000000
832420,3004_000.9,Arterial,2.0,3.0,D1,Daily,1407.36,0.080278,0.027370,0.600000,...,0.773,0.0,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.029741,0.000000


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,163.30,0.059400,0.044091,0.096920,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.000000,1.033925,0.000000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,149.20,0.065013,0.048257,0.088551,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.000000,1.037131,0.000000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,170.60,0.181125,0.151231,0.076489,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.000000,1.111841,0.000000
3,0006_149.9,Arterial,1.0,2.0,D2,AM,189.60,0.161392,0.136603,0.085007,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.000000,1.100580,0.000000
4,0006_150.6,Arterial,1.0,2.0,D1,AM,195.30,0.168971,0.141321,0.079503,...,0.938,0.00,AM,Period,0.413058,Month,JanWkAM,0.000000,1.104455,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863629,1084_001.0,Arterial,1.0,5.0,D2,Daily,357.04,0.285010,0.222832,0.400000,...,0.937,262.36,Daily,Daily,0.100282,Season,AvgSuDY,26.310078,1.168418,30.741171
863630,2865_001.3,Arterial,2.0,4.0,D1,Daily,1913.10,0.018065,0.011040,0.600000,...,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.009133,0.000000
863631,2865_001.3,Arterial,2.0,4.0,D2,Daily,1275.40,0.018065,0.011040,0.400000,...,0.773,0.00,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.009133,0.000000
863632,3004_000.9,Arterial,3.0,3.0,D1,Daily,2380.02,0.096655,0.027958,0.600000,...,0.566,0.00,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.033310,0.000000


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,194.50,0.053985,0.039589,0.107346,...,0.938,172.00575,AM,Period,0.413058,Month,JanWkAM,71.048344,1.030591,73.221803
1,0006_146.9,Arterial,1.0,2.0,D2,AM,145.90,0.071967,0.052776,0.080523,...,0.938,129.19543,AM,Period,0.413058,Month,JanWkAM,53.365201,1.040781,55.541506
2,0147_003.1,Arterial,1.0,3.0,D1,AM,63.20,0.231013,0.356013,0.071477,...,0.938,65.74442,AM,Period,0.413058,Month,JanWkAM,27.156256,1.224209,33.244929
3,0147_003.1,Arterial,1.0,3.0,D2,AM,61.30,0.213703,0.339315,0.069328,...,0.938,63.45101,AM,Period,0.413058,Month,JanWkAM,26.208945,1.212398,31.775673
4,0147_003.9,Arterial,1.0,3.0,D1,AM,61.20,0.230392,0.354575,0.072831,...,0.938,83.32723,AM,Period,0.413058,Month,JanWkAM,34.418976,1.223366,42.107005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890557,1084_001.0,Arterial,1.0,5.0,D2,Daily,259.04,0.208616,0.174336,0.400000,...,0.937,299.84000,Daily,Daily,0.100282,Season,AvgSuDY,30.068660,1.128891,33.944248
890558,2865_001.3,Arterial,2.0,4.0,D1,Daily,2150.16,0.017134,0.011441,0.600000,...,0.773,1762.44000,Daily,Daily,0.100282,Season,AvgSuDY,176.741626,1.009147,178.358322
890559,2865_001.3,Arterial,2.0,4.0,D2,Daily,1433.44,0.017134,0.011441,0.400000,...,0.773,1174.96000,Daily,Daily,0.100282,Season,AvgSuDY,117.827751,1.009147,118.905548
890560,3004_000.9,Arterial,3.0,3.0,D1,Daily,9789.30,0.145800,0.041690,0.600000,...,0.566,12055.80000,Daily,Daily,0.100282,Season,AvgSuDY,1208.983962,1.050005,1269.439459


...

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,144.20,0.068655,0.049931,0.086358,...,0.938,0.00000,AM,Period,0.413058,Month,JanWkAM,0.000000,1.038696,0.000000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,159.20,0.062186,0.045226,0.095341,...,0.938,0.00000,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035050,0.000000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,436.00,0.109174,0.078211,0.140695,...,0.938,93.26534,AM,Period,0.413058,Month,JanWkAM,38.523991,1.060940,40.871657
3,0006_149.9,Arterial,1.0,2.0,D2,AM,166.60,0.226291,0.179472,0.053761,...,0.938,35.93009,AM,Period,0.413058,Month,JanWkAM,14.841210,1.134994,16.844684
4,0006_150.6,Arterial,1.0,2.0,D1,AM,466.10,0.107702,0.077666,0.139039,...,0.938,95.55875,AM,Period,0.413058,Month,JanWkAM,39.471302,1.060373,41.854316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804877,1084_001.0,Arterial,1.0,5.0,D2,Daily,350.36,0.285992,0.206645,0.400000,...,0.937,255.80100,Daily,Daily,0.100282,Season,AvgSuDY,25.652326,1.160521,29.770052
804878,2865_001.3,Arterial,2.0,4.0,D1,Daily,1895.46,0.020259,0.009243,0.600000,...,0.773,0.00000,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.008673,0.000000
804879,2865_001.3,Arterial,2.0,4.0,D2,Daily,1263.64,0.020259,0.009243,0.400000,...,0.773,0.00000,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.008673,0.000000
804880,3004_000.9,Arterial,3.0,4.0,D1,Daily,430.08,0.078125,0.027344,0.600000,...,0.566,0.00000,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.029297,0.000000


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,156.10,0.067905,0.049327,0.086959,...,0.938,138.36907,AM,Period,0.413058,Month,JanWkAM,57.154446,1.038245,59.340301
1,0006_146.9,Arterial,1.0,2.0,D2,AM,171.10,0.061952,0.045003,0.095315,...,0.938,152.12953,AM,Period,0.413058,Month,JanWkAM,62.838314,1.034892,65.030860
2,0141_000.0,Arterial,1.0,3.0,D1,AM,285.50,0.135552,0.096322,0.143223,...,0.938,169.71234,AM,Period,0.413058,Month,JanWkAM,70.101033,1.075271,75.377640
3,0141_000.0,Arterial,1.0,3.0,D2,AM,106.90,0.260992,0.218896,0.053627,...,0.938,63.45101,AM,Period,0.413058,Month,JanWkAM,26.208945,1.161646,30.445526
4,0141_000.9,Arterial,1.0,3.0,D1,AM,332.10,0.128576,0.088829,0.134036,...,0.938,200.29114,AM,Period,0.413058,Month,JanWkAM,82.731850,1.070129,88.533792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804877,1084_001.0,Arterial,1.0,5.0,D2,Daily,297.88,0.212837,0.181550,0.400000,...,0.937,336.38300,Daily,Daily,0.100282,Season,AvgSuDY,33.733278,1.133342,38.231350
804878,2865_001.3,Arterial,2.0,4.0,D1,Daily,2126.34,0.019357,0.010779,0.600000,...,0.773,1743.88800,Daily,Daily,0.100282,Season,AvgSuDY,174.881188,1.009261,176.500760
804879,2865_001.3,Arterial,2.0,4.0,D2,Daily,1417.56,0.019357,0.010779,0.400000,...,0.773,1162.59200,Daily,Daily,0.100282,Season,AvgSuDY,116.587459,1.009261,117.667173
804880,3004_000.9,Arterial,3.0,4.0,D1,Daily,467.10,0.077200,0.025305,0.600000,...,0.566,6779.54800,Daily,Daily,0.100282,Season,AvgSuDY,679.869009,1.028092,698.968220


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,142.60,0.063114,0.046283,0.091457,...,0.938,0.000,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035764,0.000000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,145.10,0.062026,0.045486,0.093061,...,0.938,0.000,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035148,0.000000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,145.10,0.179187,0.142660,0.082579,...,0.938,0.000,AM,Period,0.413058,Month,JanWkAM,0.000000,1.107167,0.000000
3,0006_149.9,Arterial,1.0,2.0,D2,AM,146.70,0.176551,0.141786,0.083490,...,0.938,0.000,AM,Period,0.413058,Month,JanWkAM,0.000000,1.106203,0.000000
4,0006_150.6,Arterial,1.0,2.0,D1,AM,169.70,0.165586,0.132587,0.086081,...,0.938,0.000,AM,Period,0.413058,Month,JanWkAM,0.000000,1.099411,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834457,1084_001.0,Arterial,1.0,5.0,D2,Daily,326.16,0.261467,0.155016,0.400000,...,0.937,200.518,Daily,Daily,0.100282,Season,AvgSuDY,20.108416,1.129801,22.718515
834458,2865_001.3,Arterial,2.0,4.0,D1,Daily,1670.22,0.016740,0.009843,0.600000,...,0.773,0.773,Daily,Daily,0.100282,Season,AvgSuDY,0.077518,1.008270,0.078159
834459,2865_001.3,Arterial,2.0,4.0,D2,Daily,1113.48,0.016740,0.009843,0.400000,...,0.773,0.773,Daily,Daily,0.100282,Season,AvgSuDY,0.077518,1.008270,0.078159
834460,3004_000.9,Arterial,2.0,3.0,D1,Daily,1387.20,0.080796,0.026773,0.600000,...,0.773,0.000,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.029546,0.000000


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,163.50,0.059327,0.044037,0.096987,...,0.938,0.000,AM,Period,0.413058,Month,JanWkAM,0.000000,1.033884,0.000000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,149.20,0.065013,0.048257,0.088504,...,0.938,0.000,AM,Period,0.413058,Month,JanWkAM,0.000000,1.037131,0.000000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,166.80,0.180456,0.153477,0.079015,...,0.938,0.000,AM,Period,0.413058,Month,JanWkAM,0.000000,1.112830,0.000000
3,0006_149.9,Arterial,1.0,2.0,D2,AM,180.00,0.165000,0.142222,0.085268,...,0.938,0.000,AM,Period,0.413058,Month,JanWkAM,0.000000,1.104111,0.000000
4,0006_150.6,Arterial,1.0,2.0,D1,AM,192.60,0.167186,0.142783,0.082297,...,0.938,0.000,AM,Period,0.413058,Month,JanWkAM,0.000000,1.104829,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867607,1084_001.0,Arterial,1.0,5.0,D2,Daily,368.72,0.287047,0.225754,0.400000,...,0.937,273.604,Daily,Daily,0.100282,Season,AvgSuDY,27.437652,1.170286,32.109911
867608,2865_001.3,Arterial,2.0,4.0,D1,Daily,1915.38,0.018043,0.011027,0.600000,...,0.773,1.546,Daily,Daily,0.100282,Season,AvgSuDY,0.155037,1.009122,0.156451
867609,2865_001.3,Arterial,2.0,4.0,D2,Daily,1276.92,0.018043,0.011027,0.400000,...,0.773,1.546,Daily,Daily,0.100282,Season,AvgSuDY,0.155037,1.009122,0.156451
867610,3004_000.9,Arterial,3.0,3.0,D1,Daily,3062.70,0.099677,0.032285,0.600000,...,0.566,386.578,Daily,Daily,0.100282,Season,AvgSuDY,38.766951,1.036078,40.165583


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,195.50,0.053708,0.039386,0.107809,...,0.938,173.53469,AM,Period,0.413058,Month,JanWkAM,71.679885,1.030435,73.861447
1,0006_146.9,Arterial,1.0,2.0,D2,AM,146.10,0.071869,0.052704,0.080567,...,0.938,129.19543,AM,Period,0.413058,Month,JanWkAM,53.365201,1.040726,55.538527
2,0147_003.1,Arterial,1.0,3.0,D1,AM,62.40,0.230769,0.355769,0.071691,...,0.938,64.97995,AM,Period,0.413058,Month,JanWkAM,26.840486,1.224038,32.853787
3,0147_003.1,Arterial,1.0,3.0,D2,AM,60.30,0.213930,0.341625,0.069278,...,0.938,62.68654,AM,Period,0.413058,Month,JanWkAM,25.893174,1.213599,31.423922
4,0147_003.9,Arterial,1.0,3.0,D1,AM,61.10,0.229133,0.351882,0.072929,...,0.938,83.32723,AM,Period,0.413058,Month,JanWkAM,34.418976,1.221768,42.051989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890761,1084_001.0,Arterial,1.0,5.0,D2,Daily,229.04,0.202934,0.162592,0.400000,...,0.937,271.73000,Daily,Daily,0.100282,Season,AvgSuDY,27.249723,1.121883,30.570991
890762,2865_001.3,Arterial,2.0,4.0,D1,Daily,2150.16,0.017134,0.011441,0.600000,...,0.773,1762.44000,Daily,Daily,0.100282,Season,AvgSuDY,176.741626,1.009147,178.358322
890763,2865_001.3,Arterial,2.0,4.0,D2,Daily,1433.44,0.017134,0.011441,0.400000,...,0.773,1174.96000,Daily,Daily,0.100282,Season,AvgSuDY,117.827751,1.009147,118.905548
890764,3004_000.9,Arterial,3.0,3.0,D1,Daily,5086.44,0.077937,0.026411,0.600000,...,0.566,9393.90200,Daily,Daily,0.100282,Season,AvgSuDY,942.042574,1.028793,969.166830


In [36]:
df = dfs_PeakHourForecasts[0]

df = df[(df[nameSegID]=='0085_011.3') & (df[namePeriod+'_TDM']!='Daily')]

df = df.groupby([nameSegID,nameFG]).agg({'VOL':[np.sum],'VolPercentSegDY':[np.sum]})

display(df)

,,VOL,VolPercentSegDY
,,sum,sum
SEGID,FUNCGROUP,,
0085_011.3,Arterial,556824.8,13.203661


In [37]:
dfs_PeakHourForecasts_wCapacity = []

for df_PeakHourForecasts in dfs_PeakHourForecasts:

    #calculate peak hour
    df_PeakHourForecasts_wCapacity = pd.DataFrame.merge(df_PeakHourForecasts,df_PeriodDuration,on='Period',how='left')
    df_PeakHourForecasts_wCapacity['VCPeakHourPCE']  = df_PeakHourForecasts_wCapacity['ForecastPeakHourPCE'] / df_PeakHourForecasts_wCapacity[nameCap1Hr]
    
    display(df_PeakHourForecasts_wCapacity)
    
    dfs_PeakHourForecasts_wCapacity.append(df_PeakHourForecasts_wCapacity)

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,127.50,0.063529,0.044706,0.091569,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035059,0.000000,3,0.000000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,127.20,0.063679,0.044811,0.091353,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035142,0.000000,3,0.000000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,131.10,0.176201,0.145690,0.086769,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.108085,0.000000,3,0.000000
3,0006_149.9,Arterial,1.0,2.0,D2,AM,112.40,0.201957,0.169039,0.074393,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.124911,0.000000,3,0.000000
4,0006_150.6,Arterial,1.0,2.0,D1,AM,155.40,0.160232,0.131918,0.090454,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.098005,0.000000,3,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789781,1084_001.0,Arterial,1.0,5.0,D2,Daily,365.68,0.218442,0.123824,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,13.155039,1.105601,14.544218,24,0.025427
789782,2865_001.3,Arterial,2.0,4.0,D1,Daily,1397.52,0.016400,0.009274,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.007917,0.000000,24,0.000000
789783,2865_001.3,Arterial,2.0,4.0,D2,Daily,931.68,0.016400,0.009274,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.007917,0.000000,24,0.000000
789784,3004_000.9,Arterial,2.0,4.0,D1,Daily,133.26,0.072040,0.025664,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.027240,0.000000,24,0.000000


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,126.00,0.067460,0.049206,0.085960,...,AM,Period,0.413058,Month,JanWkAM,0.00000,1.038095,0.000000,3,0.000000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,139.40,0.060976,0.044476,0.095102,...,AM,Period,0.413058,Month,JanWkAM,0.00000,1.034433,0.000000,3,0.000000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,143.50,0.179094,0.144251,0.087880,...,AM,Period,0.413058,Month,JanWkAM,0.00000,1.107944,0.000000,3,0.000000
3,0006_149.9,Arterial,1.0,2.0,D2,AM,121.50,0.204115,0.167901,0.074407,...,AM,Period,0.413058,Month,JanWkAM,0.00000,1.124774,0.000000,3,0.000000
4,0006_150.6,Arterial,1.0,2.0,D1,AM,169.60,0.163915,0.130307,0.091770,...,AM,Period,0.413058,Month,JanWkAM,0.00000,1.097936,0.000000,3,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805285,1084_001.0,Arterial,1.0,5.0,D2,Daily,353.68,0.245646,0.139222,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,15.03433,1.118740,16.819508,24,0.029405
805286,2865_001.3,Arterial,2.0,4.0,D1,Daily,1514.22,0.017197,0.009906,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.00000,1.008392,0.000000,24,0.000000
805287,2865_001.3,Arterial,2.0,4.0,D2,Daily,1009.48,0.017197,0.009906,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.00000,1.008392,0.000000,24,0.000000
805288,3004_000.9,Arterial,2.0,4.0,D1,Daily,331.44,0.098117,0.060825,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.00000,1.050036,0.000000,24,0.000000


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,142.30,0.063247,0.046381,0.091276,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035840,0.000000,3,0.00000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,145.10,0.062026,0.045486,0.093072,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035148,0.000000,3,0.00000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,144.70,0.179682,0.143055,0.082427,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.107464,0.000000,3,0.00000
3,0006_149.9,Arterial,1.0,2.0,D2,AM,146.70,0.176551,0.141786,0.083566,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.106203,0.000000,3,0.00000
4,0006_150.6,Arterial,1.0,2.0,D1,AM,169.70,0.165586,0.132587,0.086112,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.099411,0.000000,3,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832417,1084_001.0,Arterial,1.0,5.0,D2,Daily,311.84,0.273345,0.161878,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,18.792913,1.135608,21.341382,24,0.03731
832418,2865_001.3,Arterial,2.0,4.0,D1,Daily,1669.02,0.016752,0.009850,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.008276,0.000000,24,0.00000
832419,2865_001.3,Arterial,2.0,4.0,D2,Daily,1112.68,0.016752,0.009850,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.008276,0.000000,24,0.00000
832420,3004_000.9,Arterial,2.0,3.0,D1,Daily,1407.36,0.080278,0.027370,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.029741,0.000000,24,0.00000


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,163.30,0.059400,0.044091,0.096920,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.033925,0.000000,3,0.000000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,149.20,0.065013,0.048257,0.088551,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.037131,0.000000,3,0.000000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,170.60,0.181125,0.151231,0.076489,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.111841,0.000000,3,0.000000
3,0006_149.9,Arterial,1.0,2.0,D2,AM,189.60,0.161392,0.136603,0.085007,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.100580,0.000000,3,0.000000
4,0006_150.6,Arterial,1.0,2.0,D1,AM,195.30,0.168971,0.141321,0.079503,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.104455,0.000000,3,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863629,1084_001.0,Arterial,1.0,5.0,D2,Daily,357.04,0.285010,0.222832,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,26.310078,1.168418,30.741171,24,0.053743
863630,2865_001.3,Arterial,2.0,4.0,D1,Daily,1913.10,0.018065,0.011040,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.009133,0.000000,24,0.000000
863631,2865_001.3,Arterial,2.0,4.0,D2,Daily,1275.40,0.018065,0.011040,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.009133,0.000000,24,0.000000
863632,3004_000.9,Arterial,3.0,3.0,D1,Daily,2380.02,0.096655,0.027958,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.033310,0.000000,24,0.000000


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,194.50,0.053985,0.039589,0.107346,...,AM,Period,0.413058,Month,JanWkAM,71.048344,1.030591,73.221803,3,0.068753
1,0006_146.9,Arterial,1.0,2.0,D2,AM,145.90,0.071967,0.052776,0.080523,...,AM,Period,0.413058,Month,JanWkAM,53.365201,1.040781,55.541506,3,0.052152
2,0147_003.1,Arterial,1.0,3.0,D1,AM,63.20,0.231013,0.356013,0.071477,...,AM,Period,0.413058,Month,JanWkAM,27.156256,1.224209,33.244929,3,0.033820
3,0147_003.1,Arterial,1.0,3.0,D2,AM,61.30,0.213703,0.339315,0.069328,...,AM,Period,0.413058,Month,JanWkAM,26.208945,1.212398,31.775673,3,0.032325
4,0147_003.9,Arterial,1.0,3.0,D1,AM,61.20,0.230392,0.354575,0.072831,...,AM,Period,0.413058,Month,JanWkAM,34.418976,1.223366,42.107005,3,0.042835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890557,1084_001.0,Arterial,1.0,5.0,D2,Daily,259.04,0.208616,0.174336,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,30.068660,1.128891,33.944248,24,0.059343
890558,2865_001.3,Arterial,2.0,4.0,D1,Daily,2150.16,0.017134,0.011441,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,176.741626,1.009147,178.358322,24,0.227208
890559,2865_001.3,Arterial,2.0,4.0,D2,Daily,1433.44,0.017134,0.011441,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,117.827751,1.009147,118.905548,24,0.151472
890560,3004_000.9,Arterial,3.0,3.0,D1,Daily,9789.30,0.145800,0.041690,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,1208.983962,1.050005,1269.439459,24,1.419955


...

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,144.20,0.068655,0.049931,0.086358,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.038696,0.000000,3,0.000000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,159.20,0.062186,0.045226,0.095341,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035050,0.000000,3,0.000000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,436.00,0.109174,0.078211,0.140695,...,AM,Period,0.413058,Month,JanWkAM,38.523991,1.060940,40.871657,3,0.038377
3,0006_149.9,Arterial,1.0,2.0,D2,AM,166.60,0.226291,0.179472,0.053761,...,AM,Period,0.413058,Month,JanWkAM,14.841210,1.134994,16.844684,3,0.015817
4,0006_150.6,Arterial,1.0,2.0,D1,AM,466.10,0.107702,0.077666,0.139039,...,AM,Period,0.413058,Month,JanWkAM,39.471302,1.060373,41.854316,3,0.039300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804877,1084_001.0,Arterial,1.0,5.0,D2,Daily,350.36,0.285992,0.206645,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,25.652326,1.160521,29.770052,24,0.052046
804878,2865_001.3,Arterial,2.0,4.0,D1,Daily,1895.46,0.020259,0.009243,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.008673,0.000000,24,0.000000
804879,2865_001.3,Arterial,2.0,4.0,D2,Daily,1263.64,0.020259,0.009243,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.008673,0.000000,24,0.000000
804880,3004_000.9,Arterial,3.0,4.0,D1,Daily,430.08,0.078125,0.027344,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.029297,0.000000,24,0.000000


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,156.10,0.067905,0.049327,0.086959,...,AM,Period,0.413058,Month,JanWkAM,57.154446,1.038245,59.340301,3,0.055719
1,0006_146.9,Arterial,1.0,2.0,D2,AM,171.10,0.061952,0.045003,0.095315,...,AM,Period,0.413058,Month,JanWkAM,62.838314,1.034892,65.030860,3,0.061062
2,0141_000.0,Arterial,1.0,3.0,D1,AM,285.50,0.135552,0.096322,0.143223,...,AM,Period,0.413058,Month,JanWkAM,70.101033,1.075271,75.377640,3,0.078437
3,0141_000.0,Arterial,1.0,3.0,D2,AM,106.90,0.260992,0.218896,0.053627,...,AM,Period,0.413058,Month,JanWkAM,26.208945,1.161646,30.445526,3,0.031681
4,0141_000.9,Arterial,1.0,3.0,D1,AM,332.10,0.128576,0.088829,0.134036,...,AM,Period,0.413058,Month,JanWkAM,82.731850,1.070129,88.533792,3,0.091840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804877,1084_001.0,Arterial,1.0,5.0,D2,Daily,297.88,0.212837,0.181550,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,33.733278,1.133342,38.231350,24,0.066838
804878,2865_001.3,Arterial,2.0,4.0,D1,Daily,2126.34,0.019357,0.010779,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,174.881188,1.009261,176.500760,24,0.224842
804879,2865_001.3,Arterial,2.0,4.0,D2,Daily,1417.56,0.019357,0.010779,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,116.587459,1.009261,117.667173,24,0.149894
804880,3004_000.9,Arterial,3.0,4.0,D1,Daily,467.10,0.077200,0.025305,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,679.869009,1.028092,698.968220,24,0.934449


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,142.60,0.063114,0.046283,0.091457,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035764,0.000000,3,0.000000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,145.10,0.062026,0.045486,0.093061,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.035148,0.000000,3,0.000000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,145.10,0.179187,0.142660,0.082579,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.107167,0.000000,3,0.000000
3,0006_149.9,Arterial,1.0,2.0,D2,AM,146.70,0.176551,0.141786,0.083490,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.106203,0.000000,3,0.000000
4,0006_150.6,Arterial,1.0,2.0,D1,AM,169.70,0.165586,0.132587,0.086081,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.099411,0.000000,3,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834457,1084_001.0,Arterial,1.0,5.0,D2,Daily,326.16,0.261467,0.155016,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,20.108416,1.129801,22.718515,24,0.039718
834458,2865_001.3,Arterial,2.0,4.0,D1,Daily,1670.22,0.016740,0.009843,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.077518,1.008270,0.078159,24,0.000100
834459,2865_001.3,Arterial,2.0,4.0,D2,Daily,1113.48,0.016740,0.009843,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.077518,1.008270,0.078159,24,0.000100
834460,3004_000.9,Arterial,2.0,3.0,D1,Daily,1387.20,0.080796,0.026773,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.000000,1.029546,0.000000,24,0.000000


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,163.50,0.059327,0.044037,0.096987,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.033884,0.000000,3,0.000000
1,0006_146.9,Arterial,1.0,2.0,D2,AM,149.20,0.065013,0.048257,0.088504,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.037131,0.000000,3,0.000000
2,0006_149.9,Arterial,1.0,2.0,D1,AM,166.80,0.180456,0.153477,0.079015,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.112830,0.000000,3,0.000000
3,0006_149.9,Arterial,1.0,2.0,D2,AM,180.00,0.165000,0.142222,0.085268,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.104111,0.000000,3,0.000000
4,0006_150.6,Arterial,1.0,2.0,D1,AM,192.60,0.167186,0.142783,0.082297,...,AM,Period,0.413058,Month,JanWkAM,0.000000,1.104829,0.000000,3,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867607,1084_001.0,Arterial,1.0,5.0,D2,Daily,368.72,0.287047,0.225754,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,27.437652,1.170286,32.109911,24,0.056136
867608,2865_001.3,Arterial,2.0,4.0,D1,Daily,1915.38,0.018043,0.011027,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.155037,1.009122,0.156451,24,0.000199
867609,2865_001.3,Arterial,2.0,4.0,D2,Daily,1276.92,0.018043,0.011027,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,0.155037,1.009122,0.156451,24,0.000199
867610,3004_000.9,Arterial,3.0,3.0,D1,Daily,3062.70,0.099677,0.032285,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,38.766951,1.036078,40.165583,24,0.044928


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,...,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0006_146.9,Arterial,1.0,2.0,D1,AM,195.50,0.053708,0.039386,0.107809,...,AM,Period,0.413058,Month,JanWkAM,71.679885,1.030435,73.861447,3,0.069353
1,0006_146.9,Arterial,1.0,2.0,D2,AM,146.10,0.071869,0.052704,0.080567,...,AM,Period,0.413058,Month,JanWkAM,53.365201,1.040726,55.538527,3,0.052149
2,0147_003.1,Arterial,1.0,3.0,D1,AM,62.40,0.230769,0.355769,0.071691,...,AM,Period,0.413058,Month,JanWkAM,26.840486,1.224038,32.853787,3,0.033422
3,0147_003.1,Arterial,1.0,3.0,D2,AM,60.30,0.213930,0.341625,0.069278,...,AM,Period,0.413058,Month,JanWkAM,25.893174,1.213599,31.423922,3,0.031967
4,0147_003.9,Arterial,1.0,3.0,D1,AM,61.10,0.229133,0.351882,0.072929,...,AM,Period,0.413058,Month,JanWkAM,34.418976,1.221768,42.051989,3,0.042779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890761,1084_001.0,Arterial,1.0,5.0,D2,Daily,229.04,0.202934,0.162592,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,27.249723,1.121883,30.570991,24,0.053446
890762,2865_001.3,Arterial,2.0,4.0,D1,Daily,2150.16,0.017134,0.011441,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,176.741626,1.009147,178.358322,24,0.227208
890763,2865_001.3,Arterial,2.0,4.0,D2,Daily,1433.44,0.017134,0.011441,0.400000,...,Daily,Daily,0.100282,Season,AvgSuDY,117.827751,1.009147,118.905548,24,0.151472
890764,3004_000.9,Arterial,3.0,3.0,D1,Daily,5086.44,0.077937,0.026411,0.600000,...,Daily,Daily,0.100282,Season,AvgSuDY,942.042574,1.028793,969.166830,24,1.084079


In [101]:
#15-Min Analysis

dfs_Peak_VC = []

for df_PeakHourForecasts_wCapacity in dfs_PeakHourForecasts_wCapacity:

    df_Peak_VC = df_PeakHourForecasts_wCapacity
    
    #PHF conditions (from Chad's spreadsheet for Urban (0.92,0.95,0.97,0.98). HCM2000 Exhibit 9.2 says default PHF for Urban Areas is 0.92, Rural is 0.88. Rural estimated at (0.88,0.91,0.93,0.95)
    conditions = [
        (df_Peak_VC[nameAT] >= 3) & (df_Peak_VC['VCPeakHourPCE'] <= 0.85),
        (df_Peak_VC[nameAT] >= 3) & (df_Peak_VC['VCPeakHourPCE'] >  0.85) & (df_Peak_VC['VCPeakHourPCE'] <= 0.95),
        (df_Peak_VC[nameAT] >= 3) & (df_Peak_VC['VCPeakHourPCE'] >  0.95) & (df_Peak_VC['VCPeakHourPCE'] <= 1.00),
        (df_Peak_VC[nameAT] >= 3) & (df_Peak_VC['VCPeakHourPCE'] >  1.00),
        (df_Peak_VC[nameAT] <= 2) & (df_Peak_VC['VCPeakHourPCE'] <= 0.85),
        (df_Peak_VC[nameAT] <= 2) & (df_Peak_VC['VCPeakHourPCE'] >  0.85) & (df_Peak_VC['VCPeakHourPCE'] <= 0.95),
        (df_Peak_VC[nameAT] <= 2) & (df_Peak_VC['VCPeakHourPCE'] >  0.95) & (df_Peak_VC['VCPeakHourPCE'] <= 1.00),
        (df_Peak_VC[nameAT] <= 2) & (df_Peak_VC['VCPeakHourPCE'] >  1.00)
    ]
    choices = [0.92,0.95,0.97,0.98,0.88,0.91,0.93,0.95]

    df_Peak_VC['PHF'] = np.select(conditions, choices, default=0)


    #15-min volume and v/c calcs
    df_Peak_VC['ForecastPeak15MinPCE'] = df_Peak_VC['ForecastPeakHourPCE']  / df_Peak_VC['PHF']
    df_Peak_VC['VCPeak15MinPCE']  = df_Peak_VC['ForecastPeak15MinPCE'] / df_Peak_VC[nameCap1Hr]

    #effective K-factor
    #df_Peak_VC['kFactor_effective']  = df_Peak_VC['ForecastPeakHour']  / df_Peak_VC['ForecastAADT']

    #convert to int and round columns after calcs
    df_Peak_VC = df_Peak_VC.fillna(0)
    df_Peak_VC['AREATYPE'] = df_Peak_VC['AREATYPE'].astype(int)
    df_Peak_VC['FT'] = df_Peak_VC['FT'].astype(int)
    df_Peak_VC['LANES'] = df_Peak_VC['LANES'].astype(int)
    df_Peak_VC['Cap1Hr'] = df_Peak_VC['Cap1Hr'].astype(int)
    df_Peak_VC['ForecastAADT'] = df_Peak_VC['ForecastAADT'].astype(int)
    df_Peak_VC['ForecastPeakHour'] = df_Peak_VC['ForecastPeakHour'].astype(int)
    df_Peak_VC['ForecastPeakHourPCE'] = df_Peak_VC['ForecastPeakHourPCE'].astype(int)
    df_Peak_VC['ForecastPeak15MinPCE'] = df_Peak_VC['ForecastPeak15MinPCE'].astype(int)
    df_Peak_VC = df_Peak_VC.round({'SeasonFactor':2,'DOWFactor':2,'VCPeak15MinPCE':2,'PercentTrucksMD':2,'PercentTrucksHV':2,'VolPercentSegDY':2,'VolMaxHourPercent':2,'FactorTrucks':2,'VCPeak15MinPCE':2})
    
    display(df_Peak_VC)
    
    dfs_Peak_VC.append(df_Peak_VC)
    

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE
0,0006_146.9,Arterial,1,2,D1,AM,127.50,0.06,0.04,0.09,1392.4,1,1065,Arterial,StatewideArterial,2019,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.04,0,3,0.000000,0.88,0,0.00
1,0006_146.9,Arterial,1,2,D2,AM,127.20,0.06,0.04,0.09,1392.4,1,1065,Arterial,StatewideArterial,2019,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.04,0,3,0.000000,0.88,0,0.00
2,0006_149.9,Arterial,1,2,D1,AM,131.10,0.18,0.15,0.09,1510.9,1,1065,Arterial,StatewideArterial,2019,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.11,0,3,0.000000,0.88,0,0.00
3,0006_149.9,Arterial,1,2,D2,AM,112.40,0.20,0.17,0.07,1510.9,1,1065,Arterial,StatewideArterial,2019,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.12,0,3,0.000000,0.88,0,0.00
4,0006_150.6,Arterial,1,2,D1,AM,155.40,0.16,0.13,0.09,1718.0,1,1065,Arterial,StatewideArterial,2019,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.10,0,3,0.000000,0.88,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789781,1084_001.0,Arterial,1,5,D2,Daily,365.68,0.22,0.12,0.40,914.2,1,572,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S00-Avg,1.00,140.0,WD0,7-Sunday,0.94,131.18,Daily,Daily,0.10,Season,AvgSuDY,13,1.11,14,24,0.025427,0.88,16,0.03
789782,2865_001.3,Arterial,2,4,D1,Daily,1397.52,0.02,0.01,0.60,2329.2,1,785,Arterial,RuralRecLowVol,2019,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.00,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000000,0.88,0,0.00
789783,2865_001.3,Arterial,2,4,D2,Daily,931.68,0.02,0.01,0.40,2329.2,1,785,Arterial,RuralRecLowVol,2019,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.00,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000000,0.88,0,0.00
789784,3004_000.9,Arterial,2,4,D1,Daily,133.26,0.07,0.03,0.60,222.1,1,785,Arterial,RuralRecLowVol,2019,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.00,Daily,Daily,0.10,Season,AvgSuDY,0,1.03,0,24,0.000000,0.88,0,0.00


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE
0,0006_146.9,Arterial,1,2,D1,AM,126.00,0.07,0.05,0.09,1465.8,1,1065,Arterial,StatewideArterial,2024,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.04,0,3,0.000000,0.88,0,0.00
1,0006_146.9,Arterial,1,2,D2,AM,139.40,0.06,0.04,0.10,1465.8,1,1065,Arterial,StatewideArterial,2024,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.03,0,3,0.000000,0.88,0,0.00
2,0006_149.9,Arterial,1,2,D1,AM,143.50,0.18,0.14,0.09,1632.9,1,1065,Arterial,StatewideArterial,2024,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.11,0,3,0.000000,0.88,0,0.00
3,0006_149.9,Arterial,1,2,D2,AM,121.50,0.20,0.17,0.07,1632.9,1,1065,Arterial,StatewideArterial,2024,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.12,0,3,0.000000,0.88,0,0.00
4,0006_150.6,Arterial,1,2,D1,AM,169.60,0.16,0.13,0.09,1848.1,1,1065,Arterial,StatewideArterial,2024,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.10,0,3,0.000000,0.88,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805285,1084_001.0,Arterial,1,5,D2,Daily,353.68,0.25,0.14,0.40,884.2,1,572,Arterial,RuralRecLowVol,2024,400,160.0,0,1000000,SNL,S00-Avg,1.00,160.0,WD0,7-Sunday,0.94,149.92,Daily,Daily,0.10,Season,AvgSuDY,15,1.12,16,24,0.029405,0.88,19,0.03
805286,2865_001.3,Arterial,2,4,D1,Daily,1514.22,0.02,0.01,0.60,2523.7,1,785,Arterial,RuralRecLowVol,2024,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.00,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000000,0.88,0,0.00
805287,2865_001.3,Arterial,2,4,D2,Daily,1009.48,0.02,0.01,0.40,2523.7,1,785,Arterial,RuralRecLowVol,2024,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.00,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000000,0.88,0,0.00
805288,3004_000.9,Arterial,2,4,D1,Daily,331.44,0.10,0.06,0.60,552.4,1,785,Arterial,RuralRecLowVol,2024,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.00,Daily,Daily,0.10,Season,AvgSuDY,0,1.05,0,24,0.000000,0.88,0,0.00


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE
0,0006_146.9,Arterial,1,2,D1,AM,142.30,0.06,0.05,0.09,1559.0,1,1065,Arterial,StatewideArterial,2030,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.0,AM,Period,0.41,Month,JanWkAM,0,1.04,0,3,0.00000,0.88,0,0.00
1,0006_146.9,Arterial,1,2,D2,AM,145.10,0.06,0.05,0.09,1559.0,1,1065,Arterial,StatewideArterial,2030,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.0,AM,Period,0.41,Month,JanWkAM,0,1.04,0,3,0.00000,0.88,0,0.00
2,0006_149.9,Arterial,1,2,D1,AM,144.70,0.18,0.14,0.08,1755.5,1,1065,Arterial,StatewideArterial,2030,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.0,AM,Period,0.41,Month,JanWkAM,0,1.11,0,3,0.00000,0.88,0,0.00
3,0006_149.9,Arterial,1,2,D2,AM,146.70,0.18,0.14,0.08,1755.5,1,1065,Arterial,StatewideArterial,2030,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.0,AM,Period,0.41,Month,JanWkAM,0,1.11,0,3,0.00000,0.88,0,0.00
4,0006_150.6,Arterial,1,2,D1,AM,169.70,0.17,0.13,0.09,1970.7,1,1065,Arterial,StatewideArterial,2030,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.0,AM,Period,0.41,Month,JanWkAM,0,1.10,0,3,0.00000,0.88,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832417,1084_001.0,Arterial,1,5,D2,Daily,311.84,0.27,0.16,0.40,779.6,1,572,Arterial,RuralRecLowVol,2030,500,200.0,0,1000000,SNL,S00-Avg,1.00,200.0,WD0,7-Sunday,0.94,187.4,Daily,Daily,0.10,Season,AvgSuDY,18,1.14,21,24,0.03731,0.88,24,0.04
832418,2865_001.3,Arterial,2,4,D1,Daily,1669.02,0.02,0.01,0.60,2781.7,1,785,Arterial,RuralRecLowVol,2030,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.0,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.00000,0.88,0,0.00
832419,2865_001.3,Arterial,2,4,D2,Daily,1112.68,0.02,0.01,0.40,2781.7,1,785,Arterial,RuralRecLowVol,2030,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.0,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.00000,0.88,0,0.00
832420,3004_000.9,Arterial,2,3,D1,Daily,1407.36,0.08,0.03,0.60,2345.6,1,939,Arterial,RuralRecLowVol,2030,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.0,Daily,Daily,0.10,Season,AvgSuDY,0,1.03,0,24,0.00000,0.88,0,0.00


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE
0,0006_146.9,Arterial,1,2,D1,AM,163.30,0.06,0.04,0.10,1684.9,1,1065,Arterial,StatewideArterial,2040,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.03,0,3,0.000000,0.88,0,0.00
1,0006_146.9,Arterial,1,2,D2,AM,149.20,0.07,0.05,0.09,1684.9,1,1065,Arterial,StatewideArterial,2040,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.04,0,3,0.000000,0.88,0,0.00
2,0006_149.9,Arterial,1,2,D1,AM,170.60,0.18,0.15,0.08,2230.4,1,1065,Arterial,StatewideArterial,2040,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.11,0,3,0.000000,0.88,0,0.00
3,0006_149.9,Arterial,1,2,D2,AM,189.60,0.16,0.14,0.09,2230.4,1,1065,Arterial,StatewideArterial,2040,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.10,0,3,0.000000,0.88,0,0.00
4,0006_150.6,Arterial,1,2,D1,AM,195.30,0.17,0.14,0.08,2456.5,1,1065,Arterial,StatewideArterial,2040,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.10,0,3,0.000000,0.88,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863629,1084_001.0,Arterial,1,5,D2,Daily,357.04,0.29,0.22,0.40,892.6,1,572,Arterial,RuralRecLowVol,2040,700,280.0,0,1000000,SNL,S00-Avg,1.00,280.0,WD0,7-Sunday,0.94,262.36,Daily,Daily,0.10,Season,AvgSuDY,26,1.17,30,24,0.053743,0.88,34,0.06
863630,2865_001.3,Arterial,2,4,D1,Daily,1913.10,0.02,0.01,0.60,3188.5,1,785,Arterial,RuralRecLowVol,2040,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.00,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000000,0.88,0,0.00
863631,2865_001.3,Arterial,2,4,D2,Daily,1275.40,0.02,0.01,0.40,3188.5,1,785,Arterial,RuralRecLowVol,2040,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.00,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000000,0.88,0,0.00
863632,3004_000.9,Arterial,3,3,D1,Daily,2380.02,0.10,0.03,0.60,3966.7,1,894,Arterial,RuralRecLowVol,2040,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD2,7-Sunday,0.57,0.00,Daily,Daily,0.10,Season,AvgSuDY,0,1.03,0,24,0.000000,0.92,0,0.00


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE
0,0006_146.9,Arterial,1,2,D1,AM,194.50,0.05,0.04,0.11,1811.9,1,1065,Arterial,StatewideArterial,2050,2100,225.0,0,2500,SNP,M01-Jan,0.82,183.375,WD0,W1-Weekday,0.94,172.00575,AM,Period,0.41,Month,JanWkAM,71,1.03,73,3,0.068753,0.88,83,0.08
1,0006_146.9,Arterial,1,2,D2,AM,145.90,0.07,0.05,0.08,1811.9,1,1065,Arterial,StatewideArterial,2050,2100,169.0,0,2500,SNP,M01-Jan,0.82,137.735,WD0,W1-Weekday,0.94,129.19543,AM,Period,0.41,Month,JanWkAM,53,1.04,55,3,0.052152,0.88,63,0.06
2,0147_003.1,Arterial,1,3,D1,AM,63.20,0.23,0.36,0.07,884.2,1,983,Arterial,StatewideArterial,2050,1200,86.0,0,2500,SNP,M01-Jan,0.82,70.090,WD0,W1-Weekday,0.94,65.74442,AM,Period,0.41,Month,JanWkAM,27,1.22,33,3,0.033820,0.88,37,0.04
3,0147_003.1,Arterial,1,3,D2,AM,61.30,0.21,0.34,0.07,884.2,1,983,Arterial,StatewideArterial,2050,1200,83.0,0,2500,SNP,M01-Jan,0.82,67.645,WD0,W1-Weekday,0.94,63.45101,AM,Period,0.41,Month,JanWkAM,26,1.21,31,3,0.032325,0.88,36,0.04
4,0147_003.9,Arterial,1,3,D1,AM,61.20,0.23,0.35,0.07,840.3,1,983,Arterial,StatewideArterial,2050,1500,109.0,0,2500,SNP,M01-Jan,0.82,88.835,WD0,W1-Weekday,0.94,83.32723,AM,Period,0.41,Month,JanWkAM,34,1.22,42,3,0.042835,0.88,47,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890557,1084_001.0,Arterial,1,5,D2,Daily,259.04,0.21,0.17,0.40,647.6,1,572,Arterial,RuralRecLowVol,2050,800,320.0,0,1000000,SNL,S00-Avg,1.00,320.000,WD0,7-Sunday,0.94,299.84000,Daily,Daily,0.10,Season,AvgSuDY,30,1.13,33,24,0.059343,0.88,38,0.07
890558,2865_001.3,Arterial,2,4,D1,Daily,2150.16,0.02,0.01,0.60,3583.6,1,785,Arterial,RuralRecLowVol,2050,3800,2280.0,0,1000000,SNL,S00-Avg,1.00,2280.000,WD1,7-Sunday,0.77,1762.44000,Daily,Daily,0.10,Season,AvgSuDY,176,1.01,178,24,0.227208,0.88,202,0.26
890559,2865_001.3,Arterial,2,4,D2,Daily,1433.44,0.02,0.01,0.40,3583.6,1,785,Arterial,RuralRecLowVol,2050,3800,1520.0,0,1000000,SNL,S00-Avg,1.00,1520.000,WD1,7-Sunday,0.77,1174.96000,Daily,Daily,0.10,Season,AvgSuDY,117,1.01,118,24,0.151472,0.88,135,0.17
890560,3004_000.9,Arterial,3,3,D1,Daily,9789.30,0.15,0.04,0.60,16315.5,1,894,Arterial,RuralRecLowVol,2050,35500,21300.0,0,1000000,SNL,S00-Avg,1.00,21300.000,WD2,7-Sunday,0.57,12055.80000,Daily,Daily,0.10,Season,AvgSuDY,1208,1.05,1269,24,1.419955,0.98,1295,1.45


...

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE
0,0006_146.9,Arterial,1,2,D1,AM,144.20,0.07,0.05,0.09,1669.8,1,1065,Arterial,StatewideArterial,2040,0,0.0,0,2500,SNP,M01-Jan,0.82,0.000,WD0,W1-Weekday,0.94,0.00000,AM,Period,0.41,Month,JanWkAM,0,1.04,0,3,0.000000,0.88,0,0.00
1,0006_146.9,Arterial,1,2,D2,AM,159.20,0.06,0.05,0.10,1669.8,1,1065,Arterial,StatewideArterial,2040,0,0.0,0,2500,SNP,M01-Jan,0.82,0.000,WD0,W1-Weekday,0.94,0.00000,AM,Period,0.41,Month,JanWkAM,0,1.04,0,3,0.000000,0.88,0,0.00
2,0006_149.9,Arterial,1,2,D1,AM,436.00,0.11,0.08,0.14,3098.9,1,1065,Arterial,StatewideArterial,2040,868,122.0,0,2500,SNP,M01-Jan,0.82,99.430,WD0,W1-Weekday,0.94,93.26534,AM,Period,0.41,Month,JanWkAM,38,1.06,40,3,0.038377,0.88,46,0.04
3,0006_149.9,Arterial,1,2,D2,AM,166.60,0.23,0.18,0.05,3098.9,1,1065,Arterial,StatewideArterial,2040,868,47.0,0,2500,SNP,M01-Jan,0.82,38.305,WD0,W1-Weekday,0.94,35.93009,AM,Period,0.41,Month,JanWkAM,14,1.13,16,3,0.015817,0.88,19,0.02
4,0006_150.6,Arterial,1,2,D1,AM,466.10,0.11,0.08,0.14,3352.3,1,1065,Arterial,StatewideArterial,2040,895,125.0,0,2500,SNP,M01-Jan,0.82,101.875,WD0,W1-Weekday,0.94,95.55875,AM,Period,0.41,Month,JanWkAM,39,1.06,41,3,0.039300,0.88,47,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804877,1084_001.0,Arterial,1,5,D2,Daily,350.36,0.29,0.21,0.40,875.9,1,572,Arterial,RuralRecLowVol,2040,683,273.0,0,1000000,SNL,S00-Avg,1.00,273.000,WD0,7-Sunday,0.94,255.80100,Daily,Daily,0.10,Season,AvgSuDY,25,1.16,29,24,0.052046,0.88,33,0.06
804878,2865_001.3,Arterial,2,4,D1,Daily,1895.46,0.02,0.01,0.60,3159.1,1,785,Arterial,RuralRecLowVol,2040,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.000,WD1,7-Sunday,0.77,0.00000,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000000,0.88,0,0.00
804879,2865_001.3,Arterial,2,4,D2,Daily,1263.64,0.02,0.01,0.40,3159.1,1,785,Arterial,RuralRecLowVol,2040,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.000,WD1,7-Sunday,0.77,0.00000,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000000,0.88,0,0.00
804880,3004_000.9,Arterial,3,4,D1,Daily,430.08,0.08,0.03,0.60,716.8,1,748,Arterial,RuralRecLowVol,2040,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.000,WD2,7-Sunday,0.57,0.00000,Daily,Daily,0.10,Season,AvgSuDY,0,1.03,0,24,0.000000,0.92,0,0.00


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE
0,0006_146.9,Arterial,1,2,D1,AM,156.10,0.07,0.05,0.09,1795.1,1,1065,Arterial,StatewideArterial,2050,2083,181.0,0,2500,SNP,M01-Jan,0.82,147.515,WD0,W1-Weekday,0.94,138.36907,AM,Period,0.41,Month,JanWkAM,57,1.04,59,3,0.055719,0.88,67,0.06
1,0006_146.9,Arterial,1,2,D2,AM,171.10,0.06,0.05,0.10,1795.1,1,1065,Arterial,StatewideArterial,2050,2083,199.0,0,2500,SNP,M01-Jan,0.82,162.185,WD0,W1-Weekday,0.94,152.12953,AM,Period,0.41,Month,JanWkAM,62,1.03,65,3,0.061062,0.88,73,0.07
2,0141_000.0,Arterial,1,3,D1,AM,285.50,0.14,0.10,0.14,1993.4,1,961,Arterial,StatewideArterial,2050,1551,222.0,0,2500,SNP,M01-Jan,0.82,180.930,WD0,W1-Weekday,0.94,169.71234,AM,Period,0.41,Month,JanWkAM,70,1.08,75,3,0.078437,0.88,85,0.09
3,0141_000.0,Arterial,1,3,D2,AM,106.90,0.26,0.22,0.05,1993.4,1,961,Arterial,StatewideArterial,2050,1551,83.0,0,2500,SNP,M01-Jan,0.82,67.645,WD0,W1-Weekday,0.94,63.45101,AM,Period,0.41,Month,JanWkAM,26,1.16,30,3,0.031681,0.88,34,0.04
4,0141_000.9,Arterial,1,3,D1,AM,332.10,0.13,0.09,0.13,2477.7,1,964,Arterial,StatewideArterial,2050,1957,262.0,0,2500,SNP,M01-Jan,0.82,213.530,WD0,W1-Weekday,0.94,200.29114,AM,Period,0.41,Month,JanWkAM,82,1.07,88,3,0.091840,0.88,100,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804877,1084_001.0,Arterial,1,5,D2,Daily,297.88,0.21,0.18,0.40,744.7,1,572,Arterial,RuralRecLowVol,2050,897,359.0,0,1000000,SNL,S00-Avg,1.00,359.000,WD0,7-Sunday,0.94,336.38300,Daily,Daily,0.10,Season,AvgSuDY,33,1.13,38,24,0.066838,0.88,43,0.08
804878,2865_001.3,Arterial,2,4,D1,Daily,2126.34,0.02,0.01,0.60,3543.9,1,785,Arterial,RuralRecLowVol,2050,3760,2256.0,0,1000000,SNL,S00-Avg,1.00,2256.000,WD1,7-Sunday,0.77,1743.88800,Daily,Daily,0.10,Season,AvgSuDY,174,1.01,176,24,0.224842,0.88,200,0.26
804879,2865_001.3,Arterial,2,4,D2,Daily,1417.56,0.02,0.01,0.40,3543.9,1,785,Arterial,RuralRecLowVol,2050,3760,1504.0,0,1000000,SNL,S00-Avg,1.00,1504.000,WD1,7-Sunday,0.77,1162.59200,Daily,Daily,0.10,Season,AvgSuDY,116,1.01,117,24,0.149894,0.88,133,0.17
804880,3004_000.9,Arterial,3,4,D1,Daily,467.10,0.08,0.03,0.60,778.5,1,748,Arterial,RuralRecLowVol,2050,19963,11978.0,0,1000000,SNL,S00-Avg,1.00,11978.000,WD2,7-Sunday,0.57,6779.54800,Daily,Daily,0.10,Season,AvgSuDY,679,1.03,698,24,0.934449,0.95,735,0.98


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE
0,0006_146.9,Arterial,1,2,D1,AM,142.60,0.06,0.05,0.09,1559.2,1,1065,Arterial,StatewideArterial,2030,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.000,AM,Period,0.41,Month,JanWkAM,0,1.04,0,3,0.000000,0.88,0,0.00
1,0006_146.9,Arterial,1,2,D2,AM,145.10,0.06,0.05,0.09,1559.2,1,1065,Arterial,StatewideArterial,2030,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.000,AM,Period,0.41,Month,JanWkAM,0,1.04,0,3,0.000000,0.88,0,0.00
2,0006_149.9,Arterial,1,2,D1,AM,145.10,0.18,0.14,0.08,1757.1,1,1065,Arterial,StatewideArterial,2030,1,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.000,AM,Period,0.41,Month,JanWkAM,0,1.11,0,3,0.000000,0.88,0,0.00
3,0006_149.9,Arterial,1,2,D2,AM,146.70,0.18,0.14,0.08,1757.1,1,1065,Arterial,StatewideArterial,2030,1,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.000,AM,Period,0.41,Month,JanWkAM,0,1.11,0,3,0.000000,0.88,0,0.00
4,0006_150.6,Arterial,1,2,D1,AM,169.70,0.17,0.13,0.09,1971.4,1,1065,Arterial,StatewideArterial,2030,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.000,AM,Period,0.41,Month,JanWkAM,0,1.10,0,3,0.000000,0.88,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834457,1084_001.0,Arterial,1,5,D2,Daily,326.16,0.26,0.16,0.40,815.4,1,572,Arterial,RuralRecLowVol,2030,535,214.0,0,1000000,SNL,S00-Avg,1.00,214.0,WD0,7-Sunday,0.94,200.518,Daily,Daily,0.10,Season,AvgSuDY,20,1.13,22,24,0.039718,0.88,25,0.05
834458,2865_001.3,Arterial,2,4,D1,Daily,1670.22,0.02,0.01,0.60,2783.7,1,785,Arterial,RuralRecLowVol,2030,2,1.0,0,1000000,SNL,S00-Avg,1.00,1.0,WD1,7-Sunday,0.77,0.773,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000100,0.88,0,0.00
834459,2865_001.3,Arterial,2,4,D2,Daily,1113.48,0.02,0.01,0.40,2783.7,1,785,Arterial,RuralRecLowVol,2030,2,1.0,0,1000000,SNL,S00-Avg,1.00,1.0,WD1,7-Sunday,0.77,0.773,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000100,0.88,0,0.00
834460,3004_000.9,Arterial,2,3,D1,Daily,1387.20,0.08,0.03,0.60,2312.0,1,939,Arterial,RuralRecLowVol,2030,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.000,Daily,Daily,0.10,Season,AvgSuDY,0,1.03,0,24,0.000000,0.88,0,0.00


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE
0,0006_146.9,Arterial,1,2,D1,AM,163.50,0.06,0.04,0.10,1685.8,1,1065,Arterial,StatewideArterial,2040,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.000,AM,Period,0.41,Month,JanWkAM,0,1.03,0,3,0.000000,0.88,0,0.00
1,0006_146.9,Arterial,1,2,D2,AM,149.20,0.07,0.05,0.09,1685.8,1,1065,Arterial,StatewideArterial,2040,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.000,AM,Period,0.41,Month,JanWkAM,0,1.04,0,3,0.000000,0.88,0,0.00
2,0006_149.9,Arterial,1,2,D1,AM,166.80,0.18,0.15,0.08,2111.0,1,1065,Arterial,StatewideArterial,2040,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.000,AM,Period,0.41,Month,JanWkAM,0,1.11,0,3,0.000000,0.88,0,0.00
3,0006_149.9,Arterial,1,2,D2,AM,180.00,0.16,0.14,0.09,2111.0,1,1065,Arterial,StatewideArterial,2040,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.000,AM,Period,0.41,Month,JanWkAM,0,1.10,0,3,0.000000,0.88,0,0.00
4,0006_150.6,Arterial,1,2,D1,AM,192.60,0.17,0.14,0.08,2340.3,1,1065,Arterial,StatewideArterial,2040,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.000,AM,Period,0.41,Month,JanWkAM,0,1.10,0,3,0.000000,0.88,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867607,1084_001.0,Arterial,1,5,D2,Daily,368.72,0.29,0.23,0.40,921.8,1,572,Arterial,RuralRecLowVol,2040,729,292.0,0,1000000,SNL,S00-Avg,1.00,292.0,WD0,7-Sunday,0.94,273.604,Daily,Daily,0.10,Season,AvgSuDY,27,1.17,32,24,0.056136,0.88,36,0.06
867608,2865_001.3,Arterial,2,4,D1,Daily,1915.38,0.02,0.01,0.60,3192.3,1,785,Arterial,RuralRecLowVol,2040,3,2.0,0,1000000,SNL,S00-Avg,1.00,2.0,WD1,7-Sunday,0.77,1.546,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000199,0.88,0,0.00
867609,2865_001.3,Arterial,2,4,D2,Daily,1276.92,0.02,0.01,0.40,3192.3,1,785,Arterial,RuralRecLowVol,2040,3,2.0,0,1000000,SNL,S00-Avg,1.00,2.0,WD1,7-Sunday,0.77,1.546,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000199,0.88,0,0.00
867610,3004_000.9,Arterial,3,3,D1,Daily,3062.70,0.10,0.03,0.60,5104.5,1,894,Arterial,RuralRecLowVol,2040,1137,683.0,0,1000000,SNL,S00-Avg,1.00,683.0,WD2,7-Sunday,0.57,386.578,Daily,Daily,0.10,Season,AvgSuDY,38,1.04,40,24,0.044928,0.92,43,0.05


,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE
0,0006_146.9,Arterial,1,2,D1,AM,195.50,0.05,0.04,0.11,1813.4,1,1065,Arterial,StatewideArterial,2050,2101,227.0,0,2500,SNP,M01-Jan,0.82,185.005,WD0,W1-Weekday,0.94,173.53469,AM,Period,0.41,Month,JanWkAM,71,1.03,73,3,0.069353,0.88,83,0.08
1,0006_146.9,Arterial,1,2,D2,AM,146.10,0.07,0.05,0.08,1813.4,1,1065,Arterial,StatewideArterial,2050,2101,169.0,0,2500,SNP,M01-Jan,0.82,137.735,WD0,W1-Weekday,0.94,129.19543,AM,Period,0.41,Month,JanWkAM,53,1.04,55,3,0.052149,0.88,63,0.06
2,0147_003.1,Arterial,1,3,D1,AM,62.40,0.23,0.36,0.07,870.4,1,983,Arterial,StatewideArterial,2050,1186,85.0,0,2500,SNP,M01-Jan,0.82,69.275,WD0,W1-Weekday,0.94,64.97995,AM,Period,0.41,Month,JanWkAM,26,1.22,32,3,0.033422,0.88,37,0.04
3,0147_003.1,Arterial,1,3,D2,AM,60.30,0.21,0.34,0.07,870.4,1,983,Arterial,StatewideArterial,2050,1186,82.0,0,2500,SNP,M01-Jan,0.82,66.830,WD0,W1-Weekday,0.94,62.68654,AM,Period,0.41,Month,JanWkAM,25,1.21,31,3,0.031967,0.88,35,0.04
4,0147_003.9,Arterial,1,3,D1,AM,61.10,0.23,0.35,0.07,837.8,1,983,Arterial,StatewideArterial,2050,1497,109.0,0,2500,SNP,M01-Jan,0.82,88.835,WD0,W1-Weekday,0.94,83.32723,AM,Period,0.41,Month,JanWkAM,34,1.22,42,3,0.042779,0.88,47,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890761,1084_001.0,Arterial,1,5,D2,Daily,229.04,0.20,0.16,0.40,572.6,1,572,Arterial,RuralRecLowVol,2050,725,290.0,0,1000000,SNL,S00-Avg,1.00,290.000,WD0,7-Sunday,0.94,271.73000,Daily,Daily,0.10,Season,AvgSuDY,27,1.12,30,24,0.053446,0.88,34,0.06
890762,2865_001.3,Arterial,2,4,D1,Daily,2150.16,0.02,0.01,0.60,3583.6,1,785,Arterial,RuralRecLowVol,2050,3800,2280.0,0,1000000,SNL,S00-Avg,1.00,2280.000,WD1,7-Sunday,0.77,1762.44000,Daily,Daily,0.10,Season,AvgSuDY,176,1.01,178,24,0.227208,0.88,202,0.26
890763,2865_001.3,Arterial,2,4,D2,Daily,1433.44,0.02,0.01,0.40,3583.6,1,785,Arterial,RuralRecLowVol,2050,3800,1520.0,0,1000000,SNL,S00-Avg,1.00,1520.000,WD1,7-Sunday,0.77,1174.96000,Daily,Daily,0.10,Season,AvgSuDY,117,1.01,118,24,0.151472,0.88,135,0.17
890764,3004_000.9,Arterial,3,3,D1,Daily,5086.44,0.08,0.03,0.60,8477.4,1,894,Arterial,RuralRecLowVol,2050,27661,16597.0,0,1000000,SNL,S00-Avg,1.00,16597.000,WD2,7-Sunday,0.57,9393.90200,Daily,Daily,0.10,Season,AvgSuDY,942,1.03,969,24,1.084079,0.98,988,1.11


In [102]:
dfs_Peak_VC[0]

,SEGID,FUNCGROUP,AREATYPE,FT,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,SegTotalDYVol,LANES,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,SeasonType,VCGroupCode,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE
0,0006_146.9,Arterial,1,2,D1,AM,127.50,0.06,0.04,0.09,1392.4,1,1065,Arterial,StatewideArterial,2019,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.04,0,3,0.000000,0.88,0,0.00
1,0006_146.9,Arterial,1,2,D2,AM,127.20,0.06,0.04,0.09,1392.4,1,1065,Arterial,StatewideArterial,2019,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.04,0,3,0.000000,0.88,0,0.00
2,0006_149.9,Arterial,1,2,D1,AM,131.10,0.18,0.15,0.09,1510.9,1,1065,Arterial,StatewideArterial,2019,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.11,0,3,0.000000,0.88,0,0.00
3,0006_149.9,Arterial,1,2,D2,AM,112.40,0.20,0.17,0.07,1510.9,1,1065,Arterial,StatewideArterial,2019,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.12,0,3,0.000000,0.88,0,0.00
4,0006_150.6,Arterial,1,2,D1,AM,155.40,0.16,0.13,0.09,1718.0,1,1065,Arterial,StatewideArterial,2019,0,0.0,0,2500,SNP,M01-Jan,0.82,0.0,WD0,W1-Weekday,0.94,0.00,AM,Period,0.41,Month,JanWkAM,0,1.10,0,3,0.000000,0.88,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789781,1084_001.0,Arterial,1,5,D2,Daily,365.68,0.22,0.12,0.40,914.2,1,572,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S00-Avg,1.00,140.0,WD0,7-Sunday,0.94,131.18,Daily,Daily,0.10,Season,AvgSuDY,13,1.11,14,24,0.025427,0.88,16,0.03
789782,2865_001.3,Arterial,2,4,D1,Daily,1397.52,0.02,0.01,0.60,2329.2,1,785,Arterial,RuralRecLowVol,2019,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.00,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000000,0.88,0,0.00
789783,2865_001.3,Arterial,2,4,D2,Daily,931.68,0.02,0.01,0.40,2329.2,1,785,Arterial,RuralRecLowVol,2019,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.00,Daily,Daily,0.10,Season,AvgSuDY,0,1.01,0,24,0.000000,0.88,0,0.00
789784,3004_000.9,Arterial,2,4,D1,Daily,133.26,0.07,0.03,0.60,222.1,1,785,Arterial,RuralRecLowVol,2019,0,0.0,0,1000000,SNL,S00-Avg,1.00,0.0,WD1,7-Sunday,0.77,0.00,Daily,Daily,0.10,Season,AvgSuDY,0,1.03,0,24,0.000000,0.88,0,0.00


In [103]:
df_Peak_VC.columns

Index(['SEGID', 'FUNCGROUP', 'AREATYPE', 'FT', 'Dir', 'Period_TDM', 'VOL',
       'PercentTrucksMD', 'PercentTrucksHV', 'VolPercentSegDY',
       'SegTotalDYVol', 'LANES', 'Cap1Hr', 'DOWFACFC', 'SsnGrp', 'FYEAR',
       'ForecastAADT', 'ForecastAAPeriodT', 'Vol_From', 'Vol_To',
       'SiteGroupSeason', 'SeasonGroup', 'SeasonFactor',
       'ForecastASeasonPeriodT', 'SiteGroupDOW', 'DOWGroup', 'DOWFactor',
       'ForecastASeasonDOWPeriodT', 'Period', 'HourPercentOf',
       'VolMaxHourPercent', 'SeasonType', 'VCGroupCode', 'ForecastPeakHour',
       'FactorTrucks', 'ForecastPeakHourPCE', 'PeriodHours', 'VCPeakHourPCE',
       'PHF', 'ForecastPeak15MinPCE', 'VCPeak15MinPCE'],
      dtype='object')

In [104]:
df = dfs_Peak_VC[0]

df = df[(df[nameSegID]=='0015_295.6') & (df['SeasonGroup']=='S00-Avg') & (df['DOWGroup']=='W1-Weekday') & (df['HourPercentOf']=='Period')]

pd.set_option('display.max_columns', None)

#display(df)

df2 = df.groupby(['SEGID','VCGroupCode']).agg({nameVol:[np.sum],namePercentSegTotal:[np.sum],'ForecastAAPeriodT':[np.sum]})

display(df2)

#display(df2[df2['VCGroupCode']=='AvgWkPM'])

VOL VolPercentSegDY ForecastAAPeriodT
                             sum             sum               sum
SEGID      VCGroupCode                                            
0015_295.6 AvgWkAM       61606.7            0.18           50565.0
           AvgWkMD      113523.5            0.33           93178.0
           AvgWkPM       72254.6            0.21           59305.0

In [105]:
dfs_Peak_VC_MaxDirFG = []

for df_Peak_VC in dfs_Peak_VC:

    df_Peak_VC_MaxDirFG = df_Peak_VC.groupby([nameSegID,namePeriod,'SeasonGroup','DOWGroup','HourPercentOf','VCGroupCode'], as_index=False).agg({nameDir:[np.size],'ForecastPeakHourPCE':[np.sum],'ForecastPeak15MinPCE':[np.sum],'VCPeakHourPCE':[np.max],'VCPeak15MinPCE':[np.max]})

    df_Peak_VC_MaxDirFG.columns = ([nameSegID,namePeriod,'SeasonGroup','DOWGroup','HourPercentOf','VCGroupCode','numRecords','ForecastPeakHourPCE','ForecastPeak15MinPCE','VCPeakHourPCE','VCPeak15MinPCE'])
    
    dfs_Peak_VC_MaxDirFG.append(df_Peak_VC_MaxDirFG)
    
    display(df_Peak_VC_MaxDirFG)
                                            
    #df_Peak_VC_wGroups = pd.DataFrame.merge(df_VCGroups,df_Peak_VC,on=('SeasonGroup','DOWGroup','Period','HourPercentOf'))

,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,VCGroupCode,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0006_146.9,AM,M01-Jan,W1-Weekday,Period,JanWkAM,2,0,0,0.000000,0.00
1,0006_146.9,AM,M02-Feb,W1-Weekday,Period,FebWkAM,2,0,0,0.000000,0.00
2,0006_146.9,AM,M03-Mar,W1-Weekday,Period,MarWkAM,2,0,0,0.000000,0.00
3,0006_146.9,AM,M04-Apr,W1-Weekday,Period,AprWkAM,2,0,0,0.000000,0.00
4,0006_146.9,AM,M05-May,W1-Weekday,Period,MayWkAM,2,0,0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...
369031,WFRC_8218,PM,S02-Spr,W1-Weekday,Period,SprWkPM,2,2922,2997,1.026435,1.05
369032,WFRC_8218,PM,S03-Sum,5-Friday,Daily,SumFrPM,2,2666,2791,1.086392,1.11
369033,WFRC_8218,PM,S03-Sum,W1-Weekday,Period,SumWkPM,2,2924,2998,1.026838,1.05
369034,WFRC_8218,PM,S04-Fal,5-Friday,Daily,FalFrPM,2,2698,2825,1.099328,1.12


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,VCGroupCode,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0006_146.9,AM,M01-Jan,W1-Weekday,Period,JanWkAM,2,0,0,0.000000,0.00
1,0006_146.9,AM,M02-Feb,W1-Weekday,Period,FebWkAM,2,0,0,0.000000,0.00
2,0006_146.9,AM,M03-Mar,W1-Weekday,Period,MarWkAM,2,0,0,0.000000,0.00
3,0006_146.9,AM,M04-Apr,W1-Weekday,Period,AprWkAM,2,0,0,0.000000,0.00
4,0006_146.9,AM,M05-May,W1-Weekday,Period,MayWkAM,2,0,0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...
373723,WFRC_8218,PM,S02-Spr,W1-Weekday,Period,SprWkPM,2,3141,3205,1.085687,1.11
373724,WFRC_8218,PM,S03-Sum,5-Friday,Daily,SumFrPM,2,2880,3015,1.173436,1.20
373725,WFRC_8218,PM,S03-Sum,W1-Weekday,Period,SumWkPM,2,3142,3206,1.086113,1.11
373726,WFRC_8218,PM,S04-Fal,5-Friday,Daily,FalFrPM,2,2915,3051,1.187409,1.21


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,VCGroupCode,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0006_146.9,AM,M01-Jan,W1-Weekday,Period,JanWkAM,2,0,0,0.000000,0.00
1,0006_146.9,AM,M02-Feb,W1-Weekday,Period,FebWkAM,2,0,0,0.000000,0.00
2,0006_146.9,AM,M03-Mar,W1-Weekday,Period,MarWkAM,2,0,0,0.000000,0.00
3,0006_146.9,AM,M04-Apr,W1-Weekday,Period,AprWkAM,2,0,0,0.000000,0.00
4,0006_146.9,AM,M05-May,W1-Weekday,Period,MayWkAM,2,0,0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...
382393,WFRC_8219,PM,S02-Spr,W1-Weekday,Period,SprWkPM,2,721,819,0.389236,0.44
382394,WFRC_8219,PM,S03-Sum,5-Friday,Daily,SumFrPM,2,693,788,0.443288,0.50
382395,WFRC_8219,PM,S03-Sum,W1-Weekday,Period,SumWkPM,2,762,866,0.411566,0.47
382396,WFRC_8219,PM,S04-Fal,5-Friday,Daily,FalFrPM,2,695,790,0.444345,0.50


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,VCGroupCode,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0006_146.9,AM,M01-Jan,W1-Weekday,Period,JanWkAM,2,0,0,0.000000,0.00
1,0006_146.9,AM,M02-Feb,W1-Weekday,Period,FebWkAM,2,0,0,0.000000,0.00
2,0006_146.9,AM,M03-Mar,W1-Weekday,Period,MarWkAM,2,0,0,0.000000,0.00
3,0006_146.9,AM,M04-Apr,W1-Weekday,Period,AprWkAM,2,0,0,0.000000,0.00
4,0006_146.9,AM,M05-May,W1-Weekday,Period,MayWkAM,2,0,0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...
387595,WFRC_8219,PM,S02-Spr,W1-Weekday,Period,SprWkPM,2,708,805,0.388336,0.44
387596,WFRC_8219,PM,S03-Sum,5-Friday,Daily,SumFrPM,2,716,813,0.458073,0.52
387597,WFRC_8219,PM,S03-Sum,W1-Weekday,Period,SumWkPM,2,749,851,0.410614,0.47
387598,WFRC_8219,PM,S04-Fal,5-Friday,Daily,FalFrPM,2,718,815,0.459165,0.52


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,VCGroupCode,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0006_146.9,AM,M01-Jan,W1-Weekday,Period,JanWkAM,2,128,146,0.068753,0.08
1,0006_146.9,AM,M02-Feb,W1-Weekday,Period,FebWkAM,2,136,155,0.073081,0.08
2,0006_146.9,AM,M03-Mar,W1-Weekday,Period,MarWkAM,2,148,169,0.079756,0.09
3,0006_146.9,AM,M04-Apr,W1-Weekday,Period,AprWkAM,2,161,183,0.086563,0.10
4,0006_146.9,AM,M05-May,W1-Weekday,Period,MayWkAM,2,168,191,0.090172,0.10
...,...,...,...,...,...,...,...,...,...,...,...
396571,WFRC_8219,PM,S02-Spr,W1-Weekday,Period,SprWkPM,2,1857,1995,1.035207,1.09
396572,WFRC_8219,PM,S03-Sum,5-Friday,Daily,SumFrPM,2,1543,1696,0.986256,1.06
396573,WFRC_8219,PM,S03-Sum,W1-Weekday,Period,SumWkPM,2,1963,2087,1.094596,1.15
396574,WFRC_8219,PM,S04-Fal,5-Friday,Daily,FalFrPM,2,1546,1701,0.988608,1.06


...

,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,VCGroupCode,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0006_146.9,AM,M01-Jan,W1-Weekday,Period,JanWkAM,2,0,0,0.000000,0.00
1,0006_146.9,AM,M02-Feb,W1-Weekday,Period,FebWkAM,2,0,0,0.000000,0.00
2,0006_146.9,AM,M03-Mar,W1-Weekday,Period,MarWkAM,2,0,0,0.000000,0.00
3,0006_146.9,AM,M04-Apr,W1-Weekday,Period,AprWkAM,2,0,0,0.000000,0.00
4,0006_146.9,AM,M05-May,W1-Weekday,Period,MayWkAM,2,0,0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...
373417,WFRC_8218,PM,S02-Spr,W1-Weekday,Period,SprWkPM,2,3791,3868,1.310556,1.34
373418,WFRC_8218,PM,S03-Sum,5-Friday,Daily,SumFrPM,2,3610,3699,1.470910,1.50
373419,WFRC_8218,PM,S03-Sum,W1-Weekday,Period,SumWkPM,2,3793,3870,1.311070,1.34
373420,WFRC_8218,PM,S04-Fal,5-Friday,Daily,FalFrPM,2,3653,3743,1.488424,1.52


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,VCGroupCode,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0006_146.9,AM,M01-Jan,W1-Weekday,Period,JanWkAM,2,124,140,0.061062,0.07
1,0006_146.9,AM,M02-Feb,W1-Weekday,Period,FebWkAM,2,132,149,0.064906,0.07
2,0006_146.9,AM,M03-Mar,W1-Weekday,Period,MarWkAM,2,143,163,0.070835,0.08
3,0006_146.9,AM,M04-Apr,W1-Weekday,Period,AprWkAM,2,155,177,0.076880,0.09
4,0006_146.9,AM,M05-May,W1-Weekday,Period,MayWkAM,2,162,184,0.080085,0.09
...,...,...,...,...,...,...,...,...,...,...,...
373417,WFRC_8218,PM,S02-Spr,W1-Weekday,Period,SprWkPM,2,3859,3938,1.325798,1.35
373418,WFRC_8218,PM,S03-Sum,5-Friday,Daily,SumFrPM,2,3936,4016,1.603670,1.64
373419,WFRC_8218,PM,S03-Sum,W1-Weekday,Period,SumWkPM,2,3861,3939,1.326318,1.35
373420,WFRC_8218,PM,S04-Fal,5-Friday,Daily,FalFrPM,2,3983,4065,1.622765,1.66


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,VCGroupCode,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0006_146.9,AM,M01-Jan,W1-Weekday,Period,JanWkAM,2,0,0,0.000000,0.00
1,0006_146.9,AM,M02-Feb,W1-Weekday,Period,FebWkAM,2,0,0,0.000000,0.00
2,0006_146.9,AM,M03-Mar,W1-Weekday,Period,MarWkAM,2,0,0,0.000000,0.00
3,0006_146.9,AM,M04-Apr,W1-Weekday,Period,AprWkAM,2,0,0,0.000000,0.00
4,0006_146.9,AM,M05-May,W1-Weekday,Period,MayWkAM,2,0,0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...
382393,WFRC_8219,PM,S02-Spr,W1-Weekday,Period,SprWkPM,2,664,755,0.369066,0.42
382394,WFRC_8219,PM,S03-Sum,5-Friday,Daily,SumFrPM,2,670,762,0.428945,0.49
382395,WFRC_8219,PM,S03-Sum,W1-Weekday,Period,SumWkPM,2,703,798,0.390240,0.44
382396,WFRC_8219,PM,S04-Fal,5-Friday,Daily,FalFrPM,2,672,763,0.429967,0.49


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,VCGroupCode,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0006_146.9,AM,M01-Jan,W1-Weekday,Period,JanWkAM,2,0,0,0.000000,0.00
1,0006_146.9,AM,M02-Feb,W1-Weekday,Period,FebWkAM,2,0,0,0.000000,0.00
2,0006_146.9,AM,M03-Mar,W1-Weekday,Period,MarWkAM,2,0,0,0.000000,0.00
3,0006_146.9,AM,M04-Apr,W1-Weekday,Period,AprWkAM,2,0,0,0.000000,0.00
4,0006_146.9,AM,M05-May,W1-Weekday,Period,MayWkAM,2,0,0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...
387595,WFRC_8219,PM,S02-Spr,W1-Weekday,Period,SprWkPM,2,719,818,0.391840,0.45
387596,WFRC_8219,PM,S03-Sum,5-Friday,Daily,SumFrPM,2,717,815,0.458694,0.52
387597,WFRC_8219,PM,S03-Sum,W1-Weekday,Period,SumWkPM,2,762,865,0.414320,0.47
387598,WFRC_8219,PM,S04-Fal,5-Friday,Daily,FalFrPM,2,718,817,0.459788,0.52


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,VCGroupCode,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0006_146.9,AM,M01-Jan,W1-Weekday,Period,JanWkAM,2,128,146,0.069353,0.08
1,0006_146.9,AM,M02-Feb,W1-Weekday,Period,FebWkAM,2,137,156,0.073720,0.08
2,0006_146.9,AM,M03-Mar,W1-Weekday,Period,MarWkAM,2,149,170,0.080453,0.09
3,0006_146.9,AM,M04-Apr,W1-Weekday,Period,AprWkAM,2,161,184,0.087319,0.10
4,0006_146.9,AM,M05-May,W1-Weekday,Period,MayWkAM,2,168,192,0.090960,0.10
...,...,...,...,...,...,...,...,...,...,...,...
396571,WFRC_8219,PM,S02-Spr,W1-Weekday,Period,SprWkPM,2,1719,1868,0.953657,1.03
396572,WFRC_8219,PM,S03-Sum,5-Friday,Daily,SumFrPM,2,1563,1720,0.999920,1.08
396573,WFRC_8219,PM,S03-Sum,W1-Weekday,Period,SumWkPM,2,1817,1954,1.008368,1.06
396574,WFRC_8219,PM,S04-Fal,5-Friday,Daily,FalFrPM,2,1568,1703,1.002304,1.06


In [106]:
df = dfs_Peak_VC_MaxDirFG[0]

df = df[(df[nameSegID]=='0015_295.6')]

pd.set_option('display.max_columns', None)

display(df)

,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,VCGroupCode,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
7650,0015_295.6,AM,M01-Jan,W1-Weekday,Period,JanWkAM,6,18200,19461,0.904877,0.95
7651,0015_295.6,AM,M02-Feb,W1-Weekday,Period,FebWkAM,6,19400,20528,0.964587,0.99
7652,0015_295.6,AM,M03-Mar,W1-Weekday,Period,MarWkAM,6,20386,21252,1.013520,1.03
7653,0015_295.6,AM,M04-Apr,W1-Weekday,Period,AprWkAM,6,21257,22163,1.056874,1.08
7654,0015_295.6,AM,M05-May,W1-Weekday,Period,MayWkAM,6,21017,21912,1.044964,1.07
7655,0015_295.6,AM,M06-Jun,W1-Weekday,Period,JunWkAM,6,21000,21893,1.044055,1.07
7656,0015_295.6,AM,M07-Jul,W1-Weekday,Period,JulWkAM,6,20757,21641,1.031972,1.05
7657,0015_295.6,AM,M08-Aug,W1-Weekday,Period,AugWkAM,6,21219,22122,1.054936,1.08
7658,0015_295.6,AM,M09-Sep,W1-Weekday,Period,SepWkAM,6,20836,21721,1.035873,1.06
7659,0015_295.6,AM,M10-Oct,W1-Weekday,Period,OctWkAM,6,20977,21868,1.042862,1.06


# VC Groups as Columns

In [107]:
df_VCGroups

,SeasonGroup,SeasonType,DOWGroup,HourPercentOf,Period,VCGroupCode
VCGroupID,,,,,,
0,S00-Avg,Season,W1-Weekday,Period,AM,AvgWkAM
1,S01-Win,Season,W1-Weekday,Period,AM,WinWkAM
2,S02-Spr,Season,W1-Weekday,Period,AM,SprWkAM
3,S03-Sum,Season,W1-Weekday,Period,AM,SumWkAM
4,S04-Fal,Season,W1-Weekday,Period,AM,FalWkAM
5,M01-Jan,Month,W1-Weekday,Period,AM,JanWkAM
6,M02-Feb,Month,W1-Weekday,Period,AM,FebWkAM
7,M03-Mar,Month,W1-Weekday,Period,AM,MarWkAM
8,M04-Apr,Month,W1-Weekday,Period,AM,AprWkAM


In [108]:
dfs_Peak_VC_MaxDirFG_wVCGroups = []

for df_Peak_VC_MaxDirFG in dfs_Peak_VC_MaxDirFG:

    df_Peak_VC_MaxDirFG_wVCGroups = df_Peak_VC_MaxDirFG[['VCGroupCode','SEGID','VCPeak15MinPCE']]
    
    df_Peak_VC_MaxDirFG_wVCGroups = df_Peak_VC_MaxDirFG_wVCGroups.round({'VCPeak15MinPCE': 2})
    
    df_Peak_VC_MaxDirFG_wVCGroups = df_Peak_VC_MaxDirFG_wVCGroups.pivot(index='SEGID', columns="VCGroupCode",values='VCPeak15MinPCE')
    
    df_Peak_VC_MaxDirFG_wVCGroups = df_Peak_VC_MaxDirFG_wVCGroups.reset_index()
    
    dfs_Peak_VC_MaxDirFG_wVCGroups.append(df_Peak_VC_MaxDirFG_wVCGroups)

    display(df_Peak_VC_MaxDirFG_wVCGroups)
    

VCGroupCode,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0006_149.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0006_150.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0006_152.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0006_152.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3613,WFRC_8208,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,

VCGroupCode,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0006_149.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0006_150.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0006_152.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0006_152.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3659,WFRC_8213,0.08,0.06,0.04,0.04,0.05,0.07,0.08,0.06,0.04,0.04,0.05,0.07,0.08,0.06,0.04,0.04,0.05,0.07,0.08,0.06,0.04,0.04,0.05,0.07,

VCGroupCode,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0006_149.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0006_150.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0006_152.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0006_152.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3744,WFRC_8215,0.30,0.20,0.16,0.25,0.19,0.32,0.31,0.19,0.15,0.25,0.19,0.33,0.30,0.20,0.16,0.24,0.18,0.31,0.29,0.20,0.15,0.23,0.18,0.30,

VCGroupCode,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0006_149.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0006_150.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0006_152.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0006_152.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,WFRC_8215,0.42,0.28,0.22,0.36,0.25,0.45,0.42,0.27,0.21,0.36,0.26,0.46,0.42,0.27,0.22,0.35,0.25,0.44,0.40,0.27,0.21,0.34,0.25,0.42,

VCGroupCode,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.16,0.14,0.14,0.10,0.07,0.12,0.15,0.15,0.14,0.10,0.08,0.12,0.14,0.14,0.13,0.09,0.07,0.11,0.12,0.12,0.12,0.08,0.06,0.10,0.15,0.14,0.14,0.10,0.07,0.12,0.12,0.12,0.12,0.08,0.06,0.10,0.12,0.12,0.11,0.08,0.06,0.09,0.14,0.14,0.14,0.10,0.08,0.12,0.14,0.15,0.14,0.10,0.08,0.12,0.14,0.13,0.13,0.09,0.07,0.11,0.15,0.15,0.14,0.10,0.08,0.12,0.15,0.14,0.13,0.09,0.07,0.11,0.16,0.14,0.15,0.10,0.08,0.12,0.15,0.14,0.14,0.10,0.07,0.12,0.15,0.14,0.14,0.10,0.07,0.11,0.14,0.14,0.14,0.10,0.08,0.12,0.12,0.12,0.12,0.08,0.06,0.10
1,0006_149.9,0.29,0.27,0.26,0.20,0.15,0.27,0.34,0.32,0.30,0.24,0.17,0.31,0.30,0.29,0.27,0.21,0.15,0.27,0.28,0.26,0.24,0.18,0.13,0.24,0.31,0.29,0.27,0.21,0.15,0.27,0.27,0.25,0.23,0.17,0.13,0.23,0.25,0.24,0.22,0.16,0.12,0.21,0.35,0.33,0.31,0.25,0.18,0.33,0.34,0.32,0.30,0.24,0.17,0.32,0.28,0.27,0.26,0.19,0.14,0.25,0.32,0.30,0.28,0.22,0.16,0.29,0.28,0.26,0.25,0.18,0.13,0.24,0.32,0.30,0.29,0.21,0.16,0.28,0.33,0.30,0.28,0.22,0.16,0.29,0.30,0.28,0.27,0.21,0.15,0.27,0.34,0.32,0.31,0.24,0.18,0.32,0.26,0.25,0.23,0.17,0.12,0.23
2,0006_150.6,0.29,0.27,0.26,0.19,0.15,0.26,0.34,0.32,0.30,0.22,0.17,0.30,0.30,0.29,0.27,0.20,0.15,0.27,0.28,0.26,0.24,0.17,0.13,0.23,0.31,0.29,0.27,0.20,0.15,0.26,0.27,0.25,0.23,0.16,0.13,0.22,0.25,0.24,0.22,0.15,0.12,0.20,0.35,0.33,0.31,0.23,0.18,0.32,0.34,0.32,0.30,0.23,0.18,0.31,0.28,0.27,0.26,0.18,0.14,0.24,0.32,0.30,0.28,0.21,0.16,0.28,0.28,0.26,0.25,0.17,0.14,0.24,0.32,0.30,0.29,0.20,0.16,0.28,0.33,0.30,0.28,0.21,0.16,0.28,0.30,0.28,0.27,0.20,0.15,0.26,0.34,0.32,0.31,0.23,0.18,0.31,0.26,0.25,0.23,0.16,0.13,0.22
3,0006_152.6,0.32,0.30,0.29,0.20,0.16,0.24,0.38,0.35,0.33,0.23,0.19,0.28,0.33,0.31,0.30,0.21,0.17,0.25,0.30,0.29,0.27,0.18,0.15,0.22,0.34,0.32,0.30,0.20,0.17,0.25,0.29,0.27,0.26,0.17,0.14,0.20,0.28,0.26,0.24,0.16,0.13,0.19,0.38,0.37,0.34,0.24,0.20,0.30,0.37,0.35,0.33,0.24,0.20,0.29,0.31,0.30,0.28,0.19,0.16,0.23,0.35,0.32,0.30,0.22,0.18,0.26,0.30,0.29,0.27,0.18,0.15,0.22,0.35,0.33,0.32,0.21,0.18,0.26,0.36,0.33,0.31,0.22,0.18,0.26,0.33,0.31,0.29,0.20,0.17,0.24,0.38,0.36,0.34,0.24,0.20,0.29,0.29,0.27,0.25,0.17,0.14,0.20
4,0006_152.9,0.47,0.31,0.24,0.40,0.29,0.45,0.48,0.30,0.24,0.40,0.29,0.47,0.47,0.31,0.24,0.39,0.29,0.45,0.45,0.31,0.24,0.38,0.28,0.43,0.48,0.32,0.25,0.40,0.30,0.45,0.46,0.31,0.25,0.38,0.28,0.43,0.43,0.29,0.23,0.35,0.26,0.40,0.46,0.29,0.23,0.38,0.28,0.46,0.47,0.30,0.24,0.40,0.29,0.47,0.47,0.31,0.25,0.40,0.29,0.45,0.47,0.31,0.25,0.40,0.30,0.46,0.47,0.32,0.24,0.39,0.29,0.44,0.49,0.32,0.25,0.40,0.30,0.46,0.48,0.32,0.25,0.41,0.30,0.47,0.47,0.31,0.25,0.40,0.29,0.45,0.47,0.30,0.24,0.39,0.29,0.47,0.45,0.30,0.24,0.37,0.27,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,WFRC_8215,0.55,0.37,0.29,0.48,0.34,0.58,0.56,0.35,0.28,0.48,0.34,0.60,0.55,0.36,0.28,0.47,0.33,0.58,0.53,0.36,0.28,0.45,0.33,0.55,

...

VCGroupCode,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0006_149.9,0.07,0.06,0.06,0.05,0.03,0.07,0.06,0.06,0.06,0.06,0.03,0.08,0.06,0.06,0.06,0.05,0.03,0.07,0.05,0.05,0.05,0.05,0.03,0.06,0.06,0.06,0.06,0.05,0.03,0.07,0.05,0.05,0.05,0.05,0.03,0.06,0.05,0.05,0.05,0.04,0.03,0.06,0.06,0.06,0.06,0.06,0.03,0.08,0.06,0.06,0.06,0.06,0.03,0.08,0.06,0.06,0.06,0.05,0.03,0.07,0.07,0.07,0.06,0.06,0.03,0.08,0.06,0.06,0.06,0.05,0.03,0.07,0.07,0.06,0.06,0.06,0.03,0.08,0.06,0.06,0.06,0.06,0.03,0.08,0.06,0.06,0.06,0.05,0.03,0.07,0.06,0.06,0.06,0.06,0.03,0.08,0.05,0.05,0.05,0.05,0.03,0.06
2,0006_150.6,0.07,0.06,0.06,0.06,0.03,0.07,0.06,0.06,0.06,0.06,0.04,0.08,0.06,0.06,0.06,0.05,0.03,0.07,0.06,0.06,0.05,0.05,0.03,0.06,0.07,0.06,0.06,0.06,0.03,0.07,0.05,0.05,0.05,0.05,0.03,0.06,0.05,0.05,0.05,0.04,0.03,0.06,0.06,0.06,0.06,0.06,0.04,0.08,0.06,0.07,0.06,0.06,0.04,0.08,0.06,0.06,0.06,0.05,0.03,0.07,0.07,0.07,0.06,0.06,0.04,0.08,0.07,0.06,0.06,0.05,0.03,0.07,0.07,0.06,0.07,0.06,0.04,0.08,0.07,0.06,0.06,0.06,0.03,0.08,0.07,0.06,0.06,0.06,0.03,0.07,0.06,0.06,0.06,0.06,0.04,0.08,0.05,0.05,0.05,0.05,0.03,0.06
3,0006_152.6,0.09,0.09,0.09,0.08,0.04,0.10,0.09,0.09,0.08,0.08,0.05,0.10,0.08,0.08,0.08,0.07,0.04,0.09,0.08,0.08,0.07,0.07,0.04,0.08,0.09,0.08,0.08,0.08,0.05,0.10,0.07,0.07,0.07,0.06,0.04,0.08,0.07,0.07,0.07,0.06,0.04,0.08,0.08,0.08,0.08,0.08,0.05,0.10,0.09,0.09,0.08,0.08,0.05,0.10,0.08,0.08,0.08,0.07,0.04,0.09,0.09,0.09,0.08,0.08,0.05,0.10,0.09,0.08,0.08,0.07,0.04,0.09,0.09,0.09,0.09,0.08,0.05,0.10,0.09,0.09,0.08,0.08,0.05,0.10,0.09,0.08,0.08,0.08,0.04,0.09,0.09,0.09,0.08,0.08,0.05,0.10,0.07,0.07,0.07,0.06,0.04,0.08
4,0006_152.9,0.10,0.08,0.07,0.09,0.05,0.11,0.10,0.08,0.07,0.09,0.05,0.12,0.09,0.07,0.06,0.09,0.05,0.11,0.09,0.07,0.06,0.08,0.05,0.10,0.10,0.08,0.07,0.09,0.05,0.11,0.08,0.07,0.06,0.08,0.04,0.10,0.08,0.07,0.06,0.07,0.04,0.09,0.09,0.08,0.06,0.09,0.05,0.11,0.10,0.08,0.07,0.09,0.05,0.12,0.09,0.07,0.07,0.08,0.05,0.11,0.10,0.08,0.07,0.09,0.05,0.11,0.09,0.08,0.07,0.08,0.05,0.10,0.10,0.08,0.07,0.09,0.05,0.11,0.10,0.08,0.07,0.09,0.05,0.11,0.10,0.08,0.07,0.09,0.05,0.11,0.10,0.08,0.07,0.09,0.05,0.12,0.08,0.07,0.06,0.08,0.04,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3656,WFRC_8213,0.09,0.07,0.05,0.05,0.06,0.10,0.09,0.06,0.05,0.04,0.06,0.10,0.09,0.07,0.05,0.04,0.06,0.10,0.09,0.07,0.05,0.04,0.06,0.09,

VCGroupCode,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.16,0.14,0.14,0.09,0.07,0.11,0.14,0.14,0.14,0.09,0.08,0.11,0.14,0.14,0.13,0.08,0.07,0.11,0.12,0.12,0.12,0.07,0.06,0.09,0.15,0.14,0.14,0.09,0.07,0.11,0.12,0.12,0.12,0.07,0.06,0.09,0.12,0.11,0.11,0.07,0.06,0.09,0.14,0.14,0.13,0.09,0.08,0.11,0.14,0.15,0.14,0.09,0.08,0.12,0.14,0.13,0.13,0.08,0.07,0.10,0.15,0.15,0.14,0.09,0.08,0.11,0.14,0.14,0.13,0.08,0.07,0.10,0.16,0.14,0.15,0.09,0.08,0.11,0.15,0.14,0.14,0.09,0.08,0.11,0.15,0.14,0.14,0.09,0.07,0.11,0.14,0.14,0.14,0.09,0.08,0.11,0.12,0.12,0.12,0.07,0.06,0.09
1,0006_149.9,0.36,0.34,0.33,0.33,0.18,0.38,0.43,0.40,0.38,0.38,0.21,0.45,0.38,0.36,0.34,0.33,0.19,0.39,0.34,0.33,0.31,0.30,0.16,0.34,0.38,0.36,0.34,0.33,0.19,0.39,0.33,0.31,0.29,0.28,0.15,0.32,0.32,0.30,0.27,0.26,0.14,0.30,0.44,0.42,0.39,0.40,0.22,0.47,0.43,0.40,0.38,0.39,0.22,0.46,0.36,0.34,0.32,0.31,0.17,0.36,0.40,0.37,0.35,0.35,0.19,0.41,0.35,0.33,0.31,0.30,0.17,0.35,0.40,0.37,0.36,0.34,0.19,0.41,0.41,0.38,0.35,0.35,0.20,0.42,0.37,0.35,0.33,0.33,0.18,0.38,0.43,0.41,0.38,0.39,0.22,0.46,0.33,0.31,0.29,0.28,0.15,0.32
2,0006_150.6,0.37,0.34,0.33,0.33,0.18,0.38,0.43,0.40,0.38,0.38,0.22,0.45,0.38,0.36,0.34,0.33,0.19,0.39,0.35,0.33,0.31,0.29,0.16,0.34,0.39,0.36,0.34,0.33,0.19,0.39,0.33,0.31,0.29,0.28,0.16,0.32,0.32,0.30,0.28,0.26,0.15,0.30,0.44,0.42,0.39,0.40,0.23,0.47,0.43,0.41,0.38,0.39,0.22,0.46,0.36,0.34,0.33,0.31,0.17,0.36,0.40,0.37,0.35,0.35,0.20,0.41,0.35,0.33,0.31,0.30,0.17,0.35,0.40,0.38,0.37,0.34,0.20,0.41,0.41,0.38,0.35,0.35,0.20,0.41,0.37,0.35,0.34,0.33,0.18,0.38,0.43,0.41,0.39,0.39,0.22,0.46,0.33,0.32,0.29,0.28,0.16,0.32
3,0006_152.6,0.43,0.40,0.39,0.39,0.21,0.44,0.51,0.47,0.44,0.45,0.25,0.52,0.45,0.42,0.40,0.39,0.22,0.45,0.41,0.38,0.36,0.35,0.19,0.39,0.45,0.42,0.40,0.39,0.22,0.45,0.39,0.37,0.34,0.33,0.18,0.37,0.37,0.35,0.32,0.31,0.17,0.35,0.51,0.49,0.46,0.47,0.26,0.54,0.50,0.48,0.45,0.46,0.26,0.53,0.42,0.40,0.38,0.37,0.20,0.41,0.47,0.44,0.41,0.42,0.23,0.47,0.41,0.39,0.37,0.35,0.20,0.40,0.47,0.44,0.43,0.41,0.23,0.47,0.48,0.45,0.41,0.42,0.23,0.48,0.44,0.41,0.39,0.39,0.22,0.44,0.51,0.48,0.45,0.46,0.26,0.53,0.39,0.37,0.34,0.33,0.18,0.37
4,0006_152.9,0.59,0.39,0.31,0.64,0.35,0.73,0.60,0.38,0.30,0.64,0.36,0.76,0.59,0.39,0.30,0.63,0.35,0.72,0.57,0.38,0.30,0.60,0.34,0.68,0.60,0.40,0.31,0.64,0.36,0.73,0.58,0.39,0.31,0.60,0.34,0.69,0.55,0.37,0.28,0.56,0.32,0.64,0.57,0.37,0.29,0.62,0.34,0.74,0.59,0.38,0.30,0.64,0.36,0.76,0.60,0.40,0.31,0.64,0.35,0.72,0.59,0.39,0.31,0.64,0.36,0.74,0.59,0.40,0.31,0.62,0.35,0.70,0.61,0.41,0.32,0.65,0.37,0.74,0.61,0.40,0.31,0.66,0.37,0.75,0.59,0.39,0.31,0.64,0.36,0.73,0.59,0.38,0.30,0.63,0.35,0.75,0.57,0.38,0.30,0.59,0.33,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3656,WFRC_8213,0.13,0.09,0.07,0.07,0.08,0.15,0.12,0.09,0.06,0.07,0.08,0.15,0.12,0.09,0.07,0.07,0.08,0.15,0.12,0.09,0.07,0.07,0.08,0.14,

VCGroupCode,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0006_149.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0006_150.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0006_152.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0006_152.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3744,WFRC_8215,0.27,0.18,0.14,0.21,0.17,0.27,0.27,0.17,0.14,0.21,0.17,0.29,0.27,0.18,0.14,0.21,0.16,0.27,0.26,0.17,0.14,0.20,0.16,0.26,

VCGroupCode,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0006_149.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0006_150.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0006_152.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0006_152.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,WFRC_8215,0.41,0.27,0.21,0.34,0.25,0.42,0.41,0.26,0.21,0.34,0.25,0.44,0.41,0.27,0.21,0.33,0.25,0.42,0.39,0.27,0.21,0.32,0.24,0.40,

VCGroupCode,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.16,0.14,0.14,0.10,0.07,0.12,0.15,0.15,0.14,0.10,0.08,0.12,0.14,0.14,0.13,0.10,0.07,0.11,0.12,0.12,0.12,0.09,0.06,0.10,0.15,0.14,0.14,0.10,0.07,0.12,0.12,0.12,0.12,0.08,0.06,0.10,0.12,0.12,0.11,0.08,0.06,0.09,0.14,0.14,0.14,0.10,0.08,0.12,0.14,0.15,0.14,0.10,0.08,0.12,0.14,0.13,0.13,0.09,0.07,0.11,0.15,0.15,0.14,0.10,0.08,0.12,0.15,0.14,0.13,0.09,0.07,0.11,0.16,0.14,0.15,0.10,0.08,0.12,0.15,0.14,0.14,0.10,0.07,0.12,0.15,0.14,0.14,0.10,0.07,0.11,0.14,0.14,0.14,0.10,0.08,0.12,0.12,0.12,0.12,0.08,0.06,0.10
1,0006_149.9,0.29,0.27,0.26,0.21,0.15,0.27,0.34,0.32,0.30,0.24,0.17,0.31,0.31,0.29,0.27,0.21,0.15,0.27,0.28,0.26,0.24,0.19,0.13,0.24,0.31,0.29,0.27,0.21,0.15,0.27,0.27,0.25,0.23,0.17,0.13,0.22,0.25,0.24,0.22,0.16,0.12,0.21,0.35,0.33,0.31,0.25,0.18,0.33,0.34,0.32,0.30,0.24,0.18,0.32,0.29,0.27,0.26,0.20,0.14,0.25,0.32,0.30,0.28,0.22,0.16,0.29,0.28,0.26,0.25,0.19,0.14,0.24,0.32,0.30,0.29,0.22,0.16,0.28,0.33,0.30,0.28,0.22,0.16,0.29,0.30,0.28,0.27,0.21,0.15,0.27,0.35,0.33,0.31,0.24,0.18,0.32,0.27,0.25,0.23,0.17,0.13,0.22
2,0006_150.6,0.29,0.27,0.26,0.20,0.15,0.26,0.34,0.32,0.30,0.23,0.17,0.30,0.31,0.29,0.27,0.20,0.15,0.26,0.28,0.26,0.24,0.18,0.13,0.23,0.31,0.29,0.27,0.20,0.15,0.26,0.27,0.25,0.23,0.17,0.13,0.22,0.25,0.24,0.22,0.16,0.12,0.20,0.35,0.33,0.31,0.24,0.18,0.32,0.34,0.32,0.30,0.23,0.18,0.31,0.29,0.27,0.26,0.19,0.14,0.24,0.32,0.30,0.28,0.21,0.16,0.28,0.28,0.26,0.25,0.18,0.14,0.24,0.32,0.30,0.29,0.21,0.16,0.27,0.33,0.30,0.28,0.21,0.16,0.28,0.30,0.28,0.27,0.20,0.15,0.26,0.35,0.32,0.31,0.23,0.18,0.31,0.27,0.25,0.23,0.17,0.13,0.22
3,0006_152.6,0.32,0.30,0.29,0.20,0.16,0.24,0.38,0.35,0.33,0.23,0.19,0.28,0.33,0.31,0.30,0.20,0.17,0.24,0.30,0.29,0.27,0.18,0.15,0.21,0.34,0.32,0.30,0.20,0.17,0.24,0.29,0.27,0.26,0.17,0.14,0.20,0.28,0.26,0.24,0.16,0.13,0.19,0.38,0.37,0.34,0.24,0.20,0.29,0.37,0.35,0.33,0.24,0.20,0.29,0.31,0.30,0.28,0.19,0.16,0.23,0.35,0.32,0.30,0.22,0.18,0.26,0.30,0.29,0.27,0.18,0.15,0.22,0.35,0.33,0.32,0.21,0.18,0.25,0.36,0.33,0.31,0.22,0.18,0.26,0.33,0.31,0.29,0.20,0.17,0.24,0.38,0.36,0.34,0.24,0.20,0.29,0.29,0.27,0.25,0.17,0.14,0.20
4,0006_152.9,0.47,0.31,0.25,0.39,0.29,0.45,0.48,0.30,0.24,0.39,0.30,0.47,0.47,0.31,0.24,0.39,0.29,0.45,0.45,0.31,0.24,0.37,0.28,0.43,0.48,0.32,0.25,0.40,0.30,0.46,0.46,0.31,0.25,0.37,0.28,0.43,0.43,0.29,0.23,0.35,0.26,0.40,0.46,0.29,0.23,0.38,0.28,0.46,0.47,0.30,0.24,0.39,0.29,0.47,0.47,0.31,0.25,0.39,0.29,0.45,0.47,0.31,0.25,0.40,0.30,0.46,0.47,0.32,0.24,0.38,0.29,0.44,0.49,0.32,0.25,0.40,0.30,0.46,0.48,0.32,0.25,0.41,0.30,0.47,0.47,0.31,0.25,0.40,0.29,0.46,0.47,0.30,0.24,0.39,0.29,0.47,0.45,0.30,0.24,0.37,0.27,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,WFRC_8215,0.53,0.35,0.28,0.44,0.33,0.55,0.54,0.34,0.27,0.44,0.33,0.57,0.53,0.35,0.27,0.44,0.32,0.55,0.51,0.35,0.27,0.42,0.32,0.52,

In [109]:
df_Peak_VC_MaxDirFG_wVCGroups

VCGroupCode,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.16,0.14,0.14,0.10,0.07,0.12,0.15,0.15,0.14,0.10,0.08,0.12,0.14,0.14,0.13,0.10,0.07,0.11,0.12,0.12,0.12,0.09,0.06,0.10,0.15,0.14,0.14,0.10,0.07,0.12,0.12,0.12,0.12,0.08,0.06,0.10,0.12,0.12,0.11,0.08,0.06,0.09,0.14,0.14,0.14,0.10,0.08,0.12,0.14,0.15,0.14,0.10,0.08,0.12,0.14,0.13,0.13,0.09,0.07,0.11,0.15,0.15,0.14,0.10,0.08,0.12,0.15,0.14,0.13,0.09,0.07,0.11,0.16,0.14,0.15,0.10,0.08,0.12,0.15,0.14,0.14,0.10,0.07,0.12,0.15,0.14,0.14,0.10,0.07,0.11,0.14,0.14,0.14,0.10,0.08,0.12,0.12,0.12,0.12,0.08,0.06,0.10
1,0006_149.9,0.29,0.27,0.26,0.21,0.15,0.27,0.34,0.32,0.30,0.24,0.17,0.31,0.31,0.29,0.27,0.21,0.15,0.27,0.28,0.26,0.24,0.19,0.13,0.24,0.31,0.29,0.27,0.21,0.15,0.27,0.27,0.25,0.23,0.17,0.13,0.22,0.25,0.24,0.22,0.16,0.12,0.21,0.35,0.33,0.31,0.25,0.18,0.33,0.34,0.32,0.30,0.24,0.18,0.32,0.29,0.27,0.26,0.20,0.14,0.25,0.32,0.30,0.28,0.22,0.16,0.29,0.28,0.26,0.25,0.19,0.14,0.24,0.32,0.30,0.29,0.22,0.16,0.28,0.33,0.30,0.28,0.22,0.16,0.29,0.30,0.28,0.27,0.21,0.15,0.27,0.35,0.33,0.31,0.24,0.18,0.32,0.27,0.25,0.23,0.17,0.13,0.22
2,0006_150.6,0.29,0.27,0.26,0.20,0.15,0.26,0.34,0.32,0.30,0.23,0.17,0.30,0.31,0.29,0.27,0.20,0.15,0.26,0.28,0.26,0.24,0.18,0.13,0.23,0.31,0.29,0.27,0.20,0.15,0.26,0.27,0.25,0.23,0.17,0.13,0.22,0.25,0.24,0.22,0.16,0.12,0.20,0.35,0.33,0.31,0.24,0.18,0.32,0.34,0.32,0.30,0.23,0.18,0.31,0.29,0.27,0.26,0.19,0.14,0.24,0.32,0.30,0.28,0.21,0.16,0.28,0.28,0.26,0.25,0.18,0.14,0.24,0.32,0.30,0.29,0.21,0.16,0.27,0.33,0.30,0.28,0.21,0.16,0.28,0.30,0.28,0.27,0.20,0.15,0.26,0.35,0.32,0.31,0.23,0.18,0.31,0.27,0.25,0.23,0.17,0.13,0.22
3,0006_152.6,0.32,0.30,0.29,0.20,0.16,0.24,0.38,0.35,0.33,0.23,0.19,0.28,0.33,0.31,0.30,0.20,0.17,0.24,0.30,0.29,0.27,0.18,0.15,0.21,0.34,0.32,0.30,0.20,0.17,0.24,0.29,0.27,0.26,0.17,0.14,0.20,0.28,0.26,0.24,0.16,0.13,0.19,0.38,0.37,0.34,0.24,0.20,0.29,0.37,0.35,0.33,0.24,0.20,0.29,0.31,0.30,0.28,0.19,0.16,0.23,0.35,0.32,0.30,0.22,0.18,0.26,0.30,0.29,0.27,0.18,0.15,0.22,0.35,0.33,0.32,0.21,0.18,0.25,0.36,0.33,0.31,0.22,0.18,0.26,0.33,0.31,0.29,0.20,0.17,0.24,0.38,0.36,0.34,0.24,0.20,0.29,0.29,0.27,0.25,0.17,0.14,0.20
4,0006_152.9,0.47,0.31,0.25,0.39,0.29,0.45,0.48,0.30,0.24,0.39,0.30,0.47,0.47,0.31,0.24,0.39,0.29,0.45,0.45,0.31,0.24,0.37,0.28,0.43,0.48,0.32,0.25,0.40,0.30,0.46,0.46,0.31,0.25,0.37,0.28,0.43,0.43,0.29,0.23,0.35,0.26,0.40,0.46,0.29,0.23,0.38,0.28,0.46,0.47,0.30,0.24,0.39,0.29,0.47,0.47,0.31,0.25,0.39,0.29,0.45,0.47,0.31,0.25,0.40,0.30,0.46,0.47,0.32,0.24,0.38,0.29,0.44,0.49,0.32,0.25,0.40,0.30,0.46,0.48,0.32,0.25,0.41,0.30,0.47,0.47,0.31,0.25,0.40,0.29,0.46,0.47,0.30,0.24,0.39,0.29,0.47,0.45,0.30,0.24,0.37,0.27,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,WFRC_8215,0.53,0.35,0.28,0.44,0.33,0.55,0.54,0.34,0.27,0.44,0.33,0.57,0.53,0.35,0.27,0.44,0.32,0.55,0.51,0.35,0.27,0.42,0.32,0.52,

# Export final tables to CSV

In [12]:
if export_results: 
    df_Scenarios.to_csv(os.path.join(dirResults, r'scenarios.csv'),index=False)
    df_VCGroup_DOWPeak.to_csv(os.path.join(dirResults, r'vcgroups_dowpeak.csv'),index=False)
    df_VCGroup_Seasons.to_csv(os.path.join(dirResults, r'vcgroups_seasons.csv'),index=False)
    df_VCGroups.to_csv(os.path.join(dirResults, r'vcgroups.csv'),index=False)

In [111]:
if export_results: 

    df_Scenarios_toExport = df_Scenarios_toProcess.reset_index()
    
    for index, df_Peak_VC_MaxDirFG_wVCGroups in enumerate(dfs_Peak_VC_MaxDirFG_wVCGroups):    

        filename = os.path.join(dirResults,"3_Peak_VC_MaxDirFG_wVCGroups_" + df_Scenarios_toExport['ScenarioCode'].iloc[index] + '.csv')
        display('Exporting ' + filename)
        df_Peak_VC_MaxDirFG_wVCGroups.to_csv(filename, index=False)

    for index, df_Peak_VC_MaxDirFG in enumerate(dfs_Peak_VC_MaxDirFG):    

        filename = os.path.join(dirResults,"2_Peak_VC_MaxDirFG_" + df_Scenarios_toExport['ScenarioCode'].iloc[index] + '.csv')
        #display('Exporting ' + filename)
        df_Peak_VC_MaxDirFG.to_csv(filename, index=False)

    for index, df_Peak_VC in enumerate(dfs_Peak_VC):    

        filename = os.path.join(dirResults,"1_Peak_VC_" + df_Scenarios_toExport['ScenarioCode'].iloc[index] + '.csv')
        #display('Exporting ' + filename)
        df_Peak_VC.to_csv(filename, index=False)

'Exporting E:\\GitHub\\V-over-C-Calculations\\results\\3_Peak_VC_MaxDirFG_wVCGroups_Base19.csv'

'Exporting E:\\GitHub\\V-over-C-Calculations\\results\\3_Peak_VC_MaxDirFG_wVCGroups_TIP24.csv'

'Exporting E:\\GitHub\\V-over-C-Calculations\\results\\3_Peak_VC_MaxDirFG_wVCGroups_RTP30.csv'

'Exporting E:\\GitHub\\V-over-C-Calculations\\results\\3_Peak_VC_MaxDirFG_wVCGroups_RTP40.csv'

'Exporting E:\\GitHub\\V-over-C-Calculations\\results\\3_Peak_VC_MaxDirFG_wVCGroups_RTP50.csv'

...

'Exporting E:\\GitHub\\V-over-C-Calculations\\results\\3_Peak_VC_MaxDirFG_wVCGroups_NB40.csv'

'Exporting E:\\GitHub\\V-over-C-Calculations\\results\\3_Peak_VC_MaxDirFG_wVCGroups_NB50.csv'

'Exporting E:\\GitHub\\V-over-C-Calculations\\results\\3_Peak_VC_MaxDirFG_wVCGroups_UF30.csv'

'Exporting E:\\GitHub\\V-over-C-Calculations\\results\\3_Peak_VC_MaxDirFG_wVCGroups_UF40.csv'

'Exporting E:\\GitHub\\V-over-C-Calculations\\results\\3_Peak_VC_MaxDirFG_wVCGroups_UF50.csv'

In [112]:
dfs_Peak_VC_MaxDirFG_wVCGroups[0]

VCGroupCode,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0006_149.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0006_150.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0006_152.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0006_152.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3613,WFRC_8208,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,

In [113]:
df_Scenarios_toProcess

,Process,ScenarioCode,ScenarioName,ScenarioYear,SegSummaryDBF,AdjScenarioCode
ScenarioID,,,,,,
0,Yes,Base19,2019 Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed_BH,NaN
1,Yes,TIP24,2024 TIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed_BH,NaN
2,Yes,RTP30,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed_BH,NaN
3,Yes,RTP40,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed_BH,NaN
4,Yes,RTP50,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed_BH,NaN
5,Yes,NB30,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,RTP30
6,Yes,NB40,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,RTP40
7,Yes,NB50,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,RTP50
8,Yes,UF30,2030 Needed,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,RTP30


# Exports for WebApp

In [13]:
if export_results:
    
    df_Scenarios_json = df_Scenarios.copy()
    df_Scenarios_json = df_Scenarios_json[['ScenarioCode','ScenarioName']]
    df_Scenarios_json.columns = ('value','label')
    df_Scenarios_json['filename'] = "VC_" + df_Scenarios_json['value'] + '.json'
    
    df_VCGroup_Seasons_json = df_VCGroup_Seasons.copy()
    df_VCGroup_Seasons_json = df_VCGroup_Seasons_json[['SeasonCode','SeasonName','SeasonType']]
    df_VCGroup_Seasons_json.columns = ('value','label','SeasonType')
        
    df_VCGroup_DOWPeak_json = df_VCGroup_DOWPeak.copy()
    df_VCGroup_DOWPeak_json = df_VCGroup_DOWPeak_json[['DOWPkCode','DOWPkName']]
    df_VCGroup_DOWPeak_json.columns = ('value','label')

    df_Scenarios_json.to_json(os.path.join(dirResults, r'WebAppData\scenarios.json'),orient='records')
    df_VCGroup_Seasons_json.to_json(os.path.join(dirResults, r'WebAppData\vcgroups_seasons.json'),orient='records')
    df_VCGroup_DOWPeak_json.to_json(os.path.join(dirResults, r'WebAppData\vcgroups_dowpeaks.json'),orient='records')

In [115]:
if export_results:
    
    df_Scenarios_toExport = df_Scenarios_toProcess.reset_index()
    
    for index, df_Peak_VC_MaxDirFG_wVCGroups in enumerate(dfs_Peak_VC_MaxDirFG_wVCGroups):    

        filename = "WebAppData\VC_" + df_Scenarios_toExport['ScenarioCode'].iloc[index] + '.json'
        #display(filename)

        #df_Peak_VC_MaxDirFG_wVCGroups = df_Peak_VC_MaxDirFG_wVCGroups.rename(columns={"SEGID": 'S'})

        #export to csv in data-processed
        df_Peak_VC_MaxDirFG_wVCGroups.to_json(os.path.join(dirResults, filename),orient='records')

        display('Done exporting ' + filename)

'Done exporting WebAppData\\VC_Base19.json'

'Done exporting WebAppData\\VC_TIP24.json'

'Done exporting WebAppData\\VC_RTP30.json'

'Done exporting WebAppData\\VC_RTP40.json'

'Done exporting WebAppData\\VC_RTP50.json'

...

'Done exporting WebAppData\\VC_NB40.json'

'Done exporting WebAppData\\VC_NB50.json'

'Done exporting WebAppData\\VC_UF30.json'

'Done exporting WebAppData\\VC_UF40.json'

'Done exporting WebAppData\\VC_UF50.json'

# Temp

In [116]:
dfs_Peak_VC_MaxDirFG_wVCGroups =[]
df_TDMsf = []

display ('Scenario Codes:')

for (idx, row) in df_Scenarios.iterrows():    

    print(row.loc['ScenarioCode'])
    
    df_TDM = dfs_TDM[idx]
    df_TDMsf = df_TDM[['SEGID','FT']]
    
    df_TDMsfmax = df_TDMsf.groupby('SEGID', as_index=False).agg({'FT':[np.max]})
    df_TDMsfmax.columns = ['SEGID','FT']
    #df_TDMsf.append(df_TDMsf)
    
    df_Peak_VC_MaxDirFG_wVCGroups = pd.read_csv(os.path.join(dirResults, '3_Peak_VC_MaxDirFG_wVCGroups_' + row.loc['ScenarioCode'] + '.csv'))
    display(df_Peak_VC_MaxDirFG_wVCGroups)
    
    df_new = pd.DataFrame.merge(df_TDMsfmax,df_Peak_VC_MaxDirFG_wVCGroups,on='SEGID')
    display(df_new)
    
    filename = os.path.join(dirResults,r"temp\3_Peak_VC_MaxDirFG_wVCGroups_" + row.loc['ScenarioCode'] + '.csv')
    display('Exporting ' + filename)
    df_new.to_csv(filename,index=False)
    #dfs_Peak_VC_MaxDirFG_wVCGroups.append(dfs_Peak_VC_MaxDirFG_wVCGroups)


'Scenario Codes:'

Base19


,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0006_149.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0006_150.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0006_152.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0006_152.9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3613,WFRC_8208,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0

,SEGID,FT,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0006_149.9,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0006_150.6,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0006_152.6,3.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0006_152.9,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3613,WFRC_8208,5.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,

'Exporting E:\\GitHub\\V-over-C-Calculations\\results\\temp\\3_Peak_VC_MaxDirFG_wVCGroups_Base19.csv'

...

'Exporting E:\\GitHub\\V-over-C-Calculations\\results\\temp\\3_Peak_VC_MaxDirFG_wVCGroups_UF40.csv'

UF50


,SEGID,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,0.16,0.14,0.14,0.10,0.07,0.12,0.15,0.15,0.14,0.10,0.08,0.12,0.14,0.14,0.13,0.10,0.07,0.11,0.12,0.12,0.12,0.09,0.06,0.10,0.15,0.14,0.14,0.10,0.07,0.12,0.12,0.12,0.12,0.08,0.06,0.10,0.12,0.12,0.11,0.08,0.06,0.09,0.14,0.14,0.14,0.10,0.08,0.12,0.14,0.15,0.14,0.10,0.08,0.12,0.14,0.13,0.13,0.09,0.07,0.11,0.15,0.15,0.14,0.10,0.08,0.12,0.15,0.14,0.13,0.09,0.07,0.11,0.16,0.14,0.15,0.10,0.08,0.12,0.15,0.14,0.14,0.10,0.07,0.12,0.15,0.14,0.14,0.10,0.07,0.11,0.14,0.14,0.14,0.10,0.08,0.12,0.12,0.12,0.12,0.08,0.06,0.10
1,0006_149.9,0.29,0.27,0.26,0.21,0.15,0.27,0.34,0.32,0.30,0.24,0.17,0.31,0.31,0.29,0.27,0.21,0.15,0.27,0.28,0.26,0.24,0.19,0.13,0.24,0.31,0.29,0.27,0.21,0.15,0.27,0.27,0.25,0.23,0.17,0.13,0.22,0.25,0.24,0.22,0.16,0.12,0.21,0.35,0.33,0.31,0.25,0.18,0.33,0.34,0.32,0.30,0.24,0.18,0.32,0.29,0.27,0.26,0.20,0.14,0.25,0.32,0.30,0.28,0.22,0.16,0.29,0.28,0.26,0.25,0.19,0.14,0.24,0.32,0.30,0.29,0.22,0.16,0.28,0.33,0.30,0.28,0.22,0.16,0.29,0.30,0.28,0.27,0.21,0.15,0.27,0.35,0.33,0.31,0.24,0.18,0.32,0.27,0.25,0.23,0.17,0.13,0.22
2,0006_150.6,0.29,0.27,0.26,0.20,0.15,0.26,0.34,0.32,0.30,0.23,0.17,0.30,0.31,0.29,0.27,0.20,0.15,0.26,0.28,0.26,0.24,0.18,0.13,0.23,0.31,0.29,0.27,0.20,0.15,0.26,0.27,0.25,0.23,0.17,0.13,0.22,0.25,0.24,0.22,0.16,0.12,0.20,0.35,0.33,0.31,0.24,0.18,0.32,0.34,0.32,0.30,0.23,0.18,0.31,0.29,0.27,0.26,0.19,0.14,0.24,0.32,0.30,0.28,0.21,0.16,0.28,0.28,0.26,0.25,0.18,0.14,0.24,0.32,0.30,0.29,0.21,0.16,0.27,0.33,0.30,0.28,0.21,0.16,0.28,0.30,0.28,0.27,0.20,0.15,0.26,0.35,0.32,0.31,0.23,0.18,0.31,0.27,0.25,0.23,0.17,0.13,0.22
3,0006_152.6,0.32,0.30,0.29,0.20,0.16,0.24,0.38,0.35,0.33,0.23,0.19,0.28,0.33,0.31,0.30,0.20,0.17,0.24,0.30,0.29,0.27,0.18,0.15,0.21,0.34,0.32,0.30,0.20,0.17,0.24,0.29,0.27,0.26,0.17,0.14,0.20,0.28,0.26,0.24,0.16,0.13,0.19,0.38,0.37,0.34,0.24,0.20,0.29,0.37,0.35,0.33,0.24,0.20,0.29,0.31,0.30,0.28,0.19,0.16,0.23,0.35,0.32,0.30,0.22,0.18,0.26,0.30,0.29,0.27,0.18,0.15,0.22,0.35,0.33,0.32,0.21,0.18,0.25,0.36,0.33,0.31,0.22,0.18,0.26,0.33,0.31,0.29,0.20,0.17,0.24,0.38,0.36,0.34,0.24,0.20,0.29,0.29,0.27,0.25,0.17,0.14,0.20
4,0006_152.9,0.47,0.31,0.25,0.39,0.29,0.45,0.48,0.30,0.24,0.39,0.30,0.47,0.47,0.31,0.24,0.39,0.29,0.45,0.45,0.31,0.24,0.37,0.28,0.43,0.48,0.32,0.25,0.40,0.30,0.46,0.46,0.31,0.25,0.37,0.28,0.43,0.43,0.29,0.23,0.35,0.26,0.40,0.46,0.29,0.23,0.38,0.28,0.46,0.47,0.30,0.24,0.39,0.29,0.47,0.47,0.31,0.25,0.39,0.29,0.45,0.47,0.31,0.25,0.40,0.30,0.46,0.47,0.32,0.24,0.38,0.29,0.44,0.49,0.32,0.25,0.40,0.30,0.46,0.48,0.32,0.25,0.41,0.30,0.47,0.47,0.31,0.25,0.40,0.29,0.46,0.47,0.30,0.24,0.39,0.29,0.47,0.45,0.30,0.24,0.37,0.27,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,WFRC_8215,0.53,0.35,0.28,0.44,0.33,0.55,0.54,0.34,0.27,0.44,0.33,0.57,0.53,0.35,0.27,0.44,0.32,0.55,0.51,0.35,0.27,0.42,0.32,0.52,0.54,0.36,0

,SEGID,FT,AprFrPM,AprSaDY,AprSuDY,AprWkAM,AprWkMD,AprWkPM,AugFrPM,AugSaDY,AugSuDY,AugWkAM,AugWkMD,AugWkPM,AvgFrPM,AvgSaDY,AvgSuDY,AvgWkAM,AvgWkMD,AvgWkPM,DecFrPM,DecSaDY,DecSuDY,DecWkAM,DecWkMD,DecWkPM,FalFrPM,FalSaDY,FalSuDY,FalWkAM,FalWkMD,FalWkPM,FebFrPM,FebSaDY,FebSuDY,FebWkAM,FebWkMD,FebWkPM,JanFrPM,JanSaDY,JanSuDY,JanWkAM,JanWkMD,JanWkPM,JulFrPM,JulSaDY,JulSuDY,JulWkAM,JulWkMD,JulWkPM,JunFrPM,JunSaDY,JunSuDY,JunWkAM,JunWkMD,JunWkPM,MarFrPM,MarSaDY,MarSuDY,MarWkAM,MarWkMD,MarWkPM,MayFrPM,MaySaDY,MaySuDY,MayWkAM,MayWkMD,MayWkPM,NovFrPM,NovSaDY,NovSuDY,NovWkAM,NovWkMD,NovWkPM,OctFrPM,OctSaDY,OctSuDY,OctWkAM,OctWkMD,OctWkPM,SepFrPM,SepSaDY,SepSuDY,SepWkAM,SepWkMD,SepWkPM,SprFrPM,SprSaDY,SprSuDY,SprWkAM,SprWkMD,SprWkPM,SumFrPM,SumSaDY,SumSuDY,SumWkAM,SumWkMD,SumWkPM,WinFrPM,WinSaDY,WinSuDY,WinWkAM,WinWkMD,WinWkPM
0,0006_146.9,2.0,0.16,0.14,0.14,0.10,0.07,0.12,0.15,0.15,0.14,0.10,0.08,0.12,0.14,0.14,0.13,0.10,0.07,0.11,0.12,0.12,0.12,0.09,0.06,0.10,0.15,0.14,0.14,0.10,0.07,0.12,0.12,0.12,0.12,0.08,0.06,0.10,0.12,0.12,0.11,0.08,0.06,0.09,0.14,0.14,0.14,0.10,0.08,0.12,0.14,0.15,0.14,0.10,0.08,0.12,0.14,0.13,0.13,0.09,0.07,0.11,0.15,0.15,0.14,0.10,0.08,0.12,0.15,0.14,0.13,0.09,0.07,0.11,0.16,0.14,0.15,0.10,0.08,0.12,0.15,0.14,0.14,0.10,0.07,0.12,0.15,0.14,0.14,0.10,0.07,0.11,0.14,0.14,0.14,0.10,0.08,0.12,0.12,0.12,0.12,0.08,0.06,0.10
1,0006_149.9,2.0,0.29,0.27,0.26,0.21,0.15,0.27,0.34,0.32,0.30,0.24,0.17,0.31,0.31,0.29,0.27,0.21,0.15,0.27,0.28,0.26,0.24,0.19,0.13,0.24,0.31,0.29,0.27,0.21,0.15,0.27,0.27,0.25,0.23,0.17,0.13,0.22,0.25,0.24,0.22,0.16,0.12,0.21,0.35,0.33,0.31,0.25,0.18,0.33,0.34,0.32,0.30,0.24,0.18,0.32,0.29,0.27,0.26,0.20,0.14,0.25,0.32,0.30,0.28,0.22,0.16,0.29,0.28,0.26,0.25,0.19,0.14,0.24,0.32,0.30,0.29,0.22,0.16,0.28,0.33,0.30,0.28,0.22,0.16,0.29,0.30,0.28,0.27,0.21,0.15,0.27,0.35,0.33,0.31,0.24,0.18,0.32,0.27,0.25,0.23,0.17,0.13,0.22
2,0006_150.6,2.0,0.29,0.27,0.26,0.20,0.15,0.26,0.34,0.32,0.30,0.23,0.17,0.30,0.31,0.29,0.27,0.20,0.15,0.26,0.28,0.26,0.24,0.18,0.13,0.23,0.31,0.29,0.27,0.20,0.15,0.26,0.27,0.25,0.23,0.17,0.13,0.22,0.25,0.24,0.22,0.16,0.12,0.20,0.35,0.33,0.31,0.24,0.18,0.32,0.34,0.32,0.30,0.23,0.18,0.31,0.29,0.27,0.26,0.19,0.14,0.24,0.32,0.30,0.28,0.21,0.16,0.28,0.28,0.26,0.25,0.18,0.14,0.24,0.32,0.30,0.29,0.21,0.16,0.27,0.33,0.30,0.28,0.21,0.16,0.28,0.30,0.28,0.27,0.20,0.15,0.26,0.35,0.32,0.31,0.23,0.18,0.31,0.27,0.25,0.23,0.17,0.13,0.22
3,0006_152.6,3.0,0.32,0.30,0.29,0.20,0.16,0.24,0.38,0.35,0.33,0.23,0.19,0.28,0.33,0.31,0.30,0.20,0.17,0.24,0.30,0.29,0.27,0.18,0.15,0.21,0.34,0.32,0.30,0.20,0.17,0.24,0.29,0.27,0.26,0.17,0.14,0.20,0.28,0.26,0.24,0.16,0.13,0.19,0.38,0.37,0.34,0.24,0.20,0.29,0.37,0.35,0.33,0.24,0.20,0.29,0.31,0.30,0.28,0.19,0.16,0.23,0.35,0.32,0.30,0.22,0.18,0.26,0.30,0.29,0.27,0.18,0.15,0.22,0.35,0.33,0.32,0.21,0.18,0.25,0.36,0.33,0.31,0.22,0.18,0.26,0.33,0.31,0.29,0.20,0.17,0.24,0.38,0.36,0.34,0.24,0.20,0.29,0.29,0.27,0.25,0.17,0.14,0.20
4,0006_152.9,2.0,0.47,0.31,0.25,0.39,0.29,0.45,0.48,0.30,0.24,0.39,0.30,0.47,0.47,0.31,0.24,0.39,0.29,0.45,0.45,0.31,0.24,0.37,0.28,0.43,0.48,0.32,0.25,0.40,0.30,0.46,0.46,0.31,0.25,0.37,0.28,0.43,0.43,0.29,0.23,0.35,0.26,0.40,0.46,0.29,0.23,0.38,0.28,0.46,0.47,0.30,0.24,0.39,0.29,0.47,0.47,0.31,0.25,0.39,0.29,0.45,0.47,0.31,0.25,0.40,0.30,0.46,0.47,0.32,0.24,0.38,0.29,0.44,0.49,0.32,0.25,0.40,0.30,0.46,0.48,0.32,0.25,0.41,0.30,0.47,0.47,0.31,0.25,0.40,0.29,0.46,0.47,0.30,0.24,0.39,0.29,0.47,0.45,0.30,0.24,0.37,0.27,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,WFRC_8215,4.0,0.53,0.35,0.28,0.44,0.33,0.55,0.54,0.34,0.27,0.44,0.33,0.57,0.53,0.35,0.27,0.44,0.32,0.55,0.51,0.35,

'Exporting E:\\GitHub\\V-over-C-Calculations\\results\\temp\\3_Peak_VC_MaxDirFG_wVCGroups_UF50.csv'

In [117]:
df_TDMsfmax = df_TDMsf.groupby('SEGID', as_index=False).agg({'FT':[np.max]})
df_TDMsfmax.columns = ['SEGID','FT']
df_TDMsfmax

,SEGID,FT
0,0006_146.9,2.0
1,0006_149.9,2.0
2,0006_150.6,2.0
3,0006_152.6,3.0
4,0006_152.9,2.0
...,...,...
3925,WFRC_8215,4.0
3926,WFRC_8216,33.0
3927,WFRC_8217,4.0
3928,WFRC_8218,31.0


In [118]:
df_VCGroup_Seasons

,SeasonCode,SeasonGroup,SeasonName,SeasonType
0,Avg,S00-Avg,Average,Season
1,Win,S01-Win,Winter,Season
2,Spr,S02-Spr,Spring,Season
3,Sum,S03-Sum,Summer,Season
4,Fal,S04-Fal,Fall,Season
5,Jan,M01-Jan,January,Month
6,Feb,M02-Feb,February,Month
7,Mar,M03-Mar,March,Month
8,Apr,M04-Apr,April,Month
9,May,M05-May,May,Month
